In [1]:
%env CUDA_VISIBLE_DEVICES=0

device='cuda'

env: CUDA_VISIBLE_DEVICES=0


In [2]:
import os
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn import preprocessing, metrics
from sklearn.ensemble import RandomForestClassifier
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import to_categorical
from copy import deepcopy
from tqdm import tqdm

from mtcnn import MTCNN

from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential, load_model, model_from_json
from tensorflow.keras.optimizers import Adam, SGD

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
from torchvision import datasets, transforms
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, classification_report, balanced_accuracy_score, matthews_corrcoef

print(f"Torch: {torch.__version__}")

2024-08-11 21:37:22.195689: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-11 21:37:22.233428: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-11 21:37:22.233452: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-11 21:37:22.233480: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-11 21:37:22.240628: I tensorflow/core/platform/cpu_feature_g

Torch: 2.1.0+cu121


In [3]:
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Dropout, Dense, Activation, Concatenate, Reshape
from tensorflow.keras.layers import Flatten, RepeatVector, Permute, TimeDistributed
from tensorflow.keras.layers import Multiply, Lambda, Softmax

In [4]:
DATA_DIR = '/media/deelvin_disk/echuraev/Workspace/HSE/engagement/'
BATCH_SIZE = 128
TABLE_NAME = None

# Helpers

In [5]:
import pickle

def save_features(file_name, features):
    if os.path.isfile(file_name):
        print("Error! Cannot save features because file already exists")
        return
    with open(file_name, 'wb') as f:
        pickle.dump(features, f) # , protocol=pickle.HIGHEST_PROTOCOL)
    
def load_features(file_name):
    with open(file_name, 'rb') as f:
        return pickle.load(f)

def save_weights(model, file_name):
    if os.path.isfile(file_name):
        print("Error! Cannot save features because file already exists")
        return
    model.save_weights(file_name)

def load_weights(model, file_name):
    model.load_weights(file_name)

In [6]:
def draw_confusion_matrix(all_preds, all_labels):
    from sklearn.metrics import confusion_matrix
    import pandas as pd
    import seaborn as sn

    preds = []
    labels = []
    for p in all_preds:
        preds.append(float2int[p])
    for l in all_labels:
        labels.append(float2int[l])

    labels = np.array(labels)
    preds = np.array(preds)
    data=confusion_matrix(labels, preds)

    df_cm = pd.DataFrame(data, columns=labels_list)
    df_cm.index.name = 'Actual'
    df_cm.columns.name = 'Predicted'
    plt.figure(figsize = (13,10))
    sn.set(font_scale=3)#for label size
    sn.heatmap(df_cm, cmap="Blues", annot=True,annot_kws={"size": 16*2.5}, fmt='g')# font size
    plt.yticks(np.arange(len(labels_list))+0.5, labels_list, rotation=0, va="center")

In [7]:
def transform_sequences2norm_features(features):
    ret = []
    for i in range(len(features)):
        ret.append(stat_func(features[i], axis=0))
    return ret

In [8]:
def pred2label(pred):
    if pred < 0.25:
        return 0.0
    if pred < 0.5:
        return 0.33
    if pred < 0.75:
        return 0.66
    return 1.0

def get_prediction(model, file2feat, with_processing=False):
    y_pred = []
    exp_pred = []
    for fn, x in file2feat.items():
        #print(x.shape)
        if len(x)==0:
            continue
        if CONCATENATE_STAT:
            mean_x=np.repeat(stat_func(x,axis=0).reshape((1,-1)),len(x),axis=0)            
            #preds=model.predict(np.expand_dims(np.concatenate((mean_x,x-mean_x),axis=1), axis=0))[0]
            pred=model.predict(np.expand_dims(np.concatenate((mean_x,x),axis=1), axis=0), verbose=0)
        else:
            pred=model.predict(np.expand_dims(x, axis=0), verbose=0)
        #y_pred.append(np.argmax(pred))
        #if with_processing:
        #    y_pred.append(pred2label(pred[0]))
        #else:
        #    y_pred.append(float(pred[0]))
        if ENGAGE_WILD_DATASET is True or N_CLASSES == 2:
            y_pred.append(float(pred[0]))
        else:
            y_pred.append(np.argmax(pred))
        exp_pred.append(video2label[fn])
    return np.array(y_pred), np.array(exp_pred) 

def prediction2bin(preds, threshold):
    y_pred = []
    for y in preds:
        if N_CLASSES == 2:
            y = 0 if y < threshold else 1
        y_pred.append(y)
    return y_pred

def dump_to_table(table_name, metric_name, acc, mse, uar, recall, precision, f1, num_comma=","):
    def _to_str(num, num_comma):
        if num_comma == ",":
            return str(num).replace(".", ",")
        return str(num).replace(",", ".")
    
    import pandas as pd
    if os.path.isfile(table_name):
        df = pd.read_excel(open(table_name,'rb'))
    else:
        df = pd.DataFrame(columns=['metric_name', 'accuracy', 'mse', 'uar', 'recall', 'precision', 'f1'])

    idx = len(df)
    # Remove previous recordings
    df = df[df.metric_name != metric_name]
    df.loc[idx] = [
        metric_name,
        _to_str(acc, num_comma),
        _to_str(mse, num_comma),
        _to_str(uar, num_comma),
        _to_str(recall, num_comma),
        _to_str(precision, num_comma), 
        _to_str(f1, num_comma)
    ] 
    df.to_excel(table_name, index=False) 
    

def print_results(pred, labels, table_name=None, metric_name=None, from_int=False):
    if from_int is False:
        float2int={0:0,0.33:1,0.66:2,1:3}
        if N_CLASSES == 2:
            float2int={0:1,0.33:1,0.66:0,1:0}
        pred_int = np.array([float2int[pred2label(l)] for l in pred])
        labels_int = np.array([float2int[l] for l in labels])
        #labels_float = np.array([int2float[l] for l in labels])
        #pred_float = np.array([int2float[l] for l in pred])
        acc = (labels_int==pred_int).mean()
        #if N_CLASSES == 2:
        #    mse = ((labels-pred)**2).mean()
        #else:
        #    mse = ((labels_float-pred_float)**2).mean()
        mse = ((labels-pred)**2).mean()
        uar = recall_score(y_true=labels_int,y_pred=pred_int, average='macro')
        if N_CLASSES == 2:
            recall = recall_score(y_true=labels,y_pred=pred, average='binary',pos_label=1)
            precision = precision_score(y_true=labels,y_pred=pred, average='binary',pos_label=1)
            f1 = f1_score(y_true=labels,y_pred=pred, average='binary',pos_label=1)
        else:
            recall = 'N/A'
            precision = 'N/A'
            f1 = f1_score(y_true=labels_int,y_pred=pred_int, average='macro')
    else:
        int2float = {0:0.0, 1: 0.33, 2:0.66, 3: 1.0}
        labels_float = np.array([int2float[l] for l in labels])
        pred_float = np.array([int2float[l] for l in pred])
        acc = (labels==pred).mean()
        if N_CLASSES == 2:
            mse = ((labels-pred)**2).mean()
        else:
            mse = ((labels_float-pred_float)**2).mean()
        uar = recall_score(y_true=labels,y_pred=pred, average='macro')
        if N_CLASSES == 2:
            recall = recall_score(y_true=labels,y_pred=pred, average='binary',pos_label=1)
            precision = precision_score(y_true=labels,y_pred=pred, average='binary',pos_label=1)
            f1 = f1_score(y_true=labels,y_pred=pred, average='binary',pos_label=1)
        else:
            recall = 'N/A'
            precision = 'N/A'
            f1 = f1_score(y_true=labels,y_pred=pred, average='macro')
    print('Metric_name: ', metric_name,
          'Accuracy: ', acc,
          'MSE: ', mse,
          'UAR: ', uar,
          'Recall: ', recall,
          'Precision: ', precision,
          'F1: ', f1)
    if table_name is not None and metric_name is not None:
        dump_to_table(table_name, metric_name, acc, mse, uar, recall, precision, f1)
    return acc, mse, uar, recall, precision, f1

def print_mse(preds, labels):
    diff_correct,num_total=0.0,0
    for i in range(len(preds)):
        diff_correct+=(preds[i]-labels[i])**2
        num_total+=1
    print(num_total,diff_correct/num_total)

In [9]:
def define_dataset(engage_wild_dataset, bin_classification):
    ENGAGE_WILD_DATASET = engage_wild_dataset
    BIN_CLASSIFICATION = bin_classification
    if BIN_CLASSIFICATION is True:
        N_CLASSES = 2
    else:
        N_CLASSES = 4
    if ENGAGE_WILD_DATASET is True:
        ext = "png"
        DATASET_NAME = 'EngageWild'
        BASE_DATASET_DIR = '/home/HDD6TB/datasets/emotions/EmotiW/engagement/'
        BASE_DATASET_DIR = '/media/deelvin_disk/echuraev/Workspace/HSE/datasets/EngageWild'
    else:
        ext = "jpg"
        DATASET_NAME = 'DAiSEE'
        BASE_DATASET_DIR = '/home/HDD6TB/datasets/emotions/DAiSEE/'
        BASE_DATASET_DIR = '/media/deelvin_disk/echuraev/Workspace/HSE/datasets/DAiSEE'

    return ENGAGE_WILD_DATASET, BIN_CLASSIFICATION, N_CLASSES, DATASET_NAME, BASE_DATASET_DIR, ext

In [10]:
def get_video2label():
    labels_list = ['very distracted', 'distracted', 'engaged', 'very engaged']
    float2int={0:0,0.33:1,0.66:2,1:3}
    if ENGAGE_WILD_DATASET is True:
        import csv
        video2label={}
        with open(os.path.join(BASE_DATASET_DIR,'Engagement_Labels_Engagement.csv'), mode='r') as csvfile:
            labels_reader = csv.reader(csvfile, delimiter='\t')
            for i,row in enumerate(labels_reader):
                if i==0:
                    continue
                #videoname,label=row[0],float2int[float(row[1])]
                videoname,label=row[0],float(row[1])
                video2label[videoname]=label
                #print(videoname,label)
                #if (videoname not in filename2features_val) and (videoname not in filename2features_train):
                #    print(videoname,label)
        #check if fix is incorrect
        video2label['subject_87_Vid_3']=video2label['subject_77_Vid_6']
        #video2label = to_categorical(video2label)
        #print(len(video2label))
        #print(video2label)
        if N_CLASSES == 2:
            labels_list = ['engaged', 'distracted']
            bin_labels = {0:1, 1:1, 2:0, 3:0}
            for k in video2label.keys():
                video2label[k] = bin_labels[float2int[video2label[k]]]
    else:
        import pandas as pd
        df=pd.read_csv(os.path.join(BASE_DATASET_DIR,'Labels/AllLabels.csv'))
        df.columns = df.columns.str.replace(' ', '')
        df.head()
        labels2fileAndValues=df.set_index('ClipID').to_dict()
        video2label={os.path.splitext(video)[0]:val for video,val in labels2fileAndValues['Engagement'].items()}
        #print(video2label)
        if N_CLASSES == 2:
            labels_list = ['engaged', 'distracted']
            bin_labels = {0:1, 1:1, 2:0, 3:0}
            for k in video2label.keys():
                video2label[k] = bin_labels[video2label[k]]
    return video2label, labels_list

In [11]:
def get_weights_path(metric_name):
    return WEIGHTS_DIR + "{}.h5".format(metric_name)
    
def get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, aggregator, classificator):
    return "{}_{}_{}_{}_{}".format(base_model_key, DATASET_NAME, N_CLASSES, aggregator, classificator)

In [12]:
def print_classes(labels):
    min_num = len(labels)
    for i in np.unique(labels):
        count = labels.count(i)
        if count < min_num:
            min_num = count
        percent = count * 100 / len(labels)
        print("{} {}/{}: {}%".format(i, count, len(labels), percent))

def get_labels(fn2feat):
    labels = []
    for fn in fn2feat.keys():
        labels.append(video2label[fn])
    return labels

## Dataset balancing

In [13]:
def create_dataset(filename2features):
    x = []
    y = []
    ind=0
    for fn in filename2features:
        features=filename2features[fn]
        total_features=None
        if USE_ALL_FEATURES:
            #prev=features[0].shape
            cur_features=features[0][features[-1]==1]
            #if filename2features_val==filename2features:
            #    print(cur_features.shape)
            #    cur_features=cur_features[2:]
            #print(prev,features.shape)
        else:
            cur_features=features
        
        total_features=stat_func(cur_features)
        if total_features is not None:
            x.append(total_features)
            y.append(video2label[fn])
    x=np.array(x)
    y=np.array(y)
    print(x.shape,y.shape)
    return x,y

In [14]:
import random
random.seed(7)

def get_train_test_indices(x_all, y_all, SPLIT_RATIO):
    inds_train, inds_test = [], []
    inds = np.arange(len(y_all))
    
    for lbl in np.unique(y_all):
        tmp_inds = inds[y_all == lbl]
        random.shuffle(tmp_inds)
        
        ind=int(len(tmp_inds) * SPLIT_RATIO)
        inds_train.append(tmp_inds[:ind])
        inds_test.append(tmp_inds[ind:])
    
    inds_train = np.concatenate(inds_train, axis=0)
    inds_test = np.concatenate(inds_test, axis=0)
    return inds_train, inds_test

def get_train_test():
    inds_train, inds_test = get_train_test_indices()
    X_train=x_all[inds_train]
    y_train=y_all[inds_train]
    X_test=x_all[inds_test]
    y_test=y_all[inds_test]
    return X_train,y_train,X_test,y_test

In [15]:
def prepare_new_dataset(filename2features_train, filename2features_test, SPLIT_RATIO): # =0.767):
    x_train, y_train = create_dataset(filename2features_train)
    x_test, y_test = create_dataset(filename2features_test)

    x_all=np.concatenate((x_train,x_test),axis=0)
    y_all=np.concatenate((y_train,y_test),axis=0)
    print(x_all.shape,y_all.shape)
    
    labels,counts=np.unique(y_all,return_counts=True)
    print(labels,counts)

    filenames_all=list(filename2features_train.keys())+list(filename2features_test.keys())
    print(len(filenames_all))
    
    random.seed(7)
    inds_train, inds_val = get_train_test_indices(x_all, y_all, SPLIT_RATIO)
    
    def get_filename2features_new(inds):
        filename2features_new={}
        for ind in inds:
            filename=filenames_all[ind]
            if filename in filename2features_train:
                filename2features_new[filename]=filename2features_train[filename]
            else:
                filename2features_new[filename]=filename2features_test[filename]
        print(len(filename2features_new))
        return filename2features_new
    filename2features_train_new=get_filename2features_new(inds_train)
    filename2features_val_new=get_filename2features_new(inds_val)

    return filename2features_train_new, filename2features_val_new

In [16]:
def compute_descriptor(cur_features, axis=0):
    #cur_features=cur_features[1000:]
    #cur_features=cur_features[30:-30]
    #mean_features=features.mean(axis=0)
    mean_features = np.mean(cur_features, axis=0)
    std_features = np.std(cur_features, axis=0)
    max_features = np.max(cur_features, axis=0)
    min_features = np.min(cur_features, axis=0)

    # join several features together
    #feature = np.concatenate((mean_features, std_features, min_features, max_features), axis=None)                    
    #feature = np.concatenate((mean_features, std_features, max_features), axis=None)
    #feature = np.concatenate((mean_features, min_features, max_features), axis=None)
    feature = np.concatenate((mean_features, std_features), axis=None)
    #feature = np.concatenate((max_features, std_features), axis=None)

    #feature=std_features
    #feature=mean_features
    #feature=np.percentile(cur_features, 100,axis=0)
    return feature

## Attention specific

In [17]:
IMAGE_SET_SIZE=128 #32 #20
USE_GENERATORS = False
USE_ALL_FEATURES = False

In [18]:
def get_samples(features_list, labels_list):
    X_subsample,y_subsample=[],[]
    for i in range(len(features_list)):
        features=features_list[i]
        label = labels_list[i]
        total_features=None
        if USE_ALL_FEATURES:
            x=features[0][features[-1]==1]
        else:
            x=features
        max_ind=len(x)-IMAGE_SET_SIZE
        if max_ind<=0:
            continue
        stat_x=stat_func(x,axis=0)
        num_samples=max(max_ind//(IMAGE_SET_SIZE),1)
        for frame_ind in random.sample(range(max_ind),k=num_samples):
            if CONCATENATE_STAT:
                X_current=[np.concatenate((stat_x,x[frame_ind+i])) for i in range(IMAGE_SET_SIZE)]
            else:
                X_current=[x[frame_ind+i] for i in range(IMAGE_SET_SIZE)]
            X_subsample.append(X_current)
            y_subsample.append(label)
    
 
    X_subsample = np.array(X_subsample)
    y_subsample=np.array(y_subsample)
    print(X_subsample.shape,y_subsample.shape)
    return X_subsample,y_subsample

def get_samples(filename2features):
    X_subsample,y_subsample=[],[]
    for fn in filename2features:
        features=filename2features[fn]
        total_features=None
        if USE_ALL_FEATURES:
            x=features[0][features[-1]==1]
        else:
            x=features
        max_ind=len(x)-IMAGE_SET_SIZE
        if max_ind<=0:
            continue
        stat_x=stat_func(x,axis=0)
        num_samples=max(max_ind//(IMAGE_SET_SIZE),1)
        for frame_ind in random.sample(range(max_ind),k=num_samples):
            if CONCATENATE_STAT:
                X_current=[np.concatenate((stat_x,x[frame_ind+i])) for i in range(IMAGE_SET_SIZE)]
            else:
                X_current=[x[frame_ind+i] for i in range(IMAGE_SET_SIZE)]
            X_subsample.append(X_current)
            y_subsample.append(video2label[fn])
    
 
    X_subsample = np.array(X_subsample)
    y_subsample=np.array(y_subsample)
    print(X_subsample.shape,y_subsample.shape)
    return X_subsample,y_subsample

In [19]:
def generate_batch(features, labels):
    X_subsample = []
    Y_subsample = []
    while True:
        idx = random.randrange(len(labels))
        x = features[idx]
        img_count=len(x)
        num_per_part = img_count // IMAGE_SET_SIZE

        if img_count < IMAGE_SET_SIZE:
            continue

        mean_x=stat_func(x,axis=0)
        for j in range(img_count):
            if CONCATENATE_STAT:
                #X_current=[np.concatenate((mean_x,x[frame_ind+i]-mean_x)) for i in range(IMAGE_SET_SIZE)]
                X_current=[np.concatenate((mean_x,x[random.randint(num_per_part*j, min(num_per_part*(j+1),img_count-1))])) for j in range(IMAGE_SET_SIZE)]
            else:
                X_current=[x[random.randint(num_per_part*j, min(num_per_part*(j+1),img_count-1))] for j in range(IMAGE_SET_SIZE)]
            X_subsample.append(X_current)
            Y_subsample.append(labels[idx])
            if len(Y_subsample) >= BATCH_SIZE:
                X_subsample = np.array(X_subsample)
                Y_subsample = np.array(Y_subsample)
                yield X_subsample, Y_subsample
                X_subsample = []
                Y_subsample = []


def get_num_samples(features):
    num_samples = 0
    for idx in range(len(features)):
        x = features[idx]
        img_count=len(x)
        num_per_part = img_count // IMAGE_SET_SIZE

        if img_count < IMAGE_SET_SIZE:
            continue

        num_samples += img_count

    if num_samples % BATCH_SIZE > 0:
        num_samples += BATCH_SIZE

    if CONCATENATE_STAT:
        mean_x=stat_func(x,axis=0)
        #X_current=[np.concatenate((mean_x,x[frame_ind+i]-mean_x)) for i in range(IMAGE_SET_SIZE)]
        vector_dim = len(np.concatenate((mean_x,x[random.randint(0, min(num_per_part,img_count-1))])))
    else:
        vector_dim = len(x[random.randint(0, min(num_per_part,img_count-1))])
    return num_samples, vector_dim

In [20]:
from tensorflow.keras.models import load_model,Model
class SaveBestModel(tf.keras.callbacks.Callback):
    def __init__(self, save_best_metric='val_loss', this_max=False):
        self.save_best_metric = save_best_metric
        self.max = this_max
        if this_max:
            self.best = float('-inf')
        else:
            self.best = float('inf')

    def on_epoch_end(self, epoch, logs=None):
        metric_value = logs[self.save_best_metric]
        if self.max:
            if metric_value > self.best:
                self.best = metric_value
                self.best_model_weights = self.model.get_weights()

        else:
            if metric_value < self.best:
                self.best = metric_value
                self.best_model_weights = self.model.get_weights()

# custom metrics for computing balance accuracy, mcc and f1
class SaveBestModelByUAR(tf.keras.callbacks.Callback):
    def __init__(self, X, Y):
        super(SaveBestModelByUAR, self).__init__()
        self.X = X
        self.Y = Y #.argmax(axis=-1)
        self.best = float('-inf')
        self.best_model_weights = None
    
    def on_epoch_end(self, epoch, logs=None):
        predictions = self.model.predict(self.X)
        y_pred = predictions.argmax(axis=-1)

        metric_value = logs['val_uar'] = recall_score(y_true=self.Y,y_pred=y_pred, average='macro')
        logs['val_bacc'] = balanced_accuracy_score(self.Y, y_pred)
        logs['val_f1'] = f1_score(self.Y, y_pred, average='micro')
        logs['val_mcc'] = matthews_corrcoef(self.Y, y_pred)
        if metric_value > self.best:
            self.best = metric_value
            self.best_model_weights = self.model.get_weights()

In [21]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def uar_m(y_true, preds):
    y_pred = K.argmax(preds, axis=-1)
    return recall_m(y_true, y_pred) # recall_score(y_true=y_true,y_pred=y_pred, average='macro')

In [22]:
def get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES):
    inputs = Input(shape=(None, FEATURE_VECTOR_DIM),name='image_set')  # (batch, samples, features)
    e = Dense(1, activation='linear', name='e')(inputs)
    e = Reshape([-1], name='alignment')(e)
    alpha = Activation('softmax', name='alpha')(e)
    
    alpha_repeated = Permute([2, 1],name='alpha_repeated')(RepeatVector(FEATURE_VECTOR_DIM, name='repeat')(alpha))
    
    c = Multiply(name='c')([inputs, alpha_repeated])
    x = Lambda(lambda xin: K.sum(xin, axis=1), output_shape=(FEATURE_VECTOR_DIM,), name='context')(c)
    
    #x = Dropout(0.3)(x)
    x = Dense(units=512, activation='relu', name='hidden_FC')(x)  # (batch, units) #128 64
    #x = tf.keras.activations.gelu(Dense(512, activation='linear')(x))

    #pred=Dense(N_CLASSES,activation='softmax')(x)
    if ENGAGE_WILD_DATASET is True or N_CLASSES == 2:
        pred=Dense(1,activation='sigmoid')(x)
        modelAtn=Model(inputs=inputs,outputs=pred)
        #modelAtn.compile(optimizer=Adam(lr=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        if N_CLASSES == 2:
            #modelAtn.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', metrics=['acc', f1_m,precision_m, recall_m])
            metrics=['acc',tf.keras.metrics.AUC(multi_label=True,name='auc'), tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.Recall(),tf.keras.metrics.Precision()]
            modelAtn.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', metrics=metrics)
        else:
            modelAtn.compile(optimizer=Adam(lr=1e-4), loss='mse', metrics=['mae'])
        save_best_model = SaveBestModel('val_loss',False)
    else:
        pred=Dense(N_CLASSES,activation='softmax')(x)
        modelAtn=Model(inputs=inputs,outputs=pred)
        #modelAtn.compile(optimizer=Adam(lr=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        #save_best_model = SaveBestModel('val_accuracy',True)
        #METRICS = [
        #    # keras.metrics.BinaryCrossentropy(name='cross entropy'),  # same as model's loss
        #    # keras.metrics.MeanSquaredError(name='Brier score'),
        #    # keras.metrics.TruePositives(name='tp'),
        #    # keras.metrics.FalsePositives(name='fp'),
        #    # keras.metrics.TrueNegatives(name='tn'),
        #    # keras.metrics.FalseNegatives(name='fn'), 
        #    # keras.metrics.BinaryAccuracy(name='accuracy'),
        #    #keras.metrics.Precision(name='precision'),
        #    #keras.metrics.Recall(name='recall'),
        #    # keras.metrics.AUC(name='auc'),
        #    # keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
        #    keras.metrics.F1Score(name='f1'),
        #    'accuracy',
        #    #f1_score_metric,
        #]
        modelAtn.compile(optimizer=Adam(lr=1e-4), loss='sparse_categorical_crossentropy', metrics=None)
        save_best_model = SaveBestModel('accuracy',True)
    modelAtn.summary()

    
    return modelAtn, save_best_model

In [23]:
def get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES):
    embeddings_dim=512
    embeddings_dim=FEATURE_VECTOR_DIM
    inputs = Input(shape=(None, FEATURE_VECTOR_DIM),name='image_set')  # (batch, samples, features)
    if False:
        query_seq_encoding, value_seq_encoding=inputs,inputs
    else:
        query_seq_encoding=Dense(embeddings_dim, activation='linear',use_bias=False, name='query')(inputs)
        value_seq_encoding=Dense(embeddings_dim, activation='linear',use_bias=False, name='value')(inputs)
    query_value_attention_seq = tf.keras.layers.Attention()([query_seq_encoding, value_seq_encoding])
    
    e = Dense(1, activation='linear', name='e')(query_value_attention_seq)
    e = Reshape([-1], name='alignment')(e)
    alpha = Activation('softmax', name='alpha')(e)
    
    alpha_repeated = Permute([2, 1],name='alpha_repeated')(RepeatVector(embeddings_dim, name='repeat')(alpha))
    
    c = Multiply(name='c')([query_value_attention_seq, alpha_repeated])
    x = Lambda(lambda xin: K.sum(xin, axis=1), output_shape=(embeddings_dim,), name='context')(c)

    #pred=Dense(N_CLASSES,activation='softmax')(x)
    pred=Dense(1,activation='sigmoid')(x)
    modelAtn=Model(inputs=inputs,outputs=pred)
    #modelAtn.compile(optimizer=Adam(lr=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    if N_CLASSES == 2:
        #modelAtn.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', metrics=['acc', f1_m,precision_m, recall_m])
        metrics=['acc',tf.keras.metrics.AUC(multi_label=True,name='auc'), tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.Recall(),tf.keras.metrics.Precision()]
        modelAtn.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', metrics=metrics)
    else:
        modelAtn.compile(optimizer=Adam(lr=1e-4), loss='mse', metrics=['mae'])
    modelAtn.summary()

    #save_best_model = SaveBestModel('val_accuracy',True)
    save_best_model = SaveBestModel('val_loss',False)
    return modelAtn, save_best_model

# EngageWild dataset (4 classes)

In [24]:
FEATURES_DIR = DATA_DIR + 'features_EngageWild/'
WEIGHTS_DIR = DATA_DIR + 'weights_EngageWild/'
TABLE_NAME = '02_EngageWild_4_classes.xlsx'

## enet_b0_8_best_afew

In [25]:
base_model_key = 'enet_b0_8_best_afew.pt'

In [26]:
ENGAGE_WILD_DATASET, BIN_CLASSIFICATION, N_CLASSES, DATASET_NAME, BASE_DATASET_DIR, ext = define_dataset(True, False)
video2label, labels_list = get_video2label()
print(labels_list)
print(video2label)

['very distracted', 'distracted', 'engaged', 'very engaged']
{'subject_1_Vid_1': 1.0, 'subject_1_Vid_2': 1.0, 'subject_1_Vid_3': 0.66, 'subject_1_Vid_4': 1.0, 'subject_1_Vid_5': 1.0, 'subject_31_Vid_6': 1.0, 'subject_2_Vid_6': 0.33, 'subject_3_Vid_6': 1.0, 'subject_3_Vid_1': 0.33, 'subject_3_Vid_2': 0.33, 'subject_3_Vid_3': 0.66, 'subject_3_Vid_4': 0.66, 'subject_3_Vid_5': 0.33, 'subject_3_Vid_7': 1.0, 'subject_4_Vid_6': 1.0, 'subject_5_Vid_6': 1.0, 'subject_6_Vid_6': 0.33, 'subject_7_Vid_1': 0.66, 'subject_7_Vid_2': 0.66, 'subject_7_Vid_3': 0.66, 'subject_7_Vid_4': 0.66, 'subject_7_Vid_5': 0.33, 'subject_8_Vid_6': 1.0, 'subject_9_Vid_6': 0.0, 'subject_10_Vid_6': 0.66, 'subject_11_Vid_6': 1.0, 'subject_12_Vid_6': 0.0, 'subject_13_Vid_6': 0.66, 'subject_14_Vid_6': 0.66, 'subject_15_Vid_6': 0.66, 'subject_16_Vid_6': 0.33, 'subject_17_Vid_6': 0.66, 'subject_18_Vid_6': 0.33, 'subject_19_Vid_6': 1.0, 'subject_20_Vid_6': 0.66, 'subject_20_Vid_1': 0.66, 'subject_20_Vid_2': 0.66, 'subject_20_V

### Std

In [27]:
stat_func = np.std
stat_name = "std"

#### Traditional split

In [28]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [29]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [30]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0.0 5/147: 3.401360544217687%
0.33 35/147: 23.80952380952381%
0.66 79/147: 53.74149659863946%
1.0 28/147: 19.047619047619047%
val:
0.0 4/48: 8.333333333333334%
0.33 10/48: 20.833333333333332%
0.66 19/48: 39.583333333333336%
1.0 15/48: 31.25%


##### Single attention

In [31]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_4_std_single_attention_traditional


In [32]:
CONCATENATE_STAT = True

In [33]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 2560) (5449,)
(2284, 128, 2560) (2284,)


In [34]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

2024-07-21 20:57:19.794348: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38431 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:11:00.0, compute capability: 8.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2561      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                              

In [35]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20


2024-07-21 20:57:28.377540: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0xcab7b090 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-21 20:57:28.377561: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2024-07-21 20:57:28.382472: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-21 20:57:28.415997: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-07-21 20:57:28.468254: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


43/43 [==============================] - 6s 102ms/step - loss: 0.0463 - mae: 0.1715 - val_loss: 0.0895 - val_mae: 0.2561
Epoch 2/20
43/43 [==============================] - 1s 32ms/step - loss: 0.0175 - mae: 0.1041 - val_loss: 0.0881 - val_mae: 0.2481
Epoch 3/20
43/43 [==============================] - 1s 31ms/step - loss: 0.0114 - mae: 0.0791 - val_loss: 0.0936 - val_mae: 0.2515
Epoch 4/20
43/43 [==============================] - 1s 31ms/step - loss: 0.0078 - mae: 0.0619 - val_loss: 0.0885 - val_mae: 0.2469
Epoch 5/20
43/43 [==============================] - 1s 32ms/step - loss: 0.0057 - mae: 0.0523 - val_loss: 0.0915 - val_mae: 0.2512
Epoch 6/20
43/43 [==============================] - 1s 32ms/step - loss: 0.0042 - mae: 0.0460 - val_loss: 0.1035 - val_mae: 0.2689
Epoch 7/20
43/43 [==============================] - 1s 32ms/step - loss: 0.0032 - mae: 0.0399 - val_loss: 0.0986 - val_mae: 0.2649
Epoch 8/20
43/43 [==============================] - 1s 32ms/step - loss: 0.0025 - mae: 0.0346

In [36]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3608044/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [37]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_std_single_attention_traditional Accuracy:  0.2916666666666667 MSE:  0.10009201708232872 UAR:  0.3478070175438596 Recall:  N/A Precision:  N/A F1:  0.3157444005270092


In [38]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [39]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3608044/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [40]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_std_single_attention_traditional_best Accuracy:  0.3541666666666667 MSE:  0.08007323866137694 UAR:  0.2850877192982456 Recall:  N/A Precision:  N/A F1:  0.2765014024088434


##### Self attention

In [41]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_4_std_self_attention_traditional


In [42]:
CONCATENATE_STAT = True

In [43]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 2560) (5449,)
(2284, 128, 2560) (2284,)


In [44]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention (Attention)       (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [45]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 7s 152ms/step - loss: 0.2071 - mae: 0.3759 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 2/20
43/43 [==============================] - 4s 91ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 3/20
43/43 [==============================] - 4s 91ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 4/20
43/43 [==============================] - 4s 91ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 5/20
43/43 [==============================] - 4s 91ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 6/20
43/43 [==============================] - 4s 91ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 7/20
43/43 [==============================] - 4s 91ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 8/20
43/43 [==============================] - 4s 90ms/step - loss: 0.2089 - 

In [46]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3608044/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [47]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_std_self_attention_traditional Accuracy:  0.3125 MSE:  0.22261249999999996 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.11904761904761904


In [48]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [49]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3608044/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [50]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_std_self_attention_traditional_best Accuracy:  0.3125 MSE:  0.22261249999999996 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.11904761904761904


#### Balanced split

In [51]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [52]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [53]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0.0 5/147: 3.401360544217687%
0.33 35/147: 23.80952380952381%
0.66 79/147: 53.74149659863946%
1.0 28/147: 19.047619047619047%
val:
0.0 4/48: 8.333333333333334%
0.33 10/48: 20.833333333333332%
0.66 19/48: 39.583333333333336%
1.0 15/48: 31.25%


In [54]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.767)

(147,) (147,)
(48,) (48,)
(195,) (195,)
[0.   0.33 0.66 1.  ] [ 9 45 98 43]
195
147
48


In [55]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0.0 6/147: 4.081632653061225%
0.33 34/147: 23.12925170068027%
0.66 75/147: 51.02040816326531%
1.0 32/147: 21.768707482993197%
val:
0.0 3/48: 6.25%
0.33 11/48: 22.916666666666668%
0.66 23/48: 47.916666666666664%
1.0 11/48: 22.916666666666668%


##### Single attention

In [56]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_4_std_single_attention_balanced


In [57]:
CONCATENATE_STAT = True

In [58]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5641, 128, 2560) (5641,)
(2092, 128, 2560) (2092,)


In [59]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2561      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [60]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
45/45 [==============================] - 4s 72ms/step - loss: 0.0421 - mae: 0.1615 - val_loss: 0.0632 - val_mae: 0.2078
Epoch 2/20
45/45 [==============================] - 2s 37ms/step - loss: 0.0124 - mae: 0.0855 - val_loss: 0.0632 - val_mae: 0.2054
Epoch 3/20
45/45 [==============================] - 1s 31ms/step - loss: 0.0072 - mae: 0.0624 - val_loss: 0.0682 - val_mae: 0.2172
Epoch 4/20
45/45 [==============================] - 1s 30ms/step - loss: 0.0053 - mae: 0.0538 - val_loss: 0.0716 - val_mae: 0.2219
Epoch 5/20
45/45 [==============================] - 1s 31ms/step - loss: 0.0035 - mae: 0.0407 - val_loss: 0.0742 - val_mae: 0.2249
Epoch 6/20
45/45 [==============================] - 1s 32ms/step - loss: 0.0028 - mae: 0.0368 - val_loss: 0.0769 - val_mae: 0.2310
Epoch 7/20
45/45 [==============================] - 2s 36ms/step - loss: 0.0024 - mae: 0.0345 - val_loss: 0.0777 - val_mae: 0.2317
Epoch 8/20
45/45 [==============================] - 1s 31ms/step - loss: 0.0017 - m

In [61]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3608044/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [62]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_std_single_attention_balanced Accuracy:  0.3541666666666667 MSE:  0.0828139840055649 UAR:  0.363965744400527 Recall:  N/A Precision:  N/A F1:  0.36019736842105265


In [63]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [64]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3608044/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [65]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_std_single_attention_balanced_best Accuracy:  0.4583333333333333 MSE:  0.056269898658401506 UAR:  0.4183135704874835 Recall:  N/A Precision:  N/A F1:  0.4298611111111111


##### Self attention

In [66]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_4_std_self_attention_balanced


In [67]:
CONCATENATE_STAT = True

In [68]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5641, 128, 2560) (5641,)
(2092, 128, 2560) (2092,)


In [69]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention_1 (Attention)     (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [70]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
45/45 [==============================] - 7s 138ms/step - loss: 0.2181 - mae: 0.3788 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 2/20
45/45 [==============================] - 4s 88ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 3/20
45/45 [==============================] - 4s 88ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 4/20
45/45 [==============================] - 4s 99ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 5/20
45/45 [==============================] - 4s 89ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 6/20
45/45 [==============================] - 4s 88ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 7/20
45/45 [==============================] - 4s 87ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 8/20
45/45 [==============================] - 4s 88ms/step - loss: 0.2202 - 

In [71]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3608044/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [72]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_std_self_attention_balanced Accuracy:  0.22916666666666666 MSE:  0.2207645833333333 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.09322033898305083


In [73]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [74]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3608044/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [75]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_std_self_attention_balanced_best Accuracy:  0.22916666666666666 MSE:  0.2207645833333333 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.09322033898305083


### Max

In [27]:
stat_func = np.max
stat_name = "max"

#### Traditional split

In [77]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [78]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [79]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0.0 5/147: 3.401360544217687%
0.33 35/147: 23.80952380952381%
0.66 79/147: 53.74149659863946%
1.0 28/147: 19.047619047619047%
val:
0.0 4/48: 8.333333333333334%
0.33 10/48: 20.833333333333332%
0.66 19/48: 39.583333333333336%
1.0 15/48: 31.25%


##### Single attention

In [80]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_4_max_single_attention_traditional


In [81]:
CONCATENATE_STAT = True

In [82]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 2560) (5449,)
(2284, 128, 2560) (2284,)


In [83]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2561      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [84]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 5s 89ms/step - loss: 0.2061 - mae: 0.3746 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 2/20
43/43 [==============================] - 1s 31ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 3/20
43/43 [==============================] - 1s 31ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 4/20
43/43 [==============================] - 1s 32ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 5/20
43/43 [==============================] - 1s 31ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 6/20
43/43 [==============================] - 1s 32ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 7/20
43/43 [==============================] - 1s 32ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 8/20
43/43 [==============================] - 1s 32ms/step - loss: 0.2089 - m

In [85]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3608044/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [86]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_max_single_attention_traditional Accuracy:  0.3125 MSE:  0.22261250000000113 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.11904761904761904


In [87]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [88]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3608044/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [89]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_max_single_attention_traditional_best Accuracy:  0.3125 MSE:  0.22261250000000113 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.11904761904761904


##### Self attention

In [90]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_4_max_self_attention_traditional


In [91]:
CONCATENATE_STAT = True

In [92]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 2560) (5449,)
(2284, 128, 2560) (2284,)


In [93]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention_2 (Attention)     (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [94]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 7s 135ms/step - loss: 0.2059 - mae: 0.3744 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 2/20
43/43 [==============================] - 4s 91ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 3/20
43/43 [==============================] - 4s 92ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 4/20
43/43 [==============================] - 4s 91ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 5/20
43/43 [==============================] - 4s 91ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 6/20
43/43 [==============================] - 4s 92ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 7/20
43/43 [==============================] - 4s 91ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 8/20
43/43 [==============================] - 4s 90ms/step - loss: 0.2089 - 

In [95]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3608044/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [96]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_max_self_attention_traditional Accuracy:  0.3125 MSE:  0.22261249999999996 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.11904761904761904


In [97]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [98]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3608044/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [99]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_max_self_attention_traditional_best Accuracy:  0.3125 MSE:  0.22261249999999996 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.11904761904761904


#### Balanced split

In [28]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [29]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [30]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0.0 5/147: 3.401360544217687%
0.33 35/147: 23.80952380952381%
0.66 79/147: 53.74149659863946%
1.0 28/147: 19.047619047619047%
val:
0.0 4/48: 8.333333333333334%
0.33 10/48: 20.833333333333332%
0.66 19/48: 39.583333333333336%
1.0 15/48: 31.25%


In [31]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.767)

(147,) (147,)
(48,) (48,)
(195,) (195,)
[0.   0.33 0.66 1.  ] [ 9 45 98 43]
195
147
48


In [32]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0.0 6/147: 4.081632653061225%
0.33 34/147: 23.12925170068027%
0.66 75/147: 51.02040816326531%
1.0 32/147: 21.768707482993197%
val:
0.0 3/48: 6.25%
0.33 11/48: 22.916666666666668%
0.66 23/48: 47.916666666666664%
1.0 11/48: 22.916666666666668%


##### Single attention

In [33]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_4_max_single_attention_balanced


In [34]:
CONCATENATE_STAT = True

In [35]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5641, 128, 2560) (5641,)
(2092, 128, 2560) (2092,)


In [36]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

2024-07-22 05:20:12.882923: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38431 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:11:00.0, compute capability: 8.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2561      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                              

In [37]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20


2024-07-22 05:20:22.010341: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f8a48007850 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-22 05:20:22.010361: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2024-07-22 05:20:22.015152: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-22 05:20:22.056624: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-07-22 05:20:22.109869: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


45/45 [==============================] - 6s 94ms/step - loss: 0.2172 - mae: 0.3782 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 2/20
45/45 [==============================] - 1s 30ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 3/20
45/45 [==============================] - 1s 30ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 4/20
45/45 [==============================] - 1s 30ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 5/20
45/45 [==============================] - 1s 30ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 6/20
45/45 [==============================] - 1s 30ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 7/20
45/45 [==============================] - 1s 30ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 8/20
45/45 [==============================] - 1s 30ms/step - loss: 0.2202 - mae: 0.3811 

In [38]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3803109/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [39]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_max_single_attention_balanced Accuracy:  0.22916666666666666 MSE:  0.2207645833333333 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.09322033898305083


In [40]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [41]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3803109/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [42]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_max_single_attention_balanced_best Accuracy:  0.22916666666666666 MSE:  0.2207645833333336 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.09322033898305083


##### Self attention

In [43]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_4_max_self_attention_balanced


In [44]:
CONCATENATE_STAT = True

In [45]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5641, 128, 2560) (5641,)
(2092, 128, 2560) (2092,)


In [46]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention (Attention)       (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [47]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
45/45 [==============================] - 8s 152ms/step - loss: 0.2205 - mae: 0.3819 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 2/20
45/45 [==============================] - 4s 88ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 3/20
45/45 [==============================] - 4s 87ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 4/20
45/45 [==============================] - 4s 88ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 5/20
45/45 [==============================] - 4s 87ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 6/20
45/45 [==============================] - 4s 88ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 7/20
45/45 [==============================] - 4s 88ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 8/20
45/45 [==============================] - 4s 87ms/step - loss: 0.2202 - 

In [48]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3803109/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [49]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_max_self_attention_balanced Accuracy:  0.22916666666666666 MSE:  0.2207645833333333 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.09322033898305083


In [50]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [51]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3803109/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [52]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_max_self_attention_balanced_best Accuracy:  0.22916666666666666 MSE:  0.2207645833333333 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.09322033898305083


### STAT

In [27]:
stat_func = compute_descriptor
stat_name = "STAT"

#### Traditional split

In [28]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [29]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [30]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0.0 5/147: 3.401360544217687%
0.33 35/147: 23.80952380952381%
0.66 79/147: 53.74149659863946%
1.0 28/147: 19.047619047619047%
val:
0.0 4/48: 8.333333333333334%
0.33 10/48: 20.833333333333332%
0.66 19/48: 39.583333333333336%
1.0 15/48: 31.25%


##### Single attention

In [31]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_4_STAT_single_attention_traditional


In [32]:
CONCATENATE_STAT = True

In [33]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 3840) (5449,)
(2284, 128, 3840) (2284,)


In [34]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

2024-07-22 06:05:31.848209: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38431 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:11:00.0, compute capability: 8.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3840)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              3841      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                              

In [35]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

In [36]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3828290/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [37]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_STAT_single_attention_traditional Accuracy:  0.25 MSE:  0.1146485906264167 UAR:  0.22631578947368422 Recall:  N/A Precision:  N/A F1:  0.2108556832694764


In [39]:
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    modelAtn.set_weights(best_model_weights)
    save_weights(modelAtn, weights_name)

In [40]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3828290/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [41]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_STAT_single_attention_traditional_best Accuracy:  0.3958333333333333 MSE:  0.08042476034517941 UAR:  0.3078947368421053 Recall:  N/A Precision:  N/A F1:  0.2902121374865736


##### Self attention

In [42]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_4_STAT_self_attention_traditional


In [43]:
CONCATENATE_STAT = True

In [44]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 3840) (5449,)
(2284, 128, 3840) (2284,)


In [45]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3840)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 3840)           1474560   ['image_set[0][0]']           
                                                          0                                       
                                                                                                  
 value (Dense)               (None, None, 3840)           1474560   ['image_set[0][0]']           
                                                          0                                       
                                                                                            

In [46]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

In [47]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3828290/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [48]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_STAT_self_attention_traditional Accuracy:  0.3541666666666667 MSE:  0.10893076012359715 UAR:  0.38728070175438595 Recall:  N/A Precision:  N/A F1:  0.36451612903225805


In [49]:
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    modelAtn.set_weights(best_model_weights)
    save_weights(modelAtn, weights_name)

In [50]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3828290/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [51]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_STAT_self_attention_traditional_best Accuracy:  0.4166666666666667 MSE:  0.07915359221476707 UAR:  0.4232456140350877 Recall:  N/A Precision:  N/A F1:  0.45722610722610724


#### Balanced split

In [52]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [53]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [54]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0.0 5/147: 3.401360544217687%
0.33 35/147: 23.80952380952381%
0.66 79/147: 53.74149659863946%
1.0 28/147: 19.047619047619047%
val:
0.0 4/48: 8.333333333333334%
0.33 10/48: 20.833333333333332%
0.66 19/48: 39.583333333333336%
1.0 15/48: 31.25%


In [55]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.767)

(147, 2560) (147,)
(48, 2560) (48,)
(195, 2560) (195,)
[0.   0.33 0.66 1.  ] [ 9 45 98 43]
195
147
48


In [56]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0.0 6/147: 4.081632653061225%
0.33 34/147: 23.12925170068027%
0.66 75/147: 51.02040816326531%
1.0 32/147: 21.768707482993197%
val:
0.0 3/48: 6.25%
0.33 11/48: 22.916666666666668%
0.66 23/48: 47.916666666666664%
1.0 11/48: 22.916666666666668%


##### Single attention

In [57]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_4_STAT_single_attention_balanced


In [58]:
CONCATENATE_STAT = True

In [59]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5641, 128, 3840) (5641,)
(2092, 128, 3840) (2092,)


In [60]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3840)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              3841      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [61]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20


2024-07-22 06:09:52.493356: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f7eec008940 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-22 06:09:52.493378: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2024-07-22 06:09:52.498201: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-22 06:09:52.535635: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-07-22 06:09:52.588560: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


45/45 [==============================] - 8s 146ms/step - loss: 0.0425 - mae: 0.1595 - val_loss: 0.0671 - val_mae: 0.2109
Epoch 2/20
45/45 [==============================] - 2s 43ms/step - loss: 0.0109 - mae: 0.0782 - val_loss: 0.0690 - val_mae: 0.2114
Epoch 3/20
45/45 [==============================] - 2s 47ms/step - loss: 0.0057 - mae: 0.0518 - val_loss: 0.0778 - val_mae: 0.2242
Epoch 4/20
45/45 [==============================] - 2s 43ms/step - loss: 0.0043 - mae: 0.0455 - val_loss: 0.0797 - val_mae: 0.2309
Epoch 5/20
45/45 [==============================] - 2s 44ms/step - loss: 0.0030 - mae: 0.0362 - val_loss: 0.0795 - val_mae: 0.2317
Epoch 6/20
45/45 [==============================] - 2s 44ms/step - loss: 0.0021 - mae: 0.0302 - val_loss: 0.0808 - val_mae: 0.2333
Epoch 7/20
45/45 [==============================] - 2s 45ms/step - loss: 0.0021 - mae: 0.0313 - val_loss: 0.0903 - val_mae: 0.2447
Epoch 8/20
45/45 [==============================] - 2s 44ms/step - loss: 0.0016 - mae: 0.0276

In [62]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3828290/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [63]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_STAT_single_attention_balanced Accuracy:  0.3541666666666667 MSE:  0.08639217736825654 UAR:  0.363965744400527 Recall:  N/A Precision:  N/A F1:  0.34731934731934727


In [64]:
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    modelAtn.set_weights(best_model_weights)
    save_weights(modelAtn, weights_name)

In [65]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3828290/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [66]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_STAT_single_attention_balanced_best Accuracy:  0.4791666666666667 MSE:  0.0586605660204162 UAR:  0.42918313570487476 Recall:  N/A Precision:  N/A F1:  0.4437641723356009


##### Self attention

In [67]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_4_STAT_self_attention_balanced


In [68]:
CONCATENATE_STAT = True

In [69]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5641, 128, 3840) (5641,)
(2092, 128, 3840) (2092,)


In [70]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3840)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 3840)           1474560   ['image_set[0][0]']           
                                                          0                                       
                                                                                                  
 value (Dense)               (None, None, 3840)           1474560   ['image_set[0][0]']           
                                                          0                                       
                                                                                            

In [ ]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
45/45 [==============================] - 13s 270ms/step - loss: 0.2176 - mae: 0.3791 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 2/20
45/45 [==============================] - 8s 170ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 3/20
45/45 [==============================] - 9s 197ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 4/20
45/45 [==============================] - 8s 172ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 5/20
45/45 [==============================] - 8s 171ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 6/20
45/45 [==============================] - 8s 171ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 7/20
45/45 [==============================] - 8s 170ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 8/20
45/45 [==============================] - 8s 170ms/step - loss: 0

In [ ]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

In [ ]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

In [ ]:
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    modelAtn.set_weights(best_model_weights)
    save_weights(modelAtn, weights_name)

In [ ]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

In [ ]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

## MobileNet_7

In [25]:
base_model_key = 'mobilenet_7.h5'

In [26]:
ENGAGE_WILD_DATASET, BIN_CLASSIFICATION, N_CLASSES, DATASET_NAME, BASE_DATASET_DIR, ext = define_dataset(True, False)
video2label, labels_list = get_video2label()
print(labels_list)
print(video2label)

['very distracted', 'distracted', 'engaged', 'very engaged']
{'subject_1_Vid_1': 1.0, 'subject_1_Vid_2': 1.0, 'subject_1_Vid_3': 0.66, 'subject_1_Vid_4': 1.0, 'subject_1_Vid_5': 1.0, 'subject_31_Vid_6': 1.0, 'subject_2_Vid_6': 0.33, 'subject_3_Vid_6': 1.0, 'subject_3_Vid_1': 0.33, 'subject_3_Vid_2': 0.33, 'subject_3_Vid_3': 0.66, 'subject_3_Vid_4': 0.66, 'subject_3_Vid_5': 0.33, 'subject_3_Vid_7': 1.0, 'subject_4_Vid_6': 1.0, 'subject_5_Vid_6': 1.0, 'subject_6_Vid_6': 0.33, 'subject_7_Vid_1': 0.66, 'subject_7_Vid_2': 0.66, 'subject_7_Vid_3': 0.66, 'subject_7_Vid_4': 0.66, 'subject_7_Vid_5': 0.33, 'subject_8_Vid_6': 1.0, 'subject_9_Vid_6': 0.0, 'subject_10_Vid_6': 0.66, 'subject_11_Vid_6': 1.0, 'subject_12_Vid_6': 0.0, 'subject_13_Vid_6': 0.66, 'subject_14_Vid_6': 0.66, 'subject_15_Vid_6': 0.66, 'subject_16_Vid_6': 0.33, 'subject_17_Vid_6': 0.66, 'subject_18_Vid_6': 0.33, 'subject_19_Vid_6': 1.0, 'subject_20_Vid_6': 0.66, 'subject_20_Vid_1': 0.66, 'subject_20_Vid_2': 0.66, 'subject_20_V

### Std

In [27]:
stat_func = np.std
stat_name = "std"

#### Traditional split

In [28]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [29]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [30]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0.0 5/147: 3.401360544217687%
0.33 35/147: 23.80952380952381%
0.66 79/147: 53.74149659863946%
1.0 28/147: 19.047619047619047%
val:
0.0 4/48: 8.333333333333334%
0.33 10/48: 20.833333333333332%
0.66 19/48: 39.583333333333336%
1.0 15/48: 31.25%


##### Single attention

In [31]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

mobilenet_7.h5_EngageWild_4_std_single_attention_traditional


In [32]:
CONCATENATE_STAT = True

In [33]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 2048) (5449,)
(2284, 128, 2048) (2284,)


In [34]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

2024-07-23 15:58:54.594011: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38431 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:11:00.0, compute capability: 8.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2049      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                              

In [35]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20


2024-07-23 15:59:01.869079: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x13b0e8840 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-23 15:59:01.869100: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2024-07-23 15:59:01.873846: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-23 15:59:01.905719: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-07-23 15:59:01.957648: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


43/43 [==============================] - 5s 87ms/step - loss: 0.0493 - mae: 0.1724 - val_loss: 0.0992 - val_mae: 0.2626
Epoch 2/20
43/43 [==============================] - 1s 26ms/step - loss: 0.0181 - mae: 0.1056 - val_loss: 0.0975 - val_mae: 0.2508
Epoch 3/20
43/43 [==============================] - 1s 26ms/step - loss: 0.0098 - mae: 0.0745 - val_loss: 0.1053 - val_mae: 0.2578
Epoch 4/20
43/43 [==============================] - 1s 26ms/step - loss: 0.0065 - mae: 0.0600 - val_loss: 0.1108 - val_mae: 0.2641
Epoch 5/20
43/43 [==============================] - 1s 26ms/step - loss: 0.0044 - mae: 0.0480 - val_loss: 0.1173 - val_mae: 0.2652
Epoch 6/20
43/43 [==============================] - 1s 26ms/step - loss: 0.0033 - mae: 0.0416 - val_loss: 0.1229 - val_mae: 0.2696
Epoch 7/20
43/43 [==============================] - 1s 26ms/step - loss: 0.0024 - mae: 0.0352 - val_loss: 0.1229 - val_mae: 0.2677
Epoch 8/20
43/43 [==============================] - 1s 26ms/step - loss: 0.0017 - mae: 0.0292 

In [36]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3864770/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [37]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_std_single_attention_traditional Accuracy:  0.4375 MSE:  0.11541946616558597 UAR:  0.356578947368421 Recall:  N/A Precision:  N/A F1:  0.3360759953501889


In [38]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [39]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3864770/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [40]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_std_single_attention_traditional_best Accuracy:  0.4166666666666667 MSE:  0.08423179411250159 UAR:  0.30087719298245613 Recall:  N/A Precision:  N/A F1:  0.2812820512820513


##### Self attention

In [41]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

mobilenet_7.h5_EngageWild_4_std_self_attention_traditional


In [42]:
CONCATENATE_STAT = True

In [43]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 2048) (5449,)
(2284, 128, 2048) (2284,)


In [44]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 attention (Attention)       (None, None, 2048)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [45]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 6s 129ms/step - loss: 0.2048 - mae: 0.3731 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 2/20
43/43 [==============================] - 3s 65ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 3/20
43/43 [==============================] - 3s 64ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 4/20
43/43 [==============================] - 3s 65ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 5/20
43/43 [==============================] - 3s 65ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 6/20
43/43 [==============================] - 3s 66ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 7/20
43/43 [==============================] - 3s 66ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 8/20
43/43 [==============================] - 3s 65ms/step - loss: 0.2089 - 

In [46]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3864770/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [47]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_std_self_attention_traditional Accuracy:  0.3125 MSE:  0.22261249999999996 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.11904761904761904


In [48]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [49]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3864770/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [50]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_std_self_attention_traditional_best Accuracy:  0.3125 MSE:  0.22261249999999996 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.11904761904761904


#### Balanced split

In [51]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [52]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [53]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0.0 5/147: 3.401360544217687%
0.33 35/147: 23.80952380952381%
0.66 79/147: 53.74149659863946%
1.0 28/147: 19.047619047619047%
val:
0.0 4/48: 8.333333333333334%
0.33 10/48: 20.833333333333332%
0.66 19/48: 39.583333333333336%
1.0 15/48: 31.25%


In [54]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.767)

(147,) (147,)
(48,) (48,)
(195,) (195,)
[0.   0.33 0.66 1.  ] [ 9 45 98 43]
195
147
48


In [55]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0.0 6/147: 4.081632653061225%
0.33 34/147: 23.12925170068027%
0.66 75/147: 51.02040816326531%
1.0 32/147: 21.768707482993197%
val:
0.0 3/48: 6.25%
0.33 11/48: 22.916666666666668%
0.66 23/48: 47.916666666666664%
1.0 11/48: 22.916666666666668%


##### Single attention

In [56]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

mobilenet_7.h5_EngageWild_4_std_single_attention_balanced


In [57]:
CONCATENATE_STAT = True

In [58]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5641, 128, 2048) (5641,)
(2092, 128, 2048) (2092,)


In [59]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2049      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [60]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
45/45 [==============================] - 3s 60ms/step - loss: 0.0497 - mae: 0.1738 - val_loss: 0.0687 - val_mae: 0.2079
Epoch 2/20
45/45 [==============================] - 1s 26ms/step - loss: 0.0171 - mae: 0.1051 - val_loss: 0.0673 - val_mae: 0.2081
Epoch 3/20
45/45 [==============================] - 1s 25ms/step - loss: 0.0086 - mae: 0.0705 - val_loss: 0.0686 - val_mae: 0.2080
Epoch 4/20
45/45 [==============================] - 1s 25ms/step - loss: 0.0052 - mae: 0.0536 - val_loss: 0.0678 - val_mae: 0.2071
Epoch 5/20
45/45 [==============================] - 1s 25ms/step - loss: 0.0037 - mae: 0.0443 - val_loss: 0.0726 - val_mae: 0.2110
Epoch 6/20
45/45 [==============================] - 1s 26ms/step - loss: 0.0030 - mae: 0.0398 - val_loss: 0.0736 - val_mae: 0.2154
Epoch 7/20
45/45 [==============================] - 1s 25ms/step - loss: 0.0026 - mae: 0.0379 - val_loss: 0.0758 - val_mae: 0.2195
Epoch 8/20
45/45 [==============================] - 1s 25ms/step - loss: 0.0020 - m

In [61]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3864770/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [62]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_std_single_attention_balanced Accuracy:  0.4583333333333333 MSE:  0.06568898942859348 UAR:  0.3814229249011858 Recall:  N/A Precision:  N/A F1:  0.3470916568742656


In [63]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [64]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3864770/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [65]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_std_single_attention_balanced_best Accuracy:  0.5208333333333334 MSE:  0.05547004883940995 UAR:  0.4627799736495389 Recall:  N/A Precision:  N/A F1:  0.4724120082815735


##### Self attention

In [66]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

mobilenet_7.h5_EngageWild_4_std_self_attention_balanced


In [67]:
CONCATENATE_STAT = True

In [68]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5641, 128, 2048) (5641,)
(2092, 128, 2048) (2092,)


In [69]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 attention_1 (Attention)     (None, None, 2048)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [70]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
45/45 [==============================] - 5s 94ms/step - loss: 0.2169 - mae: 0.3773 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 2/20
45/45 [==============================] - 3s 63ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 3/20
45/45 [==============================] - 3s 63ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 4/20
45/45 [==============================] - 3s 62ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 5/20
45/45 [==============================] - 3s 62ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 6/20
45/45 [==============================] - 3s 63ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 7/20
45/45 [==============================] - 3s 63ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 8/20
45/45 [==============================] - 3s 62ms/step - loss: 0.2202 - m

In [71]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3864770/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [72]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_std_self_attention_balanced Accuracy:  0.22916666666666666 MSE:  0.2207645833333333 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.09322033898305083


In [73]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [74]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3864770/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [75]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_std_self_attention_balanced_best Accuracy:  0.22916666666666666 MSE:  0.2207645833333333 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.09322033898305083


### Max

In [27]:
stat_func = np.max
stat_name = "max"

#### Traditional split

In [77]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [78]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [79]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0.0 5/147: 3.401360544217687%
0.33 35/147: 23.80952380952381%
0.66 79/147: 53.74149659863946%
1.0 28/147: 19.047619047619047%
val:
0.0 4/48: 8.333333333333334%
0.33 10/48: 20.833333333333332%
0.66 19/48: 39.583333333333336%
1.0 15/48: 31.25%


##### Single attention

In [80]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

mobilenet_7.h5_EngageWild_4_max_single_attention_traditional


In [81]:
CONCATENATE_STAT = True

In [82]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 2048) (5449,)
(2284, 128, 2048) (2284,)


In [83]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2049      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [84]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 4s 89ms/step - loss: 0.2723 - mae: 0.4406 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 2/20
43/43 [==============================] - 1s 26ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 3/20
43/43 [==============================] - 1s 26ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 4/20
43/43 [==============================] - 1s 26ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 5/20
43/43 [==============================] - 1s 26ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 6/20
43/43 [==============================] - 1s 26ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 7/20
43/43 [==============================] - 1s 34ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 8/20
43/43 [==============================] - 1s 27ms/step - loss: 0.2089 - m

In [85]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3864770/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [86]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_max_single_attention_traditional Accuracy:  0.3125 MSE:  0.2226124206764275 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.11904761904761904


In [87]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [88]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3864770/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [89]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_max_single_attention_traditional_best Accuracy:  0.3125 MSE:  0.2226123397141818 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.11904761904761904


##### Self attention

In [90]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

mobilenet_7.h5_EngageWild_4_max_self_attention_traditional


In [91]:
CONCATENATE_STAT = True

In [92]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 2048) (5449,)
(2284, 128, 2048) (2284,)


In [93]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 attention_2 (Attention)     (None, None, 2048)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [94]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 7s 131ms/step - loss: 0.4446 - mae: 0.6123 - val_loss: 0.5077 - val_mae: 0.6284
Epoch 2/20
43/43 [==============================] - 3s 65ms/step - loss: 0.4538 - mae: 0.6224 - val_loss: 0.5077 - val_mae: 0.6284
Epoch 3/20
43/43 [==============================] - 3s 65ms/step - loss: 0.4538 - mae: 0.6224 - val_loss: 0.5077 - val_mae: 0.6284
Epoch 4/20
43/43 [==============================] - 3s 65ms/step - loss: 0.4538 - mae: 0.6224 - val_loss: 0.5077 - val_mae: 0.6284
Epoch 5/20
43/43 [==============================] - 3s 65ms/step - loss: 0.4538 - mae: 0.6224 - val_loss: 0.5077 - val_mae: 0.6284
Epoch 6/20
43/43 [==============================] - 3s 65ms/step - loss: 0.4538 - mae: 0.6224 - val_loss: 0.5077 - val_mae: 0.6284
Epoch 7/20
43/43 [==============================] - 3s 65ms/step - loss: 0.4538 - mae: 0.6224 - val_loss: 0.5077 - val_mae: 0.6284
Epoch 8/20
43/43 [==============================] - 3s 65ms/step - loss: 0.4538 - 

In [95]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3864770/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [96]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_max_self_attention_traditional Accuracy:  0.08333333333333333 MSE:  0.5076125 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.038461538461538464


In [97]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [98]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3864770/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [99]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_max_self_attention_traditional_best Accuracy:  0.08333333333333333 MSE:  0.5076125 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.038461538461538464


#### Balanced split

In [28]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [29]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [30]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0.0 5/147: 3.401360544217687%
0.33 35/147: 23.80952380952381%
0.66 79/147: 53.74149659863946%
1.0 28/147: 19.047619047619047%
val:
0.0 4/48: 8.333333333333334%
0.33 10/48: 20.833333333333332%
0.66 19/48: 39.583333333333336%
1.0 15/48: 31.25%


In [31]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.767)

(147,) (147,)
(48,) (48,)
(195,) (195,)
[0.   0.33 0.66 1.  ] [ 9 45 98 43]
195
147
48


In [32]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0.0 6/147: 4.081632653061225%
0.33 34/147: 23.12925170068027%
0.66 75/147: 51.02040816326531%
1.0 32/147: 21.768707482993197%
val:
0.0 3/48: 6.25%
0.33 11/48: 22.916666666666668%
0.66 23/48: 47.916666666666664%
1.0 11/48: 22.916666666666668%


##### Single attention

In [33]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

mobilenet_7.h5_EngageWild_4_max_single_attention_balanced


In [34]:
CONCATENATE_STAT = True

In [35]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5641, 128, 2048) (5641,)
(2092, 128, 2048) (2092,)


In [36]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

2024-07-23 16:10:15.235180: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38431 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:11:00.0, compute capability: 8.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2049      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                              

In [37]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20


2024-07-23 16:10:22.778323: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fb0a4d93cb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-23 16:10:22.778478: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2024-07-23 16:10:22.782701: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-23 16:10:22.813671: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-07-23 16:10:22.865698: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


45/45 [==============================] - 5s 77ms/step - loss: 0.2181 - mae: 0.3792 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 2/20
45/45 [==============================] - 1s 25ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 3/20
45/45 [==============================] - 1s 25ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 4/20
45/45 [==============================] - 1s 26ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 5/20
45/45 [==============================] - 1s 25ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 6/20
45/45 [==============================] - 1s 25ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 7/20
45/45 [==============================] - 1s 25ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 8/20
45/45 [==============================] - 1s 25ms/step - loss: 0.2202 - mae: 0.3811 

In [38]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3881236/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [39]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_max_single_attention_balanced Accuracy:  0.22916666666666666 MSE:  0.2207645833333333 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.09322033898305083


In [40]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [41]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3881236/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [42]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_max_single_attention_balanced_best Accuracy:  0.22916666666666666 MSE:  0.2207645833333333 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.09322033898305083


##### Self attention

In [43]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

mobilenet_7.h5_EngageWild_4_max_self_attention_balanced


In [44]:
CONCATENATE_STAT = True

In [45]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5641, 128, 2048) (5641,)
(2092, 128, 2048) (2092,)


In [46]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 attention (Attention)       (None, None, 2048)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [47]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
45/45 [==============================] - 6s 118ms/step - loss: 0.2176 - mae: 0.3782 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 2/20
45/45 [==============================] - 3s 63ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 3/20
45/45 [==============================] - 3s 63ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 4/20
45/45 [==============================] - 3s 63ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 5/20
45/45 [==============================] - 3s 63ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 6/20
45/45 [==============================] - 3s 63ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 7/20
45/45 [==============================] - 3s 62ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 8/20
45/45 [==============================] - 3s 63ms/step - loss: 0.2202 - 

In [48]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3881236/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [49]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_max_self_attention_balanced Accuracy:  0.22916666666666666 MSE:  0.2207645833333333 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.09322033898305083


In [50]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [51]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3881236/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [52]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_max_self_attention_balanced_best Accuracy:  0.22916666666666666 MSE:  0.2207645833333333 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.09322033898305083


### STAT

In [27]:
stat_func = compute_descriptor
stat_name = "STAT"

#### Traditional split

In [54]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [55]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [56]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0.0 5/147: 3.401360544217687%
0.33 35/147: 23.80952380952381%
0.66 79/147: 53.74149659863946%
1.0 28/147: 19.047619047619047%
val:
0.0 4/48: 8.333333333333334%
0.33 10/48: 20.833333333333332%
0.66 19/48: 39.583333333333336%
1.0 15/48: 31.25%


##### Single attention

In [57]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

mobilenet_7.h5_EngageWild_4_STAT_single_attention_traditional


In [58]:
CONCATENATE_STAT = True

In [59]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 3072) (5449,)
(2284, 128, 3072) (2284,)


In [60]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3072)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              3073      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [61]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 5s 94ms/step - loss: 0.0865 - mae: 0.2254 - val_loss: 0.1025 - val_mae: 0.2601
Epoch 2/20
43/43 [==============================] - 2s 37ms/step - loss: 0.0176 - mae: 0.1052 - val_loss: 0.1070 - val_mae: 0.2633
Epoch 3/20
43/43 [==============================] - 2s 38ms/step - loss: 0.0099 - mae: 0.0749 - val_loss: 0.1113 - val_mae: 0.2645
Epoch 4/20
43/43 [==============================] - 2s 38ms/step - loss: 0.0064 - mae: 0.0582 - val_loss: 0.1196 - val_mae: 0.2699
Epoch 5/20
43/43 [==============================] - 2s 38ms/step - loss: 0.0043 - mae: 0.0473 - val_loss: 0.1217 - val_mae: 0.2721
Epoch 6/20
43/43 [==============================] - 2s 37ms/step - loss: 0.0030 - mae: 0.0385 - val_loss: 0.1276 - val_mae: 0.2794
Epoch 7/20
43/43 [==============================] - 2s 37ms/step - loss: 0.0025 - mae: 0.0363 - val_loss: 0.1302 - val_mae: 0.2828
Epoch 8/20
43/43 [==============================] - 2s 41ms/step - loss: 0.0017 - m

In [62]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3881236/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [63]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_STAT_single_attention_traditional Accuracy:  0.375 MSE:  0.1364128734837705 UAR:  0.3017543859649123 Recall:  N/A Precision:  N/A F1:  0.29008152173913043


In [64]:
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    modelAtn.set_weights(best_model_weights)
    save_weights(modelAtn, weights_name)

In [65]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3881236/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [66]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_STAT_single_attention_traditional_best Accuracy:  0.4791666666666667 MSE:  0.08846163625492254 UAR:  0.34868421052631576 Recall:  N/A Precision:  N/A F1:  0.319724025974026


##### Self attention

In [67]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

mobilenet_7.h5_EngageWild_4_STAT_self_attention_traditional


In [68]:
CONCATENATE_STAT = True

In [69]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 3072) (5449,)
(2284, 128, 3072) (2284,)


In [70]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3072)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 3072)           9437184   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 3072)           9437184   ['image_set[0][0]']           
                                                                                                  
 attention_1 (Attention)     (None, None, 3072)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [71]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 10s 208ms/step - loss: 0.2060 - mae: 0.3745 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 2/20
43/43 [==============================] - 5s 123ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 3/20
43/43 [==============================] - 5s 123ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 4/20
43/43 [==============================] - 5s 122ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 5/20
43/43 [==============================] - 5s 122ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 6/20
43/43 [==============================] - 5s 122ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 7/20
43/43 [==============================] - 5s 122ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 8/20
43/43 [==============================] - 5s 122ms/step - loss: 0

In [72]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3881236/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [73]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_STAT_self_attention_traditional Accuracy:  0.3125 MSE:  0.22261249999999996 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.11904761904761904


In [74]:
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    modelAtn.set_weights(best_model_weights)
    save_weights(modelAtn, weights_name)

In [75]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3881236/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [76]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_STAT_self_attention_traditional_best Accuracy:  0.3125 MSE:  0.22261249999999996 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.11904761904761904


#### Balanced split

In [28]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [29]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [30]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0.0 5/147: 3.401360544217687%
0.33 35/147: 23.80952380952381%
0.66 79/147: 53.74149659863946%
1.0 28/147: 19.047619047619047%
val:
0.0 4/48: 8.333333333333334%
0.33 10/48: 20.833333333333332%
0.66 19/48: 39.583333333333336%
1.0 15/48: 31.25%


In [31]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.767)

(147, 2048) (147,)
(48, 2048) (48,)
(195, 2048) (195,)
[0.   0.33 0.66 1.  ] [ 9 45 98 43]
195
147
48


In [32]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0.0 6/147: 4.081632653061225%
0.33 34/147: 23.12925170068027%
0.66 75/147: 51.02040816326531%
1.0 32/147: 21.768707482993197%
val:
0.0 3/48: 6.25%
0.33 11/48: 22.916666666666668%
0.66 23/48: 47.916666666666664%
1.0 11/48: 22.916666666666668%


##### Single attention

In [33]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

mobilenet_7.h5_EngageWild_4_STAT_single_attention_balanced


In [34]:
CONCATENATE_STAT = True

In [35]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5641, 128, 3072) (5641,)
(2092, 128, 3072) (2092,)


In [36]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

2024-07-23 16:25:00.604515: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38431 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:11:00.0, compute capability: 8.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3072)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              3073      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                              

In [37]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20


2024-07-23 16:25:11.002238: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f0bdc043370 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-23 16:25:11.002261: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2024-07-23 16:25:11.007070: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-23 16:25:11.041331: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-07-23 16:25:11.093106: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


45/45 [==============================] - 7s 116ms/step - loss: 0.0500 - mae: 0.1739 - val_loss: 0.0696 - val_mae: 0.2126
Epoch 2/20
45/45 [==============================] - 2s 36ms/step - loss: 0.0149 - mae: 0.0970 - val_loss: 0.0690 - val_mae: 0.2066
Epoch 3/20
45/45 [==============================] - 2s 36ms/step - loss: 0.0074 - mae: 0.0646 - val_loss: 0.0697 - val_mae: 0.2066
Epoch 4/20
45/45 [==============================] - 2s 35ms/step - loss: 0.0047 - mae: 0.0493 - val_loss: 0.0718 - val_mae: 0.2080
Epoch 5/20
45/45 [==============================] - 2s 36ms/step - loss: 0.0037 - mae: 0.0436 - val_loss: 0.0804 - val_mae: 0.2211
Epoch 6/20
45/45 [==============================] - 2s 35ms/step - loss: 0.0030 - mae: 0.0397 - val_loss: 0.0823 - val_mae: 0.2239
Epoch 7/20
45/45 [==============================] - 2s 35ms/step - loss: 0.0025 - mae: 0.0366 - val_loss: 0.0831 - val_mae: 0.2199
Epoch 8/20
45/45 [==============================] - 2s 35ms/step - loss: 0.0016 - mae: 0.0273

In [38]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3894676/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [39]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_STAT_single_attention_balanced Accuracy:  0.4166666666666667 MSE:  0.07565788414361008 UAR:  0.3596837944664032 Recall:  N/A Precision:  N/A F1:  0.3188892948483741


In [40]:
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    modelAtn.set_weights(best_model_weights)
    save_weights(modelAtn, weights_name)

In [41]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3894676/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [42]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_STAT_single_attention_balanced_best Accuracy:  0.5625 MSE:  0.056693993648760294 UAR:  0.4963768115942029 Recall:  N/A Precision:  N/A F1:  0.5041149068322982


##### Self attention

In [43]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

mobilenet_7.h5_EngageWild_4_STAT_self_attention_balanced


In [44]:
CONCATENATE_STAT = True

In [45]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5641, 128, 3072) (5641,)
(2092, 128, 3072) (2092,)


In [46]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3072)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 3072)           9437184   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 3072)           9437184   ['image_set[0][0]']           
                                                                                                  
 attention (Attention)       (None, None, 3072)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [47]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
45/45 [==============================] - 10s 204ms/step - loss: 0.2163 - mae: 0.3772 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 2/20
45/45 [==============================] - 5s 117ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 3/20
45/45 [==============================] - 5s 117ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 4/20
45/45 [==============================] - 5s 117ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 5/20
45/45 [==============================] - 5s 117ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 6/20
45/45 [==============================] - 5s 118ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 7/20
45/45 [==============================] - 5s 118ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 8/20
45/45 [==============================] - 5s 117ms/step - loss: 0

In [48]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3894676/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [49]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_STAT_self_attention_balanced Accuracy:  0.22916666666666666 MSE:  0.2207645833333333 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.09322033898305083


In [50]:
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    modelAtn.set_weights(best_model_weights)
    save_weights(modelAtn, weights_name)

In [51]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3894676/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [52]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_STAT_self_attention_balanced_best Accuracy:  0.22916666666666666 MSE:  0.2207645833333333 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.09322033898305083


# EngageWild dataset (2 classes)

In [24]:
FEATURES_DIR = DATA_DIR + 'features_EngageWild/'
WEIGHTS_DIR = DATA_DIR + 'weights_EngageWild/'
TABLE_NAME = '02_EngageWild_2_classes.xlsx'

## enet_b0_8_best_afew

In [25]:
base_model_key = 'enet_b0_8_best_afew.pt'

In [26]:
ENGAGE_WILD_DATASET, BIN_CLASSIFICATION, N_CLASSES, DATASET_NAME, BASE_DATASET_DIR, ext = define_dataset(True, True)
video2label, labels_list = get_video2label()
print(labels_list)
print(video2label)

['engaged', 'distracted']
{'subject_1_Vid_1': 0, 'subject_1_Vid_2': 0, 'subject_1_Vid_3': 0, 'subject_1_Vid_4': 0, 'subject_1_Vid_5': 0, 'subject_31_Vid_6': 0, 'subject_2_Vid_6': 1, 'subject_3_Vid_6': 0, 'subject_3_Vid_1': 1, 'subject_3_Vid_2': 1, 'subject_3_Vid_3': 0, 'subject_3_Vid_4': 0, 'subject_3_Vid_5': 1, 'subject_3_Vid_7': 0, 'subject_4_Vid_6': 0, 'subject_5_Vid_6': 0, 'subject_6_Vid_6': 1, 'subject_7_Vid_1': 0, 'subject_7_Vid_2': 0, 'subject_7_Vid_3': 0, 'subject_7_Vid_4': 0, 'subject_7_Vid_5': 1, 'subject_8_Vid_6': 0, 'subject_9_Vid_6': 1, 'subject_10_Vid_6': 0, 'subject_11_Vid_6': 0, 'subject_12_Vid_6': 1, 'subject_13_Vid_6': 0, 'subject_14_Vid_6': 0, 'subject_15_Vid_6': 0, 'subject_16_Vid_6': 1, 'subject_17_Vid_6': 0, 'subject_18_Vid_6': 1, 'subject_19_Vid_6': 0, 'subject_20_Vid_6': 0, 'subject_20_Vid_1': 0, 'subject_20_Vid_2': 0, 'subject_20_Vid_3': 0, 'subject_20_Vid_4': 1, 'subject_20_Vid_5': 0, 'subject_20_Vid_5_1': 0, 'subject_20_Vid_5_2': 0, 'subject_20_Vid_7': 1, 'su

### Std

In [27]:
stat_func = np.std
stat_name = "std"

#### Traditional split

In [29]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [30]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [31]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 107/147: 72.78911564625851%
1 40/147: 27.210884353741495%
val:
0 34/48: 70.83333333333333%
1 14/48: 29.166666666666668%


##### Single attention

In [32]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_2_std_single_attention_traditional


In [33]:
CONCATENATE_STAT = True

In [34]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 2560) (5449,)
(2284, 128, 2560) (2284,)


In [35]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

2024-07-30 07:06:57.092910: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38431 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:11:00.0, compute capability: 8.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2561      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                              

In [36]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20


2024-07-30 07:07:06.933872: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f07b81ff2b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-30 07:07:06.934044: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2024-07-30 07:07:06.938196: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-30 07:07:06.993674: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-07-30 07:07:07.046402: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


43/43 [==============================] - 8s 109ms/step - loss: 0.3125 - acc: 0.8807 - auc: 0.9209 - binary_accuracy: 0.8807 - recall: 0.6660 - precision: 0.8886 - val_loss: 0.7877 - val_acc: 0.6432 - val_auc: 0.6720 - val_binary_accuracy: 0.6432 - val_recall: 0.5172 - val_precision: 0.4822
Epoch 2/20
43/43 [==============================] - 1s 33ms/step - loss: 0.1500 - acc: 0.9453 - auc: 0.9848 - binary_accuracy: 0.9453 - recall: 0.8760 - precision: 0.9285 - val_loss: 0.8964 - val_acc: 0.7229 - val_auc: 0.7030 - val_binary_accuracy: 0.7229 - val_recall: 0.5019 - val_precision: 0.6195
Epoch 3/20
43/43 [==============================] - 1s 33ms/step - loss: 0.0845 - acc: 0.9754 - auc: 0.9969 - binary_accuracy: 0.9754 - recall: 0.9409 - precision: 0.9721 - val_loss: 0.9080 - val_acc: 0.7053 - val_auc: 0.7138 - val_binary_accuracy: 0.7053 - val_recall: 0.5873 - val_precision: 0.5691
Epoch 4/20
43/43 [==============================] - 1s 32ms/step - loss: 0.0552 - acc: 0.9840 - auc: 0.9991

In [37]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4014914/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [38]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_single_attention_traditional Accuracy:  0.5416666666666666 MSE:  0.4583333333333333 UAR:  0.5504201680672269 Recall:  0.5714285714285714 Precision:  0.3333333333333333 F1:  0.4210526315789474
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_single_attention_traditional Accuracy:  0.625 MSE:  0.375 UAR:  0.6092436974789917 Recall:  0.5714285714285714 Precision:  0.4 F1:  0.47058823529411764
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_single_attention_traditional Accuracy:  0.6458333333333334 MSE:  0.3541666666666667 UAR:  0.6239495798319328 Recall:  0.5714285714285714 Precision:  0.42105263157894735 F1:  0.48484848484848486
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_single_attention_traditional Accuracy:  0.6875 MSE:  0.3125 UAR:  0.6533613445378151 Recall:  0.5714285714285714 Precision:  0.47058823529411764 F1:  0.5161290322580646
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_single_attention_tradit

In [39]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [40]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4014914/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [41]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_single_attention_traditional_best Accuracy:  0.5208333333333334 MSE:  0.4791666666666667 UAR:  0.5777310924369747 Recall:  0.7142857142857143 Precision:  0.3448275862068966 F1:  0.46511627906976755
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_single_attention_traditional_best Accuracy:  0.6041666666666666 MSE:  0.3958333333333333 UAR:  0.5735294117647058 Recall:  0.5 Precision:  0.3684210526315789 F1:  0.4242424242424242
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_single_attention_traditional_best Accuracy:  0.625 MSE:  0.375 UAR:  0.5882352941176471 Recall:  0.5 Precision:  0.3888888888888889 F1:  0.43750000000000006
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_single_attention_traditional_best Accuracy:  0.6458333333333334 MSE:  0.3541666666666667 UAR:  0.6029411764705883 Recall:  0.5 Precision:  0.4117647058823529 F1:  0.45161290322580644
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_single_att

##### Self attention

In [42]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_traditional


In [43]:
CONCATENATE_STAT = True

In [44]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 2560) (5449,)
(2284, 128, 2560) (2284,)


In [45]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention (Attention)       (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [46]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 7s 140ms/step - loss: 0.5918 - acc: 0.7847 - auc: 0.7871 - binary_accuracy: 0.7847 - recall_1: 0.5607 - precision_1: 0.6410 - val_loss: 0.8495 - val_acc: 0.6475 - val_auc: 0.6537 - val_binary_accuracy: 0.6475 - val_recall_1: 0.5758 - val_precision_1: 0.4892
Epoch 2/20
43/43 [==============================] - 4s 93ms/step - loss: 0.2312 - acc: 0.9095 - auc: 0.9594 - binary_accuracy: 0.9095 - recall_1: 0.7868 - precision_1: 0.8838 - val_loss: 0.7742 - val_acc: 0.7408 - val_auc: 0.7044 - val_binary_accuracy: 0.7408 - val_recall_1: 0.5057 - val_precision_1: 0.6606
Epoch 3/20
43/43 [==============================] - 4s 103ms/step - loss: 0.1426 - acc: 0.9549 - auc: 0.9865 - binary_accuracy: 0.9549 - recall_1: 0.8940 - precision_1: 0.9450 - val_loss: 1.0746 - val_acc: 0.6756 - val_auc: 0.7150 - val_binary_accuracy: 0.6756 - val_recall_1: 0.2140 - val_precision_1: 0.5753
Epoch 4/20
43/43 [==============================] - 4s 93ms/step - loss

In [47]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4014914/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [48]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_traditional Accuracy:  0.8125 MSE:  0.1875 UAR:  0.7626050420168067 Recall:  0.6428571428571429 Precision:  0.6923076923076923 F1:  0.6666666666666666
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_traditional Accuracy:  0.8333333333333334 MSE:  0.16666666666666666 UAR:  0.7773109243697479 Recall:  0.6428571428571429 Precision:  0.75 F1:  0.6923076923076924
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_traditional Accuracy:  0.8333333333333334 MSE:  0.16666666666666666 UAR:  0.7773109243697479 Recall:  0.6428571428571429 Precision:  0.75 F1:  0.6923076923076924
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_traditional Accuracy:  0.8333333333333334 MSE:  0.16666666666666666 UAR:  0.7773109243697479 Recall:  0.6428571428571429 Precision:  0.75 F1:  0.6923076923076924
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_traditional Accura

In [49]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [50]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4014914/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [51]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_traditional_best Accuracy:  0.5625 MSE:  0.4375 UAR:  0.6281512605042017 Recall:  0.7857142857142857 Precision:  0.3793103448275862 F1:  0.5116279069767441
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_traditional_best Accuracy:  0.7291666666666666 MSE:  0.2708333333333333 UAR:  0.703781512605042 Recall:  0.6428571428571429 Precision:  0.5294117647058824 F1:  0.5806451612903226
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_traditional_best Accuracy:  0.7708333333333334 MSE:  0.22916666666666666 UAR:  0.7331932773109244 Recall:  0.6428571428571429 Precision:  0.6 F1:  0.6206896551724138
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_traditional_best Accuracy:  0.8125 MSE:  0.1875 UAR:  0.7626050420168067 Recall:  0.6428571428571429 Precision:  0.6923076923076923 F1:  0.6666666666666666
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_self_attentio

#### Balanced split

In [52]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [53]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [54]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 107/147: 72.78911564625851%
1 40/147: 27.210884353741495%
val:
0 34/48: 70.83333333333333%
1 14/48: 29.166666666666668%


In [55]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.759)

(147,) (147,)
(48,) (48,)
(195,) (195,)
[0 1] [141  54]
195
147
48


In [56]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 107/147: 72.78911564625851%
1 40/147: 27.210884353741495%
val:
0 34/48: 70.83333333333333%
1 14/48: 29.166666666666668%


##### Single attention

In [57]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_2_std_single_attention_balanced


In [58]:
CONCATENATE_STAT = True

In [59]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5631, 128, 2560) (5631,)
(2102, 128, 2560) (2102,)


In [60]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2561      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [61]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
44/44 [==============================] - 5s 83ms/step - loss: 0.3716 - acc: 0.8430 - auc: 0.8928 - binary_accuracy: 0.8430 - recall_2: 0.6052 - precision_2: 0.8311 - val_loss: 0.7749 - val_acc: 0.6698 - val_auc: 0.6856 - val_binary_accuracy: 0.6698 - val_recall_2: 0.4567 - val_precision_2: 0.4538
Epoch 2/20
44/44 [==============================] - 1s 32ms/step - loss: 0.1450 - acc: 0.9563 - auc: 0.9877 - binary_accuracy: 0.9563 - recall_2: 0.8910 - precision_2: 0.9620 - val_loss: 0.9226 - val_acc: 0.6613 - val_auc: 0.6917 - val_binary_accuracy: 0.6613 - val_recall_2: 0.5039 - val_precision_2: 0.4463
Epoch 3/20
44/44 [==============================] - 1s 32ms/step - loss: 0.0865 - acc: 0.9746 - auc: 0.9960 - binary_accuracy: 0.9746 - recall_2: 0.9426 - precision_2: 0.9728 - val_loss: 1.1081 - val_acc: 0.6613 - val_auc: 0.7015 - val_binary_accuracy: 0.6613 - val_recall_2: 0.4945 - val_precision_2: 0.4454
Epoch 4/20
44/44 [==============================] - 1s 32ms/step - loss: 

In [62]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4014914/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [63]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_single_attention_balanced Accuracy:  0.6875 MSE:  0.3125 UAR:  0.6323529411764706 Recall:  0.5 Precision:  0.4666666666666667 F1:  0.4827586206896552
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_single_attention_balanced Accuracy:  0.6875 MSE:  0.3125 UAR:  0.6323529411764706 Recall:  0.5 Precision:  0.4666666666666667 F1:  0.4827586206896552
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_single_attention_balanced Accuracy:  0.6875 MSE:  0.3125 UAR:  0.6323529411764706 Recall:  0.5 Precision:  0.4666666666666667 F1:  0.4827586206896552
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_single_attention_balanced Accuracy:  0.6875 MSE:  0.3125 UAR:  0.6323529411764706 Recall:  0.5 Precision:  0.4666666666666667 F1:  0.4827586206896552
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_single_attention_balanced Accuracy:  0.7083333333333334 MSE:  0.2916666666666667 UAR:  0.6470588235294117 Recall:  0.5 Precision: 

In [64]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [65]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4014914/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [66]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_single_attention_balanced_best Accuracy:  0.625 MSE:  0.375 UAR:  0.6932773109243697 Recall:  0.8571428571428571 Precision:  0.42857142857142855 F1:  0.5714285714285714
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_single_attention_balanced_best Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.7016806722689075 Recall:  0.7857142857142857 Precision:  0.4583333333333333 F1:  0.5789473684210527
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_single_attention_balanced_best Accuracy:  0.6458333333333334 MSE:  0.3541666666666667 UAR:  0.6449579831932774 Recall:  0.6428571428571429 Precision:  0.42857142857142855 F1:  0.5142857142857143
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_single_attention_balanced_best Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.5966386554621849 Recall:  0.42857142857142855 Precision:  0.42857142857142855 F1:  0.42857142857142855
Metric_name:  enet_b0_8_best_

##### Self attention

In [67]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_balanced


In [68]:
CONCATENATE_STAT = True

In [69]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5631, 128, 2560) (5631,)
(2102, 128, 2560) (2102,)


In [70]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention_1 (Attention)     (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [71]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
44/44 [==============================] - 8s 168ms/step - loss: 0.5993 - acc: 0.7878 - auc: 0.8036 - binary_accuracy: 0.7878 - recall_3: 0.6098 - precision_3: 0.6631 - val_loss: 0.6822 - val_acc: 0.6817 - val_auc: 0.6883 - val_binary_accuracy: 0.6817 - val_recall_3: 0.2850 - val_precision_3: 0.4571
Epoch 2/20
44/44 [==============================] - 4s 91ms/step - loss: 0.2049 - acc: 0.9215 - auc: 0.9697 - binary_accuracy: 0.9215 - recall_3: 0.8278 - precision_3: 0.9052 - val_loss: 0.8615 - val_acc: 0.7122 - val_auc: 0.6816 - val_binary_accuracy: 0.7122 - val_recall_3: 0.3953 - val_precision_3: 0.5318
Epoch 3/20
44/44 [==============================] - 4s 90ms/step - loss: 0.1210 - acc: 0.9549 - auc: 0.9905 - binary_accuracy: 0.9549 - recall_3: 0.9086 - precision_3: 0.9406 - val_loss: 0.8721 - val_acc: 0.7393 - val_auc: 0.7289 - val_binary_accuracy: 0.7393 - val_recall_3: 0.5764 - val_precision_3: 0.5674
Epoch 4/20
44/44 [==============================] - 4s 92ms/step - loss:

In [72]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4014914/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [73]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_balanced Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.6176470588235294 Recall:  0.5 Precision:  0.4375 F1:  0.4666666666666667
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_balanced Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.6176470588235294 Recall:  0.5 Precision:  0.4375 F1:  0.4666666666666667
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_balanced Accuracy:  0.6875 MSE:  0.3125 UAR:  0.6323529411764706 Recall:  0.5 Precision:  0.4666666666666667 F1:  0.4827586206896552
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_balanced Accuracy:  0.7083333333333334 MSE:  0.2916666666666667 UAR:  0.6470588235294117 Recall:  0.5 Precision:  0.5 F1:  0.5
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_balanced Accuracy:  0.7083333333333334 MSE:  0.2916666666666667 UAR:  0.6470588235294117 Recall:  0.5 Pre

In [74]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [75]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4014914/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [76]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_balanced_best Accuracy:  0.6041666666666666 MSE:  0.3958333333333333 UAR:  0.6785714285714286 Recall:  0.8571428571428571 Precision:  0.41379310344827586 F1:  0.5581395348837208
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_balanced_best Accuracy:  0.6875 MSE:  0.3125 UAR:  0.7163865546218487 Recall:  0.7857142857142857 Precision:  0.4782608695652174 F1:  0.5945945945945946
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_balanced_best Accuracy:  0.6875 MSE:  0.3125 UAR:  0.6533613445378151 Recall:  0.5714285714285714 Precision:  0.47058823529411764 F1:  0.5161290322580646
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_balanced_best Accuracy:  0.7083333333333334 MSE:  0.2916666666666667 UAR:  0.6050420168067226 Recall:  0.35714285714285715 Precision:  0.5 F1:  0.41666666666666663
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_balanc

### Max

In [38]:
stat_func = np.max
stat_name = "max"

#### Traditional split

In [39]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [40]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [41]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 107/147: 72.78911564625851%
1 40/147: 27.210884353741495%
val:
0 34/48: 70.83333333333333%
1 14/48: 29.166666666666668%


##### Single attention

In [42]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_traditional


In [43]:
CONCATENATE_STAT = True

In [44]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 2560) (5449,)
(2284, 128, 2560) (2284,)


In [45]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

2024-07-30 07:17:43.974052: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38431 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:11:00.0, compute capability: 8.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2561      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                              

In [46]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20


2024-07-30 07:17:53.529710: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fbadc14df40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-30 07:17:53.530074: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2024-07-30 07:17:53.534425: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-30 07:17:53.581539: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-07-30 07:17:53.646642: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


43/43 [==============================] - 8s 110ms/step - loss: 0.3723 - acc: 0.8541 - auc: 0.8832 - binary_accuracy: 0.8541 - recall: 0.6513 - precision: 0.8009 - val_loss: 0.7008 - val_acc: 0.7452 - val_auc: 0.7242 - val_binary_accuracy: 0.7452 - val_recall: 0.5045 - val_precision: 0.6723
Epoch 2/20
43/43 [==============================] - 2s 38ms/step - loss: 0.1272 - acc: 0.9638 - auc: 0.9940 - binary_accuracy: 0.9638 - recall: 0.9056 - precision: 0.9658 - val_loss: 0.7590 - val_acc: 0.7303 - val_auc: 0.7296 - val_binary_accuracy: 0.7303 - val_recall: 0.5682 - val_precision: 0.6169
Epoch 3/20
43/43 [==============================] - 1s 33ms/step - loss: 0.0557 - acc: 0.9890 - auc: 0.9997 - binary_accuracy: 0.9890 - recall: 0.9769 - precision: 0.9845 - val_loss: 0.8781 - val_acc: 0.7570 - val_auc: 0.7387 - val_binary_accuracy: 0.7570 - val_recall: 0.5682 - val_precision: 0.6737
Epoch 4/20
43/43 [==============================] - 1s 32ms/step - loss: 0.0225 - acc: 1.0000 - auc: 1.0000

In [47]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4027200/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [48]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_traditional Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.6386554621848739 Recall:  0.5714285714285714 Precision:  0.4444444444444444 F1:  0.5
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_traditional Accuracy:  0.7083333333333334 MSE:  0.2916666666666667 UAR:  0.6470588235294117 Recall:  0.5 Precision:  0.5 F1:  0.5
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_traditional Accuracy:  0.7083333333333334 MSE:  0.2916666666666667 UAR:  0.6470588235294117 Recall:  0.5 Precision:  0.5 F1:  0.5
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_traditional Accuracy:  0.7708333333333334 MSE:  0.22916666666666666 UAR:  0.6911764705882353 Recall:  0.5 Precision:  0.6363636363636364 F1:  0.56
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_traditional Accuracy:  0.7708333333333334 MSE:  0.22916666666666666 UAR:  0.69

In [49]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [50]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4027200/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [51]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_traditional_best Accuracy:  0.5208333333333334 MSE:  0.4791666666666667 UAR:  0.5357142857142857 Recall:  0.5714285714285714 Precision:  0.32 F1:  0.41025641025641024
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_traditional_best Accuracy:  0.5833333333333334 MSE:  0.4166666666666667 UAR:  0.5588235294117647 Recall:  0.5 Precision:  0.35 F1:  0.4117647058823529
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_traditional_best Accuracy:  0.6041666666666666 MSE:  0.3958333333333333 UAR:  0.5735294117647058 Recall:  0.5 Precision:  0.3684210526315789 F1:  0.4242424242424242
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_traditional_best Accuracy:  0.7291666666666666 MSE:  0.2708333333333333 UAR:  0.6617647058823529 Recall:  0.5 Precision:  0.5384615384615384 F1:  0.5185185185185186
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_single_attenti

##### Self attention

In [52]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_traditional


In [53]:
CONCATENATE_STAT = True

In [54]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 2560) (5449,)
(2284, 128, 2560) (2284,)


In [55]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention (Attention)       (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [56]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 8s 155ms/step - loss: 1.5761 - acc: 0.7234 - auc: 0.7081 - binary_accuracy: 0.7234 - recall_1: 0.5414 - precision_1: 0.5153 - val_loss: 1.2094 - val_acc: 0.6493 - val_auc: 0.6822 - val_binary_accuracy: 0.6493 - val_recall_1: 0.1108 - val_precision_1: 0.4579
Epoch 2/20
43/43 [==============================] - 4s 94ms/step - loss: 0.2213 - acc: 0.9189 - auc: 0.9613 - binary_accuracy: 0.9189 - recall_1: 0.8182 - precision_1: 0.8890 - val_loss: 0.8240 - val_acc: 0.6528 - val_auc: 0.6992 - val_binary_accuracy: 0.6528 - val_recall_1: 0.3261 - val_precision_1: 0.4923
Epoch 3/20
43/43 [==============================] - 4s 93ms/step - loss: 0.1190 - acc: 0.9660 - auc: 0.9940 - binary_accuracy: 0.9660 - recall_1: 0.9249 - precision_1: 0.9549 - val_loss: 0.9960 - val_acc: 0.6909 - val_auc: 0.6846 - val_binary_accuracy: 0.6909 - val_recall_1: 0.2586 - val_precision_1: 0.6208
Epoch 4/20
43/43 [==============================] - 4s 92ms/step - loss:

In [57]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4027200/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [58]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_traditional Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.6176470588235294 Recall:  0.5 Precision:  0.4375 F1:  0.4666666666666667
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_traditional Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.5966386554621849 Recall:  0.42857142857142855 Precision:  0.42857142857142855 F1:  0.42857142857142855
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_traditional Accuracy:  0.7083333333333334 MSE:  0.2916666666666667 UAR:  0.6260504201680672 Recall:  0.42857142857142855 Precision:  0.5 F1:  0.4615384615384615
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_traditional Accuracy:  0.75 MSE:  0.25 UAR:  0.6554621848739496 Recall:  0.42857142857142855 Precision:  0.6 F1:  0.5
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_traditional Accuracy:  0.75 MSE:  0.25 UAR:  0.655

In [59]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [60]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4027200/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [61]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_traditional_best Accuracy:  0.6041666666666666 MSE:  0.3958333333333333 UAR:  0.6785714285714286 Recall:  0.8571428571428571 Precision:  0.41379310344827586 F1:  0.5581395348837208
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_traditional_best Accuracy:  0.6875 MSE:  0.3125 UAR:  0.6533613445378151 Recall:  0.5714285714285714 Precision:  0.47058823529411764 F1:  0.5161290322580646
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_traditional_best Accuracy:  0.7083333333333334 MSE:  0.2916666666666667 UAR:  0.6470588235294117 Recall:  0.5 Precision:  0.5 F1:  0.5
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_traditional_best Accuracy:  0.7083333333333334 MSE:  0.2916666666666667 UAR:  0.6260504201680672 Recall:  0.42857142857142855 Precision:  0.5 F1:  0.4615384615384615
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_traditional_best

#### Balanced split

In [62]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [63]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [64]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 107/147: 72.78911564625851%
1 40/147: 27.210884353741495%
val:
0 34/48: 70.83333333333333%
1 14/48: 29.166666666666668%


In [65]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.759)

(147,) (147,)
(48,) (48,)
(195,) (195,)
[0 1] [141  54]
195
147
48


In [66]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 107/147: 72.78911564625851%
1 40/147: 27.210884353741495%
val:
0 34/48: 70.83333333333333%
1 14/48: 29.166666666666668%


##### Single attention

In [67]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_balanced


In [68]:
CONCATENATE_STAT = True

In [69]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5631, 128, 2560) (5631,)
(2102, 128, 2560) (2102,)


In [70]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2561      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [71]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
44/44 [==============================] - 4s 74ms/step - loss: 0.4721 - acc: 0.7949 - auc: 0.8205 - binary_accuracy: 0.7949 - recall_2: 0.5489 - precision_2: 0.7088 - val_loss: 0.6953 - val_acc: 0.5951 - val_auc: 0.7154 - val_binary_accuracy: 0.5951 - val_recall_2: 0.3984 - val_precision_2: 0.3504
Epoch 2/20
44/44 [==============================] - 1s 32ms/step - loss: 0.1780 - acc: 0.9469 - auc: 0.9814 - binary_accuracy: 0.9469 - recall_2: 0.8541 - precision_2: 0.9668 - val_loss: 0.9529 - val_acc: 0.6066 - val_auc: 0.7030 - val_binary_accuracy: 0.6066 - val_recall_2: 0.4787 - val_precision_2: 0.3800
Epoch 3/20
44/44 [==============================] - 1s 32ms/step - loss: 0.0862 - acc: 0.9798 - auc: 0.9976 - binary_accuracy: 0.9798 - recall_2: 0.9432 - precision_2: 0.9896 - val_loss: 1.1213 - val_acc: 0.6560 - val_auc: 0.6707 - val_binary_accuracy: 0.6560 - val_recall_2: 0.3984 - val_precision_2: 0.4259
Epoch 4/20
44/44 [==============================] - 1s 32ms/step - loss: 

In [72]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4027200/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [73]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_balanced Accuracy:  0.625 MSE:  0.375 UAR:  0.5672268907563025 Recall:  0.42857142857142855 Precision:  0.375 F1:  0.39999999999999997
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_balanced Accuracy:  0.625 MSE:  0.375 UAR:  0.5672268907563025 Recall:  0.42857142857142855 Precision:  0.375 F1:  0.39999999999999997
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_balanced Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.5966386554621849 Recall:  0.42857142857142855 Precision:  0.42857142857142855 F1:  0.42857142857142855
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_balanced Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.5756302521008403 Recall:  0.35714285714285715 Precision:  0.4166666666666667 F1:  0.3846153846153846
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_balanced Accuracy:  0.66666

In [74]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [75]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4027200/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [76]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_balanced_best Accuracy:  0.5416666666666666 MSE:  0.4583333333333333 UAR:  0.634453781512605 Recall:  0.8571428571428571 Precision:  0.375 F1:  0.5217391304347825
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_balanced_best Accuracy:  0.625 MSE:  0.375 UAR:  0.6302521008403361 Recall:  0.6428571428571429 Precision:  0.4090909090909091 F1:  0.5000000000000001
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_balanced_best Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.6596638655462186 Recall:  0.6428571428571429 Precision:  0.45 F1:  0.5294117647058824
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_balanced_best Accuracy:  0.625 MSE:  0.375 UAR:  0.546218487394958 Recall:  0.35714285714285715 Precision:  0.35714285714285715 F1:  0.35714285714285715
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_balanced_best A

##### Self attention

In [77]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_balanced


In [78]:
CONCATENATE_STAT = True

In [79]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5631, 128, 2560) (5631,)
(2102, 128, 2560) (2102,)


In [80]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention_1 (Attention)     (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [81]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
44/44 [==============================] - 8s 168ms/step - loss: 1.2116 - acc: 0.7714 - auc: 0.7812 - binary_accuracy: 0.7714 - recall_3: 0.6333 - precision_3: 0.6206 - val_loss: 1.0200 - val_acc: 0.6042 - val_auc: 0.6351 - val_binary_accuracy: 0.6042 - val_recall_3: 0.5496 - val_precision_3: 0.3899
Epoch 2/20
44/44 [==============================] - 4s 93ms/step - loss: 0.1467 - acc: 0.9515 - auc: 0.9866 - binary_accuracy: 0.9515 - recall_3: 0.8928 - precision_3: 0.9442 - val_loss: 0.9340 - val_acc: 0.6275 - val_auc: 0.6549 - val_binary_accuracy: 0.6275 - val_recall_3: 0.4173 - val_precision_3: 0.3909
Epoch 3/20
44/44 [==============================] - 4s 91ms/step - loss: 0.0680 - acc: 0.9876 - auc: 0.9990 - binary_accuracy: 0.9876 - recall_3: 0.9690 - precision_3: 0.9898 - val_loss: 1.1035 - val_acc: 0.6213 - val_auc: 0.6565 - val_binary_accuracy: 0.6213 - val_recall_3: 0.3732 - val_precision_3: 0.3732
Epoch 4/20
44/44 [==============================] - 5s 103ms/step - loss

In [82]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4027200/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [83]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_balanced Accuracy:  0.625 MSE:  0.375 UAR:  0.5882352941176471 Recall:  0.5 Precision:  0.3888888888888889 F1:  0.43750000000000006
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_balanced Accuracy:  0.6458333333333334 MSE:  0.3541666666666667 UAR:  0.6029411764705883 Recall:  0.5 Precision:  0.4117647058823529 F1:  0.45161290322580644
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_balanced Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.5966386554621849 Recall:  0.42857142857142855 Precision:  0.42857142857142855 F1:  0.42857142857142855
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_balanced Accuracy:  0.7083333333333334 MSE:  0.2916666666666667 UAR:  0.6260504201680672 Recall:  0.42857142857142855 Precision:  0.5 F1:  0.4615384615384615
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_balanced Accuracy:  0.6875 MSE: 

In [84]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [85]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4027200/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [86]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_balanced_best Accuracy:  0.5625 MSE:  0.4375 UAR:  0.5861344537815126 Recall:  0.6428571428571429 Precision:  0.36 F1:  0.4615384615384615
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_balanced_best Accuracy:  0.625 MSE:  0.375 UAR:  0.6302521008403361 Recall:  0.6428571428571429 Precision:  0.4090909090909091 F1:  0.5000000000000001
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_balanced_best Accuracy:  0.625 MSE:  0.375 UAR:  0.6092436974789917 Recall:  0.5714285714285714 Precision:  0.4 F1:  0.47058823529411764
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_balanced_best Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.6386554621848739 Recall:  0.5714285714285714 Precision:  0.4444444444444444 F1:  0.5
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_balanced_best Accuracy:  0.6458333333333334 MSE:  0.35416666666666

### STAT

In [28]:
stat_func = compute_descriptor
stat_name = "STAT"

#### Traditional split

In [29]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [30]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [31]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 107/147: 72.78911564625851%
1 40/147: 27.210884353741495%
val:
0 34/48: 70.83333333333333%
1 14/48: 29.166666666666668%


##### Single attention

In [31]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_2_STAT_single_attention_traditional


In [32]:
CONCATENATE_STAT = True

In [33]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 3840) (5449,)
(2284, 128, 3840) (2284,)


In [34]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

2024-07-30 07:24:49.986587: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38431 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:11:00.0, compute capability: 8.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3840)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              3841      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                              

In [35]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20


2024-07-30 07:25:03.152418: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f2ec8232af0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-30 07:25:03.152594: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2024-07-30 07:25:03.156757: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-30 07:25:03.198857: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-07-30 07:25:03.250678: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


43/43 [==============================] - 10s 159ms/step - loss: 0.4052 - acc: 0.8391 - auc: 0.8601 - binary_accuracy: 0.8391 - recall: 0.5382 - precision: 0.8414 - val_loss: 0.7601 - val_acc: 0.7084 - val_auc: 0.6645 - val_binary_accuracy: 0.7084 - val_recall: 0.2459 - val_precision: 0.7228
Epoch 2/20
43/43 [==============================] - 2s 47ms/step - loss: 0.1796 - acc: 0.9352 - auc: 0.9801 - binary_accuracy: 0.9352 - recall: 0.8343 - precision: 0.9319 - val_loss: 0.9309 - val_acc: 0.6537 - val_auc: 0.6570 - val_binary_accuracy: 0.6537 - val_recall: 0.5070 - val_precision: 0.4963
Epoch 3/20
43/43 [==============================] - 2s 46ms/step - loss: 0.1088 - acc: 0.9635 - auc: 0.9935 - binary_accuracy: 0.9635 - recall: 0.9242 - precision: 0.9467 - val_loss: 1.0701 - val_acc: 0.6821 - val_auc: 0.6794 - val_binary_accuracy: 0.6821 - val_recall: 0.4013 - val_precision: 0.5517
Epoch 4/20
43/43 [==============================] - 2s 47ms/step - loss: 0.0668 - acc: 0.9846 - auc: 0.998

In [36]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4038218/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [37]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_single_attention_traditional Accuracy:  0.5208333333333334 MSE:  0.4791666666666667 UAR:  0.5357142857142857 Recall:  0.5714285714285714 Precision:  0.32 F1:  0.41025641025641024
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_single_attention_traditional Accuracy:  0.5416666666666666 MSE:  0.4583333333333333 UAR:  0.5504201680672269 Recall:  0.5714285714285714 Precision:  0.3333333333333333 F1:  0.4210526315789474
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_single_attention_traditional Accuracy:  0.6041666666666666 MSE:  0.3958333333333333 UAR:  0.5945378151260504 Recall:  0.5714285714285714 Precision:  0.38095238095238093 F1:  0.4571428571428571
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_single_attention_traditional Accuracy:  0.5833333333333334 MSE:  0.4166666666666667 UAR:  0.5588235294117647 Recall:  0.5 Precision:  0.35 F1:  0.4117647058823529
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STA

In [38]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [39]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4038218/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [40]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_single_attention_traditional_best Accuracy:  0.5625 MSE:  0.4375 UAR:  0.5651260504201681 Recall:  0.5714285714285714 Precision:  0.34782608695652173 F1:  0.4324324324324324
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_single_attention_traditional_best Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.6176470588235294 Recall:  0.5 Precision:  0.4375 F1:  0.4666666666666667
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_single_attention_traditional_best Accuracy:  0.6875 MSE:  0.3125 UAR:  0.5903361344537815 Recall:  0.35714285714285715 Precision:  0.45454545454545453 F1:  0.4
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_single_attention_traditional_best Accuracy:  0.75 MSE:  0.25 UAR:  0.6134453781512605 Recall:  0.2857142857142857 Precision:  0.6666666666666666 F1:  0.4
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_single_attention_traditional_best Accuracy:  0.7291666666666666 MSE

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### Self attention

In [41]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attention_traditional


In [42]:
CONCATENATE_STAT = True

In [43]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 3840) (5449,)
(2284, 128, 3840) (2284,)


In [44]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3840)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 3840)           1474560   ['image_set[0][0]']           
                                                          0                                       
                                                                                                  
 value (Dense)               (None, None, 3840)           1474560   ['image_set[0][0]']           
                                                          0                                       
                                                                                            

In [45]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 13s 267ms/step - loss: 0.8178 - acc: 0.7937 - auc: 0.7978 - binary_accuracy: 0.7937 - recall_1: 0.6275 - precision_1: 0.6423 - val_loss: 0.9124 - val_acc: 0.7469 - val_auc: 0.6501 - val_binary_accuracy: 0.7469 - val_recall_1: 0.4089 - val_precision_1: 0.7379
Epoch 2/20
43/43 [==============================] - 9s 204ms/step - loss: 0.1831 - acc: 0.9378 - auc: 0.9740 - binary_accuracy: 0.9378 - recall_1: 0.8401 - precision_1: 0.9356 - val_loss: 0.9893 - val_acc: 0.6865 - val_auc: 0.6679 - val_binary_accuracy: 0.6865 - val_recall_1: 0.4255 - val_precision_1: 0.5576
Epoch 3/20
43/43 [==============================] - 8s 178ms/step - loss: 0.1127 - acc: 0.9642 - auc: 0.9914 - binary_accuracy: 0.9642 - recall_1: 0.9274 - precision_1: 0.9463 - val_loss: 1.2325 - val_acc: 0.6773 - val_auc: 0.6565 - val_binary_accuracy: 0.6773 - val_recall_1: 0.4408 - val_precision_1: 0.5373
Epoch 4/20
43/43 [==============================] - 8s 179ms/step - l

In [46]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4038218/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [47]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attention_traditional Accuracy:  0.6458333333333334 MSE:  0.3541666666666667 UAR:  0.6239495798319328 Recall:  0.5714285714285714 Precision:  0.42105263157894735 F1:  0.48484848484848486
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attention_traditional Accuracy:  0.6458333333333334 MSE:  0.3541666666666667 UAR:  0.6029411764705883 Recall:  0.5 Precision:  0.4117647058823529 F1:  0.45161290322580644
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attention_traditional Accuracy:  0.6041666666666666 MSE:  0.3958333333333333 UAR:  0.5315126050420168 Recall:  0.35714285714285715 Precision:  0.3333333333333333 F1:  0.3448275862068965
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attention_traditional Accuracy:  0.6041666666666666 MSE:  0.3958333333333333 UAR:  0.5315126050420168 Recall:  0.35714285714285715 Precision:  0.3333333333333333 F1:  0.3448275862068965
Metric_name:  enet_b0_8_best_a

In [48]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [49]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4038218/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [50]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attention_traditional_best Accuracy:  0.5833333333333334 MSE:  0.4166666666666667 UAR:  0.6008403361344539 Recall:  0.6428571428571429 Precision:  0.375 F1:  0.4736842105263159
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attention_traditional_best Accuracy:  0.6041666666666666 MSE:  0.3958333333333333 UAR:  0.5945378151260504 Recall:  0.5714285714285714 Precision:  0.38095238095238093 F1:  0.4571428571428571
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attention_traditional_best Accuracy:  0.625 MSE:  0.375 UAR:  0.6092436974789917 Recall:  0.5714285714285714 Precision:  0.4 F1:  0.47058823529411764
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attention_traditional_best Accuracy:  0.6458333333333334 MSE:  0.3541666666666667 UAR:  0.6029411764705883 Recall:  0.5 Precision:  0.4117647058823529 F1:  0.45161290322580644
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attent

#### Balanced split

In [32]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [33]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [34]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 107/147: 72.78911564625851%
1 40/147: 27.210884353741495%
val:
0 34/48: 70.83333333333333%
1 14/48: 29.166666666666668%


In [35]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.759)

(147, 2560) (147,)
(48, 2560) (48,)
(195, 2560) (195,)
[0 1] [141  54]
195
147
48


In [36]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 107/147: 72.78911564625851%
1 40/147: 27.210884353741495%
val:
0 34/48: 70.83333333333333%
1 14/48: 29.166666666666668%


##### Single attention

In [37]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_2_STAT_single_attention_balanced


In [38]:
CONCATENATE_STAT = True

In [39]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5631, 128, 3840) (5631,)
(2102, 128, 3840) (2102,)


In [40]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

2024-07-30 07:39:08.167930: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38431 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:11:00.0, compute capability: 8.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3840)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              3841      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                              

In [41]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

In [42]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [43]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_single_attention_balanced Accuracy:  0.6875 MSE:  0.3125 UAR:  0.6533613445378151 Recall:  0.5714285714285714 Precision:  0.47058823529411764 F1:  0.5161290322580646
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_single_attention_balanced Accuracy:  0.7083333333333334 MSE:  0.2916666666666667 UAR:  0.6680672268907563 Recall:  0.5714285714285714 Precision:  0.5 F1:  0.5333333333333333
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_single_attention_balanced Accuracy:  0.6875 MSE:  0.3125 UAR:  0.6323529411764706 Recall:  0.5 Precision:  0.4666666666666667 F1:  0.4827586206896552
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_single_attention_balanced Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.5966386554621849 Recall:  0.42857142857142855 Precision:  0.42857142857142855 F1:  0.42857142857142855
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_single_attention_balanced Accuracy:  0.6666

In [44]:
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    modelAtn.set_weights(best_model_weights)
    save_weights(modelAtn, weights_name)

In [45]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [46]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_single_attention_balanced_best Accuracy:  0.6041666666666666 MSE:  0.3958333333333333 UAR:  0.6785714285714286 Recall:  0.8571428571428571 Precision:  0.41379310344827586 F1:  0.5581395348837208
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_single_attention_balanced_best Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.7016806722689075 Recall:  0.7857142857142857 Precision:  0.4583333333333333 F1:  0.5789473684210527
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_single_attention_balanced_best Accuracy:  0.6458333333333334 MSE:  0.3541666666666667 UAR:  0.6449579831932774 Recall:  0.6428571428571429 Precision:  0.42857142857142855 F1:  0.5142857142857143
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_single_attention_balanced_best Accuracy:  0.7291666666666666 MSE:  0.2708333333333333 UAR:  0.6827731092436975 Recall:  0.5714285714285714 Precision:  0.5333333333333333 F1:  0.5517241379310344
Me

##### Self attention

In [47]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attention_balanced


In [48]:
CONCATENATE_STAT = True

In [49]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5631, 128, 3840) (5631,)
(2102, 128, 3840) (2102,)


In [50]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3840)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 3840)           1474560   ['image_set[0][0]']           
                                                          0                                       
                                                                                                  
 value (Dense)               (None, None, 3840)           1474560   ['image_set[0][0]']           
                                                          0                                       
                                                                                            

In [51]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20


2024-07-30 07:39:49.605252: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x25ee422a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-30 07:39:49.605415: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2024-07-30 07:39:49.609556: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-30 07:39:49.656270: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-07-30 07:39:49.708038: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


44/44 [==============================] - 16s 319ms/step - loss: 0.5446 - acc: 0.8439 - auc: 0.8782 - binary_accuracy: 0.8439 - recall_1: 0.7293 - precision_1: 0.7491 - val_loss: 1.0620 - val_acc: 0.5718 - val_auc: 0.6492 - val_binary_accuracy: 0.5718 - val_recall_1: 0.5780 - val_precision_1: 0.3674
Epoch 2/20
44/44 [==============================] - 8s 178ms/step - loss: 0.1441 - acc: 0.9499 - auc: 0.9835 - binary_accuracy: 0.9499 - recall_1: 0.8910 - precision_1: 0.9406 - val_loss: 0.8972 - val_acc: 0.7350 - val_auc: 0.7195 - val_binary_accuracy: 0.7350 - val_recall_1: 0.4819 - val_precision_1: 0.5730
Epoch 3/20
44/44 [==============================] - 8s 179ms/step - loss: 0.0828 - acc: 0.9705 - auc: 0.9953 - binary_accuracy: 0.9705 - recall_1: 0.9397 - precision_1: 0.9622 - val_loss: 1.0392 - val_acc: 0.6946 - val_auc: 0.7199 - val_binary_accuracy: 0.6946 - val_recall_1: 0.5480 - val_precision_1: 0.4950
Epoch 4/20
44/44 [==============================] - 8s 177ms/step - loss: 0.0497

In [52]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [53]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attention_balanced Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.6596638655462186 Recall:  0.6428571428571429 Precision:  0.45 F1:  0.5294117647058824
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attention_balanced Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.6386554621848739 Recall:  0.5714285714285714 Precision:  0.4444444444444444 F1:  0.5
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attention_balanced Accuracy:  0.6875 MSE:  0.3125 UAR:  0.6533613445378151 Recall:  0.5714285714285714 Precision:  0.47058823529411764 F1:  0.5161290322580646
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attention_balanced Accuracy:  0.6875 MSE:  0.3125 UAR:  0.6533613445378151 Recall:  0.5714285714285714 Precision:  0.47058823529411764 F1:  0.5161290322580646
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attention_balanced Accuracy:  0.6875 MSE:  0.31

In [54]:
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    modelAtn.set_weights(best_model_weights)
    save_weights(modelAtn, weights_name)

In [55]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [56]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attention_balanced_best Accuracy:  0.625 MSE:  0.375 UAR:  0.6302521008403361 Recall:  0.6428571428571429 Precision:  0.4090909090909091 F1:  0.5000000000000001
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attention_balanced_best Accuracy:  0.625 MSE:  0.375 UAR:  0.6092436974789917 Recall:  0.5714285714285714 Precision:  0.4 F1:  0.47058823529411764
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attention_balanced_best Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.6386554621848739 Recall:  0.5714285714285714 Precision:  0.4444444444444444 F1:  0.5
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attention_balanced_best Accuracy:  0.7083333333333334 MSE:  0.2916666666666667 UAR:  0.6680672268907563 Recall:  0.5714285714285714 Precision:  0.5 F1:  0.5333333333333333
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attention_balanced_best Accuracy:  0.6875 MSE:  

## MobileNet_7

In [25]:
base_model_key = 'mobilenet_7.h5'

In [26]:
ENGAGE_WILD_DATASET, BIN_CLASSIFICATION, N_CLASSES, DATASET_NAME, BASE_DATASET_DIR, ext = define_dataset(True, True)
video2label, labels_list = get_video2label()
print(labels_list)
print(video2label)

['engaged', 'distracted']
{'subject_1_Vid_1': 0, 'subject_1_Vid_2': 0, 'subject_1_Vid_3': 0, 'subject_1_Vid_4': 0, 'subject_1_Vid_5': 0, 'subject_31_Vid_6': 0, 'subject_2_Vid_6': 1, 'subject_3_Vid_6': 0, 'subject_3_Vid_1': 1, 'subject_3_Vid_2': 1, 'subject_3_Vid_3': 0, 'subject_3_Vid_4': 0, 'subject_3_Vid_5': 1, 'subject_3_Vid_7': 0, 'subject_4_Vid_6': 0, 'subject_5_Vid_6': 0, 'subject_6_Vid_6': 1, 'subject_7_Vid_1': 0, 'subject_7_Vid_2': 0, 'subject_7_Vid_3': 0, 'subject_7_Vid_4': 0, 'subject_7_Vid_5': 1, 'subject_8_Vid_6': 0, 'subject_9_Vid_6': 1, 'subject_10_Vid_6': 0, 'subject_11_Vid_6': 0, 'subject_12_Vid_6': 1, 'subject_13_Vid_6': 0, 'subject_14_Vid_6': 0, 'subject_15_Vid_6': 0, 'subject_16_Vid_6': 1, 'subject_17_Vid_6': 0, 'subject_18_Vid_6': 1, 'subject_19_Vid_6': 0, 'subject_20_Vid_6': 0, 'subject_20_Vid_1': 0, 'subject_20_Vid_2': 0, 'subject_20_Vid_3': 0, 'subject_20_Vid_4': 1, 'subject_20_Vid_5': 0, 'subject_20_Vid_5_1': 0, 'subject_20_Vid_5_2': 0, 'subject_20_Vid_7': 1, 'su

### Std

In [59]:
stat_func = np.std
stat_name = "std"

#### Traditional split

In [60]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [61]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [62]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 107/147: 72.78911564625851%
1 40/147: 27.210884353741495%
val:
0 34/48: 70.83333333333333%
1 14/48: 29.166666666666668%


##### Single attention

In [63]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

mobilenet_7.h5_EngageWild_2_std_single_attention_traditional


In [64]:
CONCATENATE_STAT = True

In [65]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 2048) (5449,)
(2284, 128, 2048) (2284,)


In [66]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2049      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [67]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 5s 88ms/step - loss: 0.4456 - acc: 0.8064 - auc: 0.8214 - binary_accuracy: 0.8064 - recall_2: 0.4817 - precision_2: 0.7515 - val_loss: 0.6893 - val_acc: 0.6475 - val_auc: 0.6929 - val_binary_accuracy: 0.6475 - val_recall_2: 0.2701 - val_precision_2: 0.4775
Epoch 2/20
43/43 [==============================] - 1s 28ms/step - loss: 0.2183 - acc: 0.9169 - auc: 0.9671 - binary_accuracy: 0.9169 - recall_2: 0.8112 - precision_2: 0.8882 - val_loss: 0.8184 - val_acc: 0.6690 - val_auc: 0.6946 - val_binary_accuracy: 0.6690 - val_recall_2: 0.1427 - val_precision_2: 0.5744
Epoch 3/20
43/43 [==============================] - 1s 27ms/step - loss: 0.1261 - acc: 0.9552 - auc: 0.9915 - binary_accuracy: 0.9552 - recall_2: 0.8844 - precision_2: 0.9556 - val_loss: 0.7918 - val_acc: 0.6957 - val_auc: 0.7102 - val_binary_accuracy: 0.6957 - val_recall_2: 0.3376 - val_precision_2: 0.6023
Epoch 4/20
43/43 [==============================] - 1s 28ms/step - loss: 

In [68]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [69]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_std_single_attention_traditional Accuracy:  0.5416666666666666 MSE:  0.4583333333333333 UAR:  0.5084033613445378 Recall:  0.42857142857142855 Precision:  0.3 F1:  0.3529411764705882
Metric_name:  mobilenet_7.h5_EngageWild_2_std_single_attention_traditional Accuracy:  0.625 MSE:  0.375 UAR:  0.5672268907563025 Recall:  0.42857142857142855 Precision:  0.375 F1:  0.39999999999999997
Metric_name:  mobilenet_7.h5_EngageWild_2_std_single_attention_traditional Accuracy:  0.6041666666666666 MSE:  0.3958333333333333 UAR:  0.5315126050420168 Recall:  0.35714285714285715 Precision:  0.3333333333333333 F1:  0.3448275862068965
Metric_name:  mobilenet_7.h5_EngageWild_2_std_single_attention_traditional Accuracy:  0.6041666666666666 MSE:  0.3958333333333333 UAR:  0.5315126050420168 Recall:  0.35714285714285715 Precision:  0.3333333333333333 F1:  0.3448275862068965
Metric_name:  mobilenet_7.h5_EngageWild_2_std_single_attention_traditional Accuracy:  0.604166666

In [70]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [71]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [72]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_std_single_attention_traditional_best Accuracy:  0.4791666666666667 MSE:  0.5208333333333334 UAR:  0.5903361344537815 Recall:  0.8571428571428571 Precision:  0.34285714285714286 F1:  0.4897959183673469
Metric_name:  mobilenet_7.h5_EngageWild_2_std_single_attention_traditional_best Accuracy:  0.7291666666666666 MSE:  0.2708333333333333 UAR:  0.7247899159663866 Recall:  0.7142857142857143 Precision:  0.5263157894736842 F1:  0.6060606060606061
Metric_name:  mobilenet_7.h5_EngageWild_2_std_single_attention_traditional_best Accuracy:  0.75 MSE:  0.25 UAR:  0.6554621848739496 Recall:  0.42857142857142855 Precision:  0.6 F1:  0.5
Metric_name:  mobilenet_7.h5_EngageWild_2_std_single_attention_traditional_best Accuracy:  0.7291666666666666 MSE:  0.2708333333333333 UAR:  0.5987394957983193 Recall:  0.2857142857142857 Precision:  0.5714285714285714 F1:  0.38095238095238093
Metric_name:  mobilenet_7.h5_EngageWild_2_std_single_attention_traditional_best Acc

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### Self attention

In [73]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

mobilenet_7.h5_EngageWild_2_std_self_attention_traditional


In [74]:
CONCATENATE_STAT = True

In [75]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 2048) (5449,)
(2284, 128, 2048) (2284,)


In [76]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 attention_1 (Attention)     (None, None, 2048)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [77]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 7s 137ms/step - loss: 0.5493 - acc: 0.7932 - auc: 0.7957 - binary_accuracy: 0.7932 - recall_3: 0.5549 - precision_3: 0.6656 - val_loss: 0.6722 - val_acc: 0.6848 - val_auc: 0.6613 - val_binary_accuracy: 0.6848 - val_recall_3: 0.4255 - val_precision_3: 0.5539
Epoch 2/20
43/43 [==============================] - 3s 66ms/step - loss: 0.2095 - acc: 0.9286 - auc: 0.9708 - binary_accuracy: 0.9286 - recall_3: 0.8420 - precision_3: 0.9017 - val_loss: 0.7188 - val_acc: 0.7097 - val_auc: 0.6857 - val_binary_accuracy: 0.7097 - val_recall_3: 0.3465 - val_precision_3: 0.6445
Epoch 3/20
43/43 [==============================] - 3s 67ms/step - loss: 0.1524 - acc: 0.9435 - auc: 0.9828 - binary_accuracy: 0.9435 - recall_3: 0.8741 - precision_3: 0.9240 - val_loss: 0.7936 - val_acc: 0.6515 - val_auc: 0.6645 - val_binary_accuracy: 0.6515 - val_recall_3: 0.4229 - val_precision_3: 0.4919
Epoch 4/20
43/43 [==============================] - 3s 67ms/step - loss:

In [78]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [79]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_std_self_attention_traditional Accuracy:  0.6041666666666666 MSE:  0.3958333333333333 UAR:  0.6365546218487395 Recall:  0.7142857142857143 Precision:  0.4 F1:  0.5128205128205129
Metric_name:  mobilenet_7.h5_EngageWild_2_std_self_attention_traditional Accuracy:  0.625 MSE:  0.375 UAR:  0.6512605042016807 Recall:  0.7142857142857143 Precision:  0.4166666666666667 F1:  0.5263157894736842
Metric_name:  mobilenet_7.h5_EngageWild_2_std_self_attention_traditional Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.6596638655462186 Recall:  0.6428571428571429 Precision:  0.45 F1:  0.5294117647058824
Metric_name:  mobilenet_7.h5_EngageWild_2_std_self_attention_traditional Accuracy:  0.6458333333333334 MSE:  0.3541666666666667 UAR:  0.6239495798319328 Recall:  0.5714285714285714 Precision:  0.42105263157894735 F1:  0.48484848484848486
Metric_name:  mobilenet_7.h5_EngageWild_2_std_self_attention_traditional Accuracy:  0.6458333333333334 MSE:  

In [80]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [81]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [82]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_std_self_attention_traditional_best Accuracy:  0.375 MSE:  0.625 UAR:  0.5378151260504201 Recall:  0.9285714285714286 Precision:  0.30952380952380953 F1:  0.46428571428571436
Metric_name:  mobilenet_7.h5_EngageWild_2_std_self_attention_traditional_best Accuracy:  0.5 MSE:  0.5 UAR:  0.6260504201680672 Recall:  0.9285714285714286 Precision:  0.3611111111111111 F1:  0.52
Metric_name:  mobilenet_7.h5_EngageWild_2_std_self_attention_traditional_best Accuracy:  0.5416666666666666 MSE:  0.4583333333333333 UAR:  0.6554621848739496 Recall:  0.9285714285714286 Precision:  0.38235294117647056 F1:  0.5416666666666667
Metric_name:  mobilenet_7.h5_EngageWild_2_std_self_attention_traditional_best Accuracy:  0.6875 MSE:  0.3125 UAR:  0.7584033613445378 Recall:  0.9285714285714286 Precision:  0.48148148148148145 F1:  0.6341463414634146
Metric_name:  mobilenet_7.h5_EngageWild_2_std_self_attention_traditional_best Accuracy:  0.7083333333333334 MSE:  0.2916666666

#### Balanced split

In [83]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [84]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [85]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 107/147: 72.78911564625851%
1 40/147: 27.210884353741495%
val:
0 34/48: 70.83333333333333%
1 14/48: 29.166666666666668%


In [86]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.759)

(147,) (147,)
(48,) (48,)
(195,) (195,)
[0 1] [141  54]
195
147
48


In [87]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 107/147: 72.78911564625851%
1 40/147: 27.210884353741495%
val:
0 34/48: 70.83333333333333%
1 14/48: 29.166666666666668%


##### Single attention

In [88]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

mobilenet_7.h5_EngageWild_2_std_single_attention_balanced


In [89]:
CONCATENATE_STAT = True

In [90]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5631, 128, 2048) (5631,)
(2102, 128, 2048) (2102,)


In [91]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2049      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [92]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
44/44 [==============================] - 4s 61ms/step - loss: 0.3941 - acc: 0.8283 - auc: 0.8757 - binary_accuracy: 0.8283 - recall_4: 0.5595 - precision_4: 0.8162 - val_loss: 0.6503 - val_acc: 0.7174 - val_auc: 0.6976 - val_binary_accuracy: 0.7174 - val_recall_4: 0.5465 - val_precision_4: 0.5314
Epoch 2/20
44/44 [==============================] - 1s 27ms/step - loss: 0.1789 - acc: 0.9357 - auc: 0.9808 - binary_accuracy: 0.9357 - recall_4: 0.8535 - precision_4: 0.9286 - val_loss: 0.8561 - val_acc: 0.6422 - val_auc: 0.6693 - val_binary_accuracy: 0.6422 - val_recall_4: 0.5449 - val_precision_4: 0.4277
Epoch 3/20
44/44 [==============================] - 1s 26ms/step - loss: 0.0990 - acc: 0.9663 - auc: 0.9958 - binary_accuracy: 0.9663 - recall_4: 0.9080 - precision_4: 0.9792 - val_loss: 0.9587 - val_acc: 0.6584 - val_auc: 0.6736 - val_binary_accuracy: 0.6584 - val_recall_4: 0.4945 - val_precision_4: 0.4416
Epoch 4/20
44/44 [==============================] - 1s 28ms/step - loss: 

In [93]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [94]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_std_single_attention_balanced Accuracy:  0.75 MSE:  0.25 UAR:  0.6764705882352942 Recall:  0.5 Precision:  0.5833333333333334 F1:  0.5384615384615384
Metric_name:  mobilenet_7.h5_EngageWild_2_std_single_attention_balanced Accuracy:  0.7291666666666666 MSE:  0.2708333333333333 UAR:  0.6197478991596639 Recall:  0.35714285714285715 Precision:  0.5555555555555556 F1:  0.43478260869565216
Metric_name:  mobilenet_7.h5_EngageWild_2_std_single_attention_balanced Accuracy:  0.75 MSE:  0.25 UAR:  0.634453781512605 Recall:  0.35714285714285715 Precision:  0.625 F1:  0.45454545454545453
Metric_name:  mobilenet_7.h5_EngageWild_2_std_single_attention_balanced Accuracy:  0.7291666666666666 MSE:  0.2708333333333333 UAR:  0.5987394957983193 Recall:  0.2857142857142857 Precision:  0.5714285714285714 F1:  0.38095238095238093
Metric_name:  mobilenet_7.h5_EngageWild_2_std_single_attention_balanced Accuracy:  0.7291666666666666 MSE:  0.2708333333333333 UAR:  0.59873

In [95]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [96]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [97]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_std_single_attention_balanced_best Accuracy:  0.5 MSE:  0.5 UAR:  0.6260504201680672 Recall:  0.9285714285714286 Precision:  0.3611111111111111 F1:  0.52
Metric_name:  mobilenet_7.h5_EngageWild_2_std_single_attention_balanced_best Accuracy:  0.5833333333333334 MSE:  0.4166666666666667 UAR:  0.6428571428571428 Recall:  0.7857142857142857 Precision:  0.39285714285714285 F1:  0.5238095238095237
Metric_name:  mobilenet_7.h5_EngageWild_2_std_single_attention_balanced_best Accuracy:  0.5833333333333334 MSE:  0.4166666666666667 UAR:  0.6008403361344539 Recall:  0.6428571428571429 Precision:  0.375 F1:  0.4736842105263159
Metric_name:  mobilenet_7.h5_EngageWild_2_std_single_attention_balanced_best Accuracy:  0.7083333333333334 MSE:  0.2916666666666667 UAR:  0.6890756302521008 Recall:  0.6428571428571429 Precision:  0.5 F1:  0.5625000000000001
Metric_name:  mobilenet_7.h5_EngageWild_2_std_single_attention_balanced_best Accuracy:  0.75 MSE:  0.25 UAR:  0

##### Self attention

In [98]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

mobilenet_7.h5_EngageWild_2_std_self_attention_balanced


In [99]:
CONCATENATE_STAT = True

In [100]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5631, 128, 2048) (5631,)
(2102, 128, 2048) (2102,)


In [101]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 attention_2 (Attention)     (None, None, 2048)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [102]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
44/44 [==============================] - 6s 111ms/step - loss: 0.5453 - acc: 0.7906 - auc: 0.8068 - binary_accuracy: 0.7906 - recall_5: 0.5946 - precision_5: 0.6758 - val_loss: 0.6830 - val_acc: 0.6603 - val_auc: 0.6268 - val_binary_accuracy: 0.6603 - val_recall_5: 0.4031 - val_precision_5: 0.4332
Epoch 2/20
44/44 [==============================] - 3s 66ms/step - loss: 0.1889 - acc: 0.9352 - auc: 0.9804 - binary_accuracy: 0.9352 - recall_5: 0.8436 - precision_5: 0.9363 - val_loss: 0.7894 - val_acc: 0.6456 - val_auc: 0.6385 - val_binary_accuracy: 0.6456 - val_recall_5: 0.4331 - val_precision_5: 0.4167
Epoch 3/20
44/44 [==============================] - 3s 66ms/step - loss: 0.1139 - acc: 0.9639 - auc: 0.9934 - binary_accuracy: 0.9639 - recall_5: 0.9203 - precision_5: 0.9591 - val_loss: 0.8956 - val_acc: 0.6618 - val_auc: 0.6308 - val_binary_accuracy: 0.6618 - val_recall_5: 0.3701 - val_precision_5: 0.4304
Epoch 4/20
44/44 [==============================] - 3s 66ms/step - loss:

In [103]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [104]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_std_self_attention_balanced Accuracy:  0.6458333333333334 MSE:  0.3541666666666667 UAR:  0.6449579831932774 Recall:  0.6428571428571429 Precision:  0.42857142857142855 F1:  0.5142857142857143
Metric_name:  mobilenet_7.h5_EngageWild_2_std_self_attention_balanced Accuracy:  0.6458333333333334 MSE:  0.3541666666666667 UAR:  0.6029411764705883 Recall:  0.5 Precision:  0.4117647058823529 F1:  0.45161290322580644
Metric_name:  mobilenet_7.h5_EngageWild_2_std_self_attention_balanced Accuracy:  0.625 MSE:  0.375 UAR:  0.5672268907563025 Recall:  0.42857142857142855 Precision:  0.375 F1:  0.39999999999999997
Metric_name:  mobilenet_7.h5_EngageWild_2_std_self_attention_balanced Accuracy:  0.625 MSE:  0.375 UAR:  0.5672268907563025 Recall:  0.42857142857142855 Precision:  0.375 F1:  0.39999999999999997
Metric_name:  mobilenet_7.h5_EngageWild_2_std_self_attention_balanced Accuracy:  0.6458333333333334 MSE:  0.3541666666666667 UAR:  0.5819327731092437 Recal

In [105]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [106]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [107]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_std_self_attention_balanced_best Accuracy:  0.375 MSE:  0.625 UAR:  0.5378151260504201 Recall:  0.9285714285714286 Precision:  0.30952380952380953 F1:  0.46428571428571436
Metric_name:  mobilenet_7.h5_EngageWild_2_std_self_attention_balanced_best Accuracy:  0.5416666666666666 MSE:  0.4583333333333333 UAR:  0.6554621848739496 Recall:  0.9285714285714286 Precision:  0.38235294117647056 F1:  0.5416666666666667
Metric_name:  mobilenet_7.h5_EngageWild_2_std_self_attention_balanced_best Accuracy:  0.5625 MSE:  0.4375 UAR:  0.6281512605042017 Recall:  0.7857142857142857 Precision:  0.3793103448275862 F1:  0.5116279069767441
Metric_name:  mobilenet_7.h5_EngageWild_2_std_self_attention_balanced_best Accuracy:  0.5625 MSE:  0.4375 UAR:  0.5651260504201681 Recall:  0.5714285714285714 Precision:  0.34782608695652173 F1:  0.4324324324324324
Metric_name:  mobilenet_7.h5_EngageWild_2_std_self_attention_balanced_best Accuracy:  0.5833333333333334 MSE:  0.41666

### Max

In [108]:
stat_func = np.max
stat_name = "max"

#### Traditional split

In [109]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [110]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [111]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 107/147: 72.78911564625851%
1 40/147: 27.210884353741495%
val:
0 34/48: 70.83333333333333%
1 14/48: 29.166666666666668%


##### Single attention

In [112]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

mobilenet_7.h5_EngageWild_2_max_single_attention_traditional


In [113]:
CONCATENATE_STAT = True

In [114]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 2048) (5449,)
(2284, 128, 2048) (2284,)


In [115]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2049      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [116]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 4s 66ms/step - loss: 0.6302 - acc: 0.7288 - auc: 0.6793 - binary_accuracy: 0.7288 - recall_6: 0.3455 - precision_6: 0.5396 - val_loss: 0.6465 - val_acc: 0.5898 - val_auc: 0.6219 - val_binary_accuracy: 0.5898 - val_recall_6: 0.0994 - val_precision_6: 0.2532
Epoch 2/20
43/43 [==============================] - 1s 34ms/step - loss: 0.3227 - acc: 0.8873 - auc: 0.9345 - binary_accuracy: 0.8873 - recall_6: 0.6808 - precision_6: 0.9006 - val_loss: 0.7450 - val_acc: 0.6116 - val_auc: 0.6218 - val_binary_accuracy: 0.6116 - val_recall_6: 0.1465 - val_precision_6: 0.3464
Epoch 3/20
43/43 [==============================] - 1s 34ms/step - loss: 0.2100 - acc: 0.9312 - auc: 0.9718 - binary_accuracy: 0.9312 - recall_6: 0.8247 - precision_6: 0.9264 - val_loss: 0.8559 - val_acc: 0.6620 - val_auc: 0.5774 - val_binary_accuracy: 0.6620 - val_recall_6: 0.1631 - val_precision_6: 0.5267
Epoch 4/20
43/43 [==============================] - 1s 34ms/step - loss: 

In [117]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [118]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_max_single_attention_traditional Accuracy:  0.5625 MSE:  0.4375 UAR:  0.5441176470588236 Recall:  0.5 Precision:  0.3333333333333333 F1:  0.4
Metric_name:  mobilenet_7.h5_EngageWild_2_max_single_attention_traditional Accuracy:  0.5833333333333334 MSE:  0.4166666666666667 UAR:  0.5378151260504201 Recall:  0.42857142857142855 Precision:  0.3333333333333333 F1:  0.375
Metric_name:  mobilenet_7.h5_EngageWild_2_max_single_attention_traditional Accuracy:  0.6041666666666666 MSE:  0.3958333333333333 UAR:  0.5315126050420168 Recall:  0.35714285714285715 Precision:  0.3333333333333333 F1:  0.3448275862068965
Metric_name:  mobilenet_7.h5_EngageWild_2_max_single_attention_traditional Accuracy:  0.625 MSE:  0.375 UAR:  0.546218487394958 Recall:  0.35714285714285715 Precision:  0.35714285714285715 F1:  0.35714285714285715
Metric_name:  mobilenet_7.h5_EngageWild_2_max_single_attention_traditional Accuracy:  0.6458333333333334 MSE:  0.3541666666666667 UAR:  0

In [119]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [120]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [121]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_max_single_attention_traditional_best Accuracy:  0.3125 MSE:  0.6875 UAR:  0.5147058823529411 Recall:  1.0 Precision:  0.2978723404255319 F1:  0.45901639344262296
Metric_name:  mobilenet_7.h5_EngageWild_2_max_single_attention_traditional_best Accuracy:  0.5208333333333334 MSE:  0.4791666666666667 UAR:  0.6407563025210085 Recall:  0.9285714285714286 Precision:  0.37142857142857144 F1:  0.5306122448979592
Metric_name:  mobilenet_7.h5_EngageWild_2_max_single_attention_traditional_best Accuracy:  0.5625 MSE:  0.4375 UAR:  0.5441176470588236 Recall:  0.5 Precision:  0.3333333333333333 F1:  0.4
Metric_name:  mobilenet_7.h5_EngageWild_2_max_single_attention_traditional_best Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.5336134453781513 Recall:  0.21428571428571427 Precision:  0.375 F1:  0.2727272727272727
Metric_name:  mobilenet_7.h5_EngageWild_2_max_single_attention_traditional_best Accuracy:  0.6666666666666666 MSE:  0.3333333333333

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### Self attention

In [122]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

mobilenet_7.h5_EngageWild_2_max_self_attention_traditional


In [123]:
CONCATENATE_STAT = True

In [124]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 2048) (5449,)
(2284, 128, 2048) (2284,)


In [125]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 attention_3 (Attention)     (None, None, 2048)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [126]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 6s 110ms/step - loss: 3.1838 - acc: 0.6818 - auc: 0.6361 - binary_accuracy: 0.6818 - recall_7: 0.4483 - precision_7: 0.4437 - val_loss: 1.0201 - val_acc: 0.6502 - val_auc: 0.6093 - val_binary_accuracy: 0.6502 - val_recall_7: 0.1172 - val_precision_7: 0.4646
Epoch 2/20
43/43 [==============================] - 3s 66ms/step - loss: 0.3023 - acc: 0.8789 - auc: 0.9268 - binary_accuracy: 0.8789 - recall_7: 0.7405 - precision_7: 0.8183 - val_loss: 0.7808 - val_acc: 0.6467 - val_auc: 0.5626 - val_binary_accuracy: 0.6467 - val_recall_7: 0.3146 - val_precision_7: 0.4787
Epoch 3/20
43/43 [==============================] - 3s 66ms/step - loss: 0.1850 - acc: 0.9383 - auc: 0.9809 - binary_accuracy: 0.9383 - recall_7: 0.8504 - precision_7: 0.9278 - val_loss: 0.8676 - val_acc: 0.6454 - val_auc: 0.5438 - val_binary_accuracy: 0.6454 - val_recall_7: 0.1885 - val_precision_7: 0.4611
Epoch 4/20
43/43 [==============================] - 3s 66ms/step - loss:

In [127]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [128]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_max_self_attention_traditional Accuracy:  0.5625 MSE:  0.4375 UAR:  0.5861344537815126 Recall:  0.6428571428571429 Precision:  0.36 F1:  0.4615384615384615
Metric_name:  mobilenet_7.h5_EngageWild_2_max_self_attention_traditional Accuracy:  0.625 MSE:  0.375 UAR:  0.6302521008403361 Recall:  0.6428571428571429 Precision:  0.4090909090909091 F1:  0.5000000000000001
Metric_name:  mobilenet_7.h5_EngageWild_2_max_self_attention_traditional Accuracy:  0.6041666666666666 MSE:  0.3958333333333333 UAR:  0.5525210084033614 Recall:  0.42857142857142855 Precision:  0.35294117647058826 F1:  0.3870967741935484
Metric_name:  mobilenet_7.h5_EngageWild_2_max_self_attention_traditional Accuracy:  0.625 MSE:  0.375 UAR:  0.5672268907563025 Recall:  0.42857142857142855 Precision:  0.375 F1:  0.39999999999999997
Metric_name:  mobilenet_7.h5_EngageWild_2_max_self_attention_traditional Accuracy:  0.625 MSE:  0.375 UAR:  0.5672268907563025 Recall:  0.42857142857142855

In [129]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [130]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [131]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_max_self_attention_traditional_best Accuracy:  0.3541666666666667 MSE:  0.6458333333333334 UAR:  0.5441176470588235 Recall:  1.0 Precision:  0.3111111111111111 F1:  0.4745762711864407
Metric_name:  mobilenet_7.h5_EngageWild_2_max_self_attention_traditional_best Accuracy:  0.375 MSE:  0.625 UAR:  0.5168067226890756 Recall:  0.8571428571428571 Precision:  0.3 F1:  0.4444444444444444
Metric_name:  mobilenet_7.h5_EngageWild_2_max_self_attention_traditional_best Accuracy:  0.4583333333333333 MSE:  0.5416666666666666 UAR:  0.5756302521008403 Recall:  0.8571428571428571 Precision:  0.3333333333333333 F1:  0.48
Metric_name:  mobilenet_7.h5_EngageWild_2_max_self_attention_traditional_best Accuracy:  0.625 MSE:  0.375 UAR:  0.6302521008403361 Recall:  0.6428571428571429 Precision:  0.4090909090909091 F1:  0.5000000000000001
Metric_name:  mobilenet_7.h5_EngageWild_2_max_self_attention_traditional_best Accuracy:  0.7291666666666666 MSE:  0.2708333333333333

#### Balanced split

In [132]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [133]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [134]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 107/147: 72.78911564625851%
1 40/147: 27.210884353741495%
val:
0 34/48: 70.83333333333333%
1 14/48: 29.166666666666668%


In [135]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.759)

(147,) (147,)
(48,) (48,)
(195,) (195,)
[0 1] [141  54]
195
147
48


In [136]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 107/147: 72.78911564625851%
1 40/147: 27.210884353741495%
val:
0 34/48: 70.83333333333333%
1 14/48: 29.166666666666668%


##### Single attention

In [137]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

mobilenet_7.h5_EngageWild_2_max_single_attention_balanced


In [138]:
CONCATENATE_STAT = True

In [139]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5631, 128, 2048) (5631,)
(2102, 128, 2048) (2102,)


In [140]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2049      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [141]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
44/44 [==============================] - 4s 61ms/step - loss: 0.6190 - acc: 0.7223 - auc: 0.7111 - binary_accuracy: 0.7223 - recall_8: 0.3849 - precision_8: 0.5611 - val_loss: 0.6488 - val_acc: 0.6151 - val_auc: 0.5859 - val_binary_accuracy: 0.6151 - val_recall_8: 0.0945 - val_precision_8: 0.2041
Epoch 2/20
44/44 [==============================] - 1s 27ms/step - loss: 0.3018 - acc: 0.8863 - auc: 0.9412 - binary_accuracy: 0.8863 - recall_8: 0.7036 - precision_8: 0.8996 - val_loss: 0.6827 - val_acc: 0.6813 - val_auc: 0.6146 - val_binary_accuracy: 0.6813 - val_recall_8: 0.3638 - val_precision_8: 0.4648
Epoch 3/20
44/44 [==============================] - 1s 27ms/step - loss: 0.1704 - acc: 0.9533 - auc: 0.9885 - binary_accuracy: 0.9533 - recall_8: 0.8735 - precision_8: 0.9694 - val_loss: 0.8436 - val_acc: 0.6518 - val_auc: 0.6007 - val_binary_accuracy: 0.6518 - val_recall_8: 0.3370 - val_precision_8: 0.4076
Epoch 4/20
44/44 [==============================] - 1s 28ms/step - loss: 

In [142]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [143]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_max_single_attention_balanced Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.6176470588235294 Recall:  0.5 Precision:  0.4375 F1:  0.4666666666666667
Metric_name:  mobilenet_7.h5_EngageWild_2_max_single_attention_balanced Accuracy:  0.6458333333333334 MSE:  0.3541666666666667 UAR:  0.5819327731092437 Recall:  0.42857142857142855 Precision:  0.4 F1:  0.4137931034482759
Metric_name:  mobilenet_7.h5_EngageWild_2_max_single_attention_balanced Accuracy:  0.6458333333333334 MSE:  0.3541666666666667 UAR:  0.5819327731092437 Recall:  0.42857142857142855 Precision:  0.4 F1:  0.4137931034482759
Metric_name:  mobilenet_7.h5_EngageWild_2_max_single_attention_balanced Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.5966386554621849 Recall:  0.42857142857142855 Precision:  0.42857142857142855 F1:  0.42857142857142855
Metric_name:  mobilenet_7.h5_EngageWild_2_max_single_attention_balanced Accuracy:  0.6875 MSE:  0.3125 UAR:  0.6

In [144]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [145]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [146]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_max_single_attention_balanced_best Accuracy:  0.4166666666666667 MSE:  0.5833333333333334 UAR:  0.5882352941176471 Recall:  1.0 Precision:  0.3333333333333333 F1:  0.5
Metric_name:  mobilenet_7.h5_EngageWild_2_max_single_attention_balanced_best Accuracy:  0.5 MSE:  0.5 UAR:  0.5840336134453781 Recall:  0.7857142857142857 Precision:  0.34375 F1:  0.4782608695652174
Metric_name:  mobilenet_7.h5_EngageWild_2_max_single_attention_balanced_best Accuracy:  0.5 MSE:  0.5 UAR:  0.45798319327731096 Recall:  0.35714285714285715 Precision:  0.25 F1:  0.2941176470588235
Metric_name:  mobilenet_7.h5_EngageWild_2_max_single_attention_balanced_best Accuracy:  0.5833333333333334 MSE:  0.4166666666666667 UAR:  0.4747899159663866 Recall:  0.21428571428571427 Precision:  0.25 F1:  0.23076923076923075
Metric_name:  mobilenet_7.h5_EngageWild_2_max_single_attention_balanced_best Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.5126050420168067 Recall: 

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### Self attention

In [147]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

mobilenet_7.h5_EngageWild_2_max_self_attention_balanced


In [148]:
CONCATENATE_STAT = True

In [149]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5631, 128, 2048) (5631,)
(2102, 128, 2048) (2102,)


In [150]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 attention_4 (Attention)     (None, None, 2048)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [151]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
44/44 [==============================] - 6s 108ms/step - loss: 1.0655 - acc: 0.7862 - auc: 0.7845 - binary_accuracy: 0.7862 - recall_9: 0.6233 - precision_9: 0.6548 - val_loss: 0.8465 - val_acc: 0.6089 - val_auc: 0.5998 - val_binary_accuracy: 0.6089 - val_recall_9: 0.3543 - val_precision_9: 0.3532
Epoch 2/20
44/44 [==============================] - 3s 65ms/step - loss: 0.1544 - acc: 0.9494 - auc: 0.9865 - binary_accuracy: 0.9494 - recall_9: 0.8822 - precision_9: 0.9472 - val_loss: 0.9989 - val_acc: 0.5966 - val_auc: 0.6011 - val_binary_accuracy: 0.5966 - val_recall_9: 0.1969 - val_precision_9: 0.2700
Epoch 3/20
44/44 [==============================] - 3s 65ms/step - loss: 0.0840 - acc: 0.9805 - auc: 0.9981 - binary_accuracy: 0.9805 - recall_9: 0.9514 - precision_9: 0.9836 - val_loss: 1.2105 - val_acc: 0.5861 - val_auc: 0.5777 - val_binary_accuracy: 0.5861 - val_recall_9: 0.1591 - val_precision_9: 0.2311
Epoch 4/20
44/44 [==============================] - 3s 66ms/step - loss:

In [152]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [153]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_max_self_attention_balanced Accuracy:  0.6458333333333334 MSE:  0.3541666666666667 UAR:  0.5399159663865546 Recall:  0.2857142857142857 Precision:  0.36363636363636365 F1:  0.32
Metric_name:  mobilenet_7.h5_EngageWild_2_max_self_attention_balanced Accuracy:  0.6041666666666666 MSE:  0.3958333333333333 UAR:  0.4684873949579832 Recall:  0.14285714285714285 Precision:  0.2222222222222222 F1:  0.17391304347826086
Metric_name:  mobilenet_7.h5_EngageWild_2_max_self_attention_balanced Accuracy:  0.6041666666666666 MSE:  0.3958333333333333 UAR:  0.4684873949579832 Recall:  0.14285714285714285 Precision:  0.2222222222222222 F1:  0.17391304347826086
Metric_name:  mobilenet_7.h5_EngageWild_2_max_self_attention_balanced Accuracy:  0.6041666666666666 MSE:  0.3958333333333333 UAR:  0.4684873949579832 Recall:  0.14285714285714285 Precision:  0.2222222222222222 F1:  0.17391304347826086
Metric_name:  mobilenet_7.h5_EngageWild_2_max_self_attention_balanced Accur

In [154]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [155]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [156]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_max_self_attention_balanced_best Accuracy:  0.4791666666666667 MSE:  0.5208333333333334 UAR:  0.5903361344537815 Recall:  0.8571428571428571 Precision:  0.34285714285714286 F1:  0.4897959183673469
Metric_name:  mobilenet_7.h5_EngageWild_2_max_self_attention_balanced_best Accuracy:  0.5208333333333334 MSE:  0.4791666666666667 UAR:  0.5987394957983193 Recall:  0.7857142857142857 Precision:  0.3548387096774194 F1:  0.48888888888888893
Metric_name:  mobilenet_7.h5_EngageWild_2_max_self_attention_balanced_best Accuracy:  0.6041666666666666 MSE:  0.3958333333333333 UAR:  0.657563025210084 Recall:  0.7857142857142857 Precision:  0.4074074074074074 F1:  0.5365853658536585
Metric_name:  mobilenet_7.h5_EngageWild_2_max_self_attention_balanced_best Accuracy:  0.6458333333333334 MSE:  0.3541666666666667 UAR:  0.6239495798319328 Recall:  0.5714285714285714 Precision:  0.42105263157894735 F1:  0.48484848484848486
Metric_name:  mobilenet_7.h5_EngageWild_2_max

### STAT

In [27]:
stat_func = compute_descriptor
stat_name = "STAT"

#### Traditional split

In [158]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [159]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [160]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 107/147: 72.78911564625851%
1 40/147: 27.210884353741495%
val:
0 34/48: 70.83333333333333%
1 14/48: 29.166666666666668%


##### Single attention

In [161]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

mobilenet_7.h5_EngageWild_2_STAT_single_attention_traditional


In [162]:
CONCATENATE_STAT = True

In [163]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 3072) (5449,)
(2284, 128, 3072) (2284,)


In [164]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3072)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              3073      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                           

In [165]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 5s 94ms/step - loss: 0.4465 - acc: 0.8012 - auc: 0.8244 - binary_accuracy: 0.8012 - recall_10: 0.5067 - precision_10: 0.7147 - val_loss: 0.6960 - val_acc: 0.6585 - val_auc: 0.6723 - val_binary_accuracy: 0.6585 - val_recall_10: 0.3019 - val_precision_10: 0.5053
Epoch 2/20
43/43 [==============================] - 2s 39ms/step - loss: 0.2180 - acc: 0.9095 - auc: 0.9672 - binary_accuracy: 0.9095 - recall_10: 0.7938 - precision_10: 0.8778 - val_loss: 0.8028 - val_acc: 0.6546 - val_auc: 0.6677 - val_binary_accuracy: 0.6546 - val_recall_10: 0.1962 - val_precision_10: 0.4936
Epoch 3/20
43/43 [==============================] - 2s 38ms/step - loss: 0.1432 - acc: 0.9424 - auc: 0.9876 - binary_accuracy: 0.9424 - recall_10: 0.8696 - precision_10: 0.9242 - val_loss: 0.7691 - val_acc: 0.6563 - val_auc: 0.6781 - val_binary_accuracy: 0.6563 - val_recall_10: 0.2803 - val_precision_10: 0.5000
Epoch 4/20
43/43 [==============================] - 2s 38ms/s

In [166]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [167]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_single_attention_traditional Accuracy:  0.5625 MSE:  0.4375 UAR:  0.523109243697479 Recall:  0.42857142857142855 Precision:  0.3157894736842105 F1:  0.36363636363636365
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_single_attention_traditional Accuracy:  0.5833333333333334 MSE:  0.4166666666666667 UAR:  0.5168067226890757 Recall:  0.35714285714285715 Precision:  0.3125 F1:  0.3333333333333333
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_single_attention_traditional Accuracy:  0.6041666666666666 MSE:  0.3958333333333333 UAR:  0.5105042016806722 Recall:  0.2857142857142857 Precision:  0.3076923076923077 F1:  0.29629629629629634
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_single_attention_traditional Accuracy:  0.6458333333333334 MSE:  0.3541666666666667 UAR:  0.5399159663865546 Recall:  0.2857142857142857 Precision:  0.36363636363636365 F1:  0.32
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_single_attention_traditional Accuracy:  0.6

In [168]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [169]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [170]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_single_attention_traditional_best Accuracy:  0.4791666666666667 MSE:  0.5208333333333334 UAR:  0.6113445378151261 Recall:  0.9285714285714286 Precision:  0.35135135135135137 F1:  0.5098039215686275
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_single_attention_traditional_best Accuracy:  0.6458333333333334 MSE:  0.3541666666666667 UAR:  0.6659663865546219 Recall:  0.7142857142857143 Precision:  0.43478260869565216 F1:  0.5405405405405405
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_single_attention_traditional_best Accuracy:  0.7708333333333334 MSE:  0.22916666666666666 UAR:  0.6911764705882353 Recall:  0.5 Precision:  0.6363636363636364 F1:  0.56
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_single_attention_traditional_best Accuracy:  0.7291666666666666 MSE:  0.2708333333333333 UAR:  0.5987394957983193 Recall:  0.2857142857142857 Precision:  0.5714285714285714 F1:  0.38095238095238093
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_si

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### Self attention

In [171]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

mobilenet_7.h5_EngageWild_2_STAT_self_attention_traditional


In [172]:
CONCATENATE_STAT = True

In [173]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 3072) (5449,)
(2284, 128, 3072) (2284,)


In [174]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3072)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 3072)           9437184   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 3072)           9437184   ['image_set[0][0]']           
                                                                                                  
 attention_5 (Attention)     (None, None, 3072)           0         ['query[0][0]',               
                                                                     'value[0][0]']        

In [175]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 9s 181ms/step - loss: 0.8626 - acc: 0.7851 - auc: 0.7786 - binary_accuracy: 0.7851 - recall_11: 0.5793 - precision_11: 0.6361 - val_loss: 0.7451 - val_acc: 0.6213 - val_auc: 0.6604 - val_binary_accuracy: 0.6213 - val_recall_11: 0.1554 - val_precision_11: 0.3765
Epoch 2/20
43/43 [==============================] - 5s 125ms/step - loss: 0.1910 - acc: 0.9284 - auc: 0.9751 - binary_accuracy: 0.9284 - recall_11: 0.8382 - precision_11: 0.9044 - val_loss: 0.7915 - val_acc: 0.6694 - val_auc: 0.6388 - val_binary_accuracy: 0.6694 - val_recall_11: 0.3389 - val_precision_11: 0.5299
Epoch 3/20
43/43 [==============================] - 6s 130ms/step - loss: 0.1192 - acc: 0.9604 - auc: 0.9929 - binary_accuracy: 0.9604 - recall_11: 0.9094 - precision_11: 0.9497 - val_loss: 0.7957 - val_acc: 0.6677 - val_auc: 0.6517 - val_binary_accuracy: 0.6677 - val_recall_11: 0.3643 - val_precision_11: 0.5238
Epoch 4/20
43/43 [==============================] - 5s 124

In [176]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [177]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_self_attention_traditional Accuracy:  0.5416666666666666 MSE:  0.4583333333333333 UAR:  0.592436974789916 Recall:  0.7142857142857143 Precision:  0.35714285714285715 F1:  0.4761904761904762
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_self_attention_traditional Accuracy:  0.5208333333333334 MSE:  0.4791666666666667 UAR:  0.5567226890756303 Recall:  0.6428571428571429 Precision:  0.3333333333333333 F1:  0.43902439024390244
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_self_attention_traditional Accuracy:  0.5 MSE:  0.5 UAR:  0.5210084033613445 Recall:  0.5714285714285714 Precision:  0.3076923076923077 F1:  0.4
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_self_attention_traditional Accuracy:  0.5208333333333334 MSE:  0.4791666666666667 UAR:  0.5357142857142857 Recall:  0.5714285714285714 Precision:  0.32 F1:  0.41025641025641024
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_self_attention_traditional Accuracy:  0.5208333333333334 MSE: 

In [178]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [179]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [180]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_self_attention_traditional_best Accuracy:  0.5 MSE:  0.5 UAR:  0.5840336134453781 Recall:  0.7857142857142857 Precision:  0.34375 F1:  0.4782608695652174
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_self_attention_traditional_best Accuracy:  0.625 MSE:  0.375 UAR:  0.6302521008403361 Recall:  0.6428571428571429 Precision:  0.4090909090909091 F1:  0.5000000000000001
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_self_attention_traditional_best Accuracy:  0.7083333333333334 MSE:  0.2916666666666667 UAR:  0.6470588235294117 Recall:  0.5 Precision:  0.5 F1:  0.5
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_self_attention_traditional_best Accuracy:  0.7708333333333334 MSE:  0.22916666666666666 UAR:  0.6911764705882353 Recall:  0.5 Precision:  0.6363636363636364 F1:  0.56
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_self_attention_traditional_best Accuracy:  0.7291666666666666 MSE:  0.2708333333333333 UAR:  0.5987394957983193 Recall:  0.28

#### Balanced split

In [28]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [29]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [30]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 107/147: 72.78911564625851%
1 40/147: 27.210884353741495%
val:
0 34/48: 70.83333333333333%
1 14/48: 29.166666666666668%


In [31]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.759)

(147, 2048) (147,)
(48, 2048) (48,)
(195, 2048) (195,)
[0 1] [141  54]
195
147
48


In [32]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 107/147: 72.78911564625851%
1 40/147: 27.210884353741495%
val:
0 34/48: 70.83333333333333%
1 14/48: 29.166666666666668%


##### Single attention

In [35]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

mobilenet_7.h5_EngageWild_2_STAT_single_attention_balanced


In [36]:
CONCATENATE_STAT = True

In [37]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5631, 128, 3072) (5631,)
(2102, 128, 3072) (2102,)


In [38]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

2024-07-30 08:14:20.359963: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38431 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:11:00.0, compute capability: 8.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3072)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              3073      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                              

In [39]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20


2024-07-30 08:14:31.460675: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1f36bbf0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-30 08:14:31.460844: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2024-07-30 08:14:31.464965: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-30 08:14:31.510415: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-07-30 08:14:31.563412: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


44/44 [==============================] - 8s 123ms/step - loss: 0.3858 - acc: 0.8304 - auc: 0.8784 - binary_accuracy: 0.8304 - recall: 0.6104 - precision: 0.7823 - val_loss: 0.7626 - val_acc: 0.6665 - val_auc: 0.6457 - val_binary_accuracy: 0.6665 - val_recall: 0.3291 - val_precision: 0.4318
Epoch 2/20
44/44 [==============================] - 2s 37ms/step - loss: 0.1524 - acc: 0.9433 - auc: 0.9869 - binary_accuracy: 0.9433 - recall: 0.8670 - precision: 0.9415 - val_loss: 0.9998 - val_acc: 0.6056 - val_auc: 0.6323 - val_binary_accuracy: 0.6056 - val_recall: 0.2520 - val_precision: 0.3113
Epoch 3/20
44/44 [==============================] - 2s 37ms/step - loss: 0.0812 - acc: 0.9718 - auc: 0.9975 - binary_accuracy: 0.9718 - recall: 0.9262 - precision: 0.9796 - val_loss: 1.1166 - val_acc: 0.6598 - val_auc: 0.6427 - val_binary_accuracy: 0.6598 - val_recall: 0.3669 - val_precision: 0.4267
Epoch 4/20
44/44 [==============================] - 2s 37ms/step - loss: 0.0486 - acc: 0.9874 - auc: 0.9995

In [40]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4087404/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [41]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_single_attention_balanced Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.6176470588235294 Recall:  0.5 Precision:  0.4375 F1:  0.4666666666666667
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_single_attention_balanced Accuracy:  0.7291666666666666 MSE:  0.2708333333333333 UAR:  0.6407563025210083 Recall:  0.42857142857142855 Precision:  0.5454545454545454 F1:  0.4799999999999999
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_single_attention_balanced Accuracy:  0.75 MSE:  0.25 UAR:  0.6554621848739496 Recall:  0.42857142857142855 Precision:  0.6 F1:  0.5
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_single_attention_balanced Accuracy:  0.75 MSE:  0.25 UAR:  0.634453781512605 Recall:  0.35714285714285715 Precision:  0.625 F1:  0.45454545454545453
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_single_attention_balanced Accuracy:  0.7291666666666666 MSE:  0.2708333333333333 UAR:  0.5987394957983193 Recall:  0.285714285714285

In [42]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [43]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4087404/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [44]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_single_attention_balanced_best Accuracy:  0.5208333333333334 MSE:  0.4791666666666667 UAR:  0.5777310924369747 Recall:  0.7142857142857143 Precision:  0.3448275862068966 F1:  0.46511627906976755
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_single_attention_balanced_best Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.6596638655462186 Recall:  0.6428571428571429 Precision:  0.45 F1:  0.5294117647058824
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_single_attention_balanced_best Accuracy:  0.6875 MSE:  0.3125 UAR:  0.6533613445378151 Recall:  0.5714285714285714 Precision:  0.47058823529411764 F1:  0.5161290322580646
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_single_attention_balanced_best Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.5966386554621849 Recall:  0.42857142857142855 Precision:  0.42857142857142855 F1:  0.42857142857142855
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_single_attention_balan

##### Self attention

In [33]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

mobilenet_7.h5_EngageWild_2_STAT_self_attention_balanced


In [34]:
CONCATENATE_STAT = True

In [35]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5631, 128, 3072) (5631,)
(2102, 128, 3072) (2102,)


In [36]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

2024-07-30 08:30:39.929705: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38431 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:11:00.0, compute capability: 8.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3072)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 3072)           9437184   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 3072)           9437184   ['image_set[0][0]']           
                                                                                                  
 attention (Attention)       (None, None, 3072)           0         ['query[0][0]',               
                                                                     'value[0][0]']           

In [37]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

In [38]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4095000/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [39]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_self_attention_balanced Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.6386554621848739 Recall:  0.5714285714285714 Precision:  0.4444444444444444 F1:  0.5
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_self_attention_balanced Accuracy:  0.6875 MSE:  0.3125 UAR:  0.6533613445378151 Recall:  0.5714285714285714 Precision:  0.47058823529411764 F1:  0.5161290322580646
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_self_attention_balanced Accuracy:  0.7083333333333334 MSE:  0.2916666666666667 UAR:  0.6470588235294117 Recall:  0.5 Precision:  0.5 F1:  0.5
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_self_attention_balanced Accuracy:  0.7083333333333334 MSE:  0.2916666666666667 UAR:  0.6470588235294117 Recall:  0.5 Precision:  0.5 F1:  0.5
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_self_attention_balanced Accuracy:  0.7083333333333334 MSE:  0.2916666666666667 UAR:  0.6470588235294117 Recall:  0.5 Precision:  0.5 F1:  0.5
Met

In [41]:
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    modelAtn.set_weights(best_model_weights)
    save_weights(modelAtn, weights_name)

In [42]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4095000/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [43]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_self_attention_balanced_best Accuracy:  0.4375 MSE:  0.5625 UAR:  0.5819327731092437 Recall:  0.9285714285714286 Precision:  0.3333333333333333 F1:  0.4905660377358491
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_self_attention_balanced_best Accuracy:  0.6041666666666666 MSE:  0.3958333333333333 UAR:  0.6785714285714286 Recall:  0.8571428571428571 Precision:  0.41379310344827586 F1:  0.5581395348837208
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_self_attention_balanced_best Accuracy:  0.7083333333333334 MSE:  0.2916666666666667 UAR:  0.73109243697479 Recall:  0.7857142857142857 Precision:  0.5 F1:  0.6111111111111112
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_self_attention_balanced_best Accuracy:  0.7291666666666666 MSE:  0.2708333333333333 UAR:  0.6827731092436975 Recall:  0.5714285714285714 Precision:  0.5333333333333333 F1:  0.5517241379310344
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_self_attention_balanced_best Accuracy

# DAiSEE dataset (4 classes)

In [33]:
FEATURES_DIR = DATA_DIR + 'features_DAiSEE/'
WEIGHTS_DIR = DATA_DIR + 'weights_DAiSEE/'
WEIGHTS_DIR = DATA_DIR + 'weights_DAiSEE_classifier/'
TABLE_NAME = '02_DAiSEE_4_classes.xlsx'
TABLE_NAME = '02_DAiSEE_4_classes_classifier.xlsx'

## enet_b0_8_best_afew

In [25]:
base_model_key = 'enet_b0_8_best_afew.pt'

In [26]:
ENGAGE_WILD_DATASET, BIN_CLASSIFICATION, N_CLASSES, DATASET_NAME, BASE_DATASET_DIR, ext = define_dataset(False, False)
video2label, labels_list = get_video2label()
print(labels_list)
print(video2label)

['very distracted', 'distracted', 'engaged', 'very engaged']
{'1100011002': 2, '1100011003': 2, '1100011004': 3, '1100011005': 3, '1100011006': 3, '1100011007': 2, '1100011008': 3, '1100011009': 2, '1100011010': 3, '1100011011': 3, '1100011012': 2, '1100011013': 3, '1100011014': 3, '1100011015': 3, '1100011016': 3, '1100011017': 3, '1100011018': 3, '1100011019': 3, '1100011020': 3, '1100011021': 3, '1100011022': 3, '1100011023': 3, '1100011025': 3, '1100011026': 3, '1100011027': 3, '1100011028': 3, '1100011029': 3, '1100011031': 3, '1100011032': 3, '1100011034': 3, '1100011035': 3, '1100011037': 3, '1100011038': 3, '1100011040': 2, '1100011046': 3, '1100011047': 3, '1100011048': 2, '1100011049': 3, '1100011050': 3, '1100011051': 3, '1100011052': 3, '1100011053': 3, '1100011054': 3, '1100011055': 3, '1100011056': 3, '1100011057': 3, '1100011058': 3, '1100011059': 3, '1100011060': 3, '1100011062': 3, '1100011063': 3, '1100011064': 3, '1100011066': 3, '1100011067': 3, '1100011068': 3, '11

### Std

In [27]:
stat_func = np.std
stat_name = "std"

#### Traditional split

In [28]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
val_pickle = FEATURES_DIR + 'val_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)
filename2features_val = load_features(val_pickle)

In [29]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))
print("Val: ", len(filename2features_val))

Train:  5482
Test:  1723
Val:  1720


In [30]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))
print("val:")
print_classes(get_labels(filename2features_val))

train:
0 34/5482: 0.6202116016052536%
1 214/5482: 3.903684786574243%
2 2649/5482: 48.32178037212696%
3 2585/5482: 47.15432323969354%
test:
0 4/1723: 0.2321532211259431%
1 81/1723: 4.7011027278003485%
2 861/1723: 49.97098084735926%
3 777/1723: 45.095763203714455%
val:
0 23/1720: 1.3372093023255813%
1 160/1720: 9.30232558139535%
2 912/1720: 53.02325581395349%
3 625/1720: 36.33720930232558%


##### Single attention

In [31]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_4_std_single_attention_traditional


In [32]:
CONCATENATE_STAT = True

In [33]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 2560) (5482,)
(1723, 128, 2560) (1723,)
(1720, 128, 2560) (1720,)


In [34]:
(unique, counts) = np.unique(y_subsample_train, return_counts=True)
num_classes=len(unique)
cw=1/counts
cw/=cw.min()
class_weights = {i:cwi for i,cwi in zip(unique,cw)}
print(counts, class_weights, num_classes, unique)

[  34  214 2649 2585] {0: 77.91176470588235, 1: 12.378504672897195, 2: 1.0, 3: 1.0247582205029013} 4 [0 1 2 3]


In [35]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)
save_best_model = SaveBestModelByUAR(X_subsample_val, y_subsample_val)

2024-08-10 08:55:02.459487: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38431 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:11:00.0, compute capability: 8.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2561      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                              

In [36]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val), class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20


2024-08-10 08:55:10.725784: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fa3ec9b8800 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-08-10 08:55:10.725946: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2024-08-10 08:55:10.730412: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-10 08:55:10.760558: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-08-10 08:55:10.814611: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


43/43 [==============================] - 7s 135ms/step - loss: 2.5561 - val_loss: 1.3405 - val_uar: 0.3606 - val_bacc: 0.3606 - val_f1: 0.3360 - val_mcc: 0.1279
Epoch 2/20
43/43 [==============================] - 3s 78ms/step - loss: 2.2692 - val_loss: 1.2353 - val_uar: 0.3522 - val_bacc: 0.3522 - val_f1: 0.3267 - val_mcc: 0.1383
Epoch 3/20
43/43 [==============================] - 3s 82ms/step - loss: 2.1245 - val_loss: 1.3937 - val_uar: 0.3901 - val_bacc: 0.3901 - val_f1: 0.2988 - val_mcc: 0.1048
Epoch 4/20
43/43 [==============================] - 3s 79ms/step - loss: 1.9588 - val_loss: 1.2686 - val_uar: 0.3653 - val_bacc: 0.3653 - val_f1: 0.3320 - val_mcc: 0.1072
Epoch 5/20
43/43 [==============================] - 3s 76ms/step - loss: 1.8101 - val_loss: 1.1521 - val_uar: 0.3425 - val_bacc: 0.3425 - val_f1: 0.3913 - val_mcc: 0.1114
Epoch 6/20
43/43 [==============================] - 3s 76ms/step - loss: 1.8585 - val_loss: 1.1547 - val_uar: 0.3283 - val_bacc: 0.3283 - val_f1: 0.3634 - 

In [37]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

In [38]:
print(pred)
print(test_labels)

[3 0 3 ... 3 0 3]
[2 1 3 ... 1 1 2]


In [39]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name, from_int=True)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_std_single_attention_traditional Accuracy:  0.4567614625652931 MSE:  0.09904793964016248 UAR:  0.34427022841656985 Recall:  N/A Precision:  N/A F1:  0.2888563158896742


(0.4567614625652931,
 0.09904793964016248,
 0.34427022841656985,
 'N/A',
 'N/A',
 0.2888563158896742)

In [40]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [41]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

In [42]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name, from_int=True)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_std_single_attention_traditional_best Accuracy:  0.375507835171213 MSE:  0.1615207196749855 UAR:  0.4206902169300544 Recall:  N/A Precision:  N/A F1:  0.2004876380940231


(0.375507835171213,
 0.1615207196749855,
 0.4206902169300544,
 'N/A',
 'N/A',
 0.2004876380940231)

##### Self attention

In [43]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_traditional


In [44]:
CONCATENATE_STAT = True
#IMAGE_SET_SIZE=32

In [33]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 2560) (5482,)
(1723, 128, 2560) (1723,)
(1720, 128, 2560) (1720,)


In [34]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

2024-08-07 21:09:58.494035: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38431 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:11:00.0, compute capability: 8.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention (Attention)       (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']           

In [35]:
if USE_GENERATORS:
    modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                     epochs=20, verbose=1, callbacks=[save_best_model],
                    validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
else:
    modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
             validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
best_model_weights = save_best_model.best_model_weights
print(save_best_model.best)

Epoch 1/20


2024-08-07 21:10:14.656351: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f86f8a32620 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-08-07 21:10:14.656520: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2024-08-07 21:10:14.660809: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-07 21:10:14.697393: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-08-07 21:10:14.749217: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


43/43 [==============================] - 8s 149ms/step - loss: 0.0813 - mae: 0.2003 - val_loss: 0.0812 - val_mae: 0.2037
Epoch 2/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.0812 - val_mae: 0.2037
Epoch 3/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.0812 - val_mae: 0.2037
Epoch 4/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.0812 - val_mae: 0.2037
Epoch 5/20
37/43 [========================>.....] - ETA: 0s - loss: 0.0797 - mae: 0.1962

KeyboardInterrupt: 

In [45]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 6s 120ms/step - loss: 0.0812 - mae: 0.2005 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 2/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 3/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 4/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 5/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 6/20
43/43 [==============================] - 4s 88ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 7/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 8/20
43/43 [==============================] - 4s 85ms/step - loss: 0.0796 - 

In [46]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [47]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_traditional Accuracy:  0.4509576320371445 MSE:  0.08119123621590248 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.1554


(0.4509576320371445, 0.08119123621590248, 0.25, 'N/A', 'N/A', 0.1554)

In [48]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [49]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [50]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_traditional_best Accuracy:  0.4509576320371445 MSE:  0.08119123621590248 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.1554


(0.4509576320371445, 0.08119123621590248, 0.25, 'N/A', 'N/A', 0.1554)

#### Balanced split

In [45]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [46]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  5482
Test:  1723


In [47]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))

train:
0 34/5482: 0.6202116016052536%
1 214/5482: 3.903684786574243%
2 2649/5482: 48.32178037212696%
3 2585/5482: 47.15432323969354%
test:
0 4/1723: 0.2321532211259431%
1 81/1723: 4.7011027278003485%
2 861/1723: 49.97098084735926%
3 777/1723: 45.095763203714455%


In [48]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.76)

(5482,) (5482,)
(1723,) (1723,)
(7205,) (7205,)
[0 1 2 3] [  38  295 3510 3362]
7205
5474
1731


In [49]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))

train:
0 28/5474: 0.5115089514066496%
1 224/5474: 4.092071611253197%
2 2667/5474: 48.72122762148338%
3 2555/5474: 46.67519181585678%
test:
0 10/1731: 0.5777007510109763%
1 71/1731: 4.101675332177932%
2 843/1731: 48.7001733102253%
3 807/1731: 46.62045060658579%


##### Single attention

In [50]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_4_std_single_attention_balanced


In [51]:
CONCATENATE_STAT = True

In [52]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5474, 128, 2560) (5474,)
(1731, 128, 2560) (1731,)


In [53]:
(unique, counts) = np.unique(y_subsample_train, return_counts=True)
num_classes=len(unique)
cw=1/counts
cw/=cw.min()
class_weights = {i:cwi for i,cwi in zip(unique,cw)}
print(counts, class_weights, num_classes, unique)

[  28  224 2667 2555] {0: 95.24999999999999, 1: 11.906249999999998, 2: 1.0, 3: 1.0438356164383562} 4 [0 1 2 3]


In [54]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)
save_best_model = SaveBestModelByUAR(X_subsample_test, y_subsample_test)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2561      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [55]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test), class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 5s 107ms/step - loss: 2.8604 - val_loss: 1.1903 - val_uar: 0.4301 - val_bacc: 0.4301 - val_f1: 0.4552 - val_mcc: 0.1641
Epoch 2/20
43/43 [==============================] - 3s 67ms/step - loss: 2.1280 - val_loss: 1.1279 - val_uar: 0.4507 - val_bacc: 0.4507 - val_f1: 0.4726 - val_mcc: 0.2066
Epoch 3/20
43/43 [==============================] - 3s 75ms/step - loss: 2.0309 - val_loss: 1.1296 - val_uar: 0.4706 - val_bacc: 0.4706 - val_f1: 0.4477 - val_mcc: 0.2120
Epoch 4/20
43/43 [==============================] - 3s 69ms/step - loss: 1.9098 - val_loss: 0.9823 - val_uar: 0.4863 - val_bacc: 0.4863 - val_f1: 0.5517 - val_mcc: 0.2566
Epoch 5/20
43/43 [==============================] - 3s 69ms/step - loss: 1.9175 - val_loss: 1.1944 - val_uar: 0.4808 - val_bacc: 0.4808 - val_f1: 0.4304 - val_mcc: 0.1860
Epoch 6/20
43/43 [==============================] - 3s 69ms/step - loss: 1.7380 - val_loss: 1.0721 - val_uar: 0.4593 - val_bacc: 0.4593 - val_f1

In [56]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

In [57]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name, from_int=True)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_std_single_attention_balanced Accuracy:  0.5459272097053726 MSE:  0.0820589832466782 UAR:  0.48608896318905737 Recall:  N/A Precision:  N/A F1:  0.3703024732283354


(0.5459272097053726,
 0.0820589832466782,
 0.48608896318905737,
 'N/A',
 'N/A',
 0.3703024732283354)

In [58]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [59]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

In [60]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name, from_int=True)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_std_single_attention_balanced_best Accuracy:  0.5262853841709995 MSE:  0.0799743500866551 UAR:  0.5205462568546967 Recall:  N/A Precision:  N/A F1:  0.37577872510979515


(0.5262853841709995,
 0.0799743500866551,
 0.5205462568546967,
 'N/A',
 'N/A',
 0.37577872510979515)

##### Self attention

In [75]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_balanced


In [76]:
CONCATENATE_STAT = True

In [77]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5474, 128, 2560) (5474,)
(1731, 128, 2560) (1731,)


In [78]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention_1 (Attention)     (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [79]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 6s 125ms/step - loss: 0.0812 - mae: 0.2012 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 2/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 3/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 4/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 5/20
43/43 [==============================] - 4s 88ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 6/20
43/43 [==============================] - 4s 88ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 7/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 8/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0798 - 

In [80]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [81]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_balanced Accuracy:  0.4662045060658579 MSE:  0.08048682842287694 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.15898345153664303


(0.4662045060658579,
 0.08048682842287694,
 0.25,
 'N/A',
 'N/A',
 0.15898345153664303)

In [82]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [83]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [84]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_balanced_best Accuracy:  0.4662045060658579 MSE:  0.08048682842287694 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.15898345153664303


(0.4662045060658579,
 0.08048682842287694,
 0.25,
 'N/A',
 'N/A',
 0.15898345153664303)

### Max

In [61]:
stat_func = np.max
stat_name = "max"

#### Traditional split

In [62]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
val_pickle = FEATURES_DIR + 'val_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)
filename2features_val = load_features(val_pickle)

In [63]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))
print("Val: ", len(filename2features_val))

Train:  5482
Test:  1723
Val:  1720


In [64]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))
print("val:")
print_classes(get_labels(filename2features_val))

train:
0 34/5482: 0.6202116016052536%
1 214/5482: 3.903684786574243%
2 2649/5482: 48.32178037212696%
3 2585/5482: 47.15432323969354%
test:
0 4/1723: 0.2321532211259431%
1 81/1723: 4.7011027278003485%
2 861/1723: 49.97098084735926%
3 777/1723: 45.095763203714455%
val:
0 23/1720: 1.3372093023255813%
1 160/1720: 9.30232558139535%
2 912/1720: 53.02325581395349%
3 625/1720: 36.33720930232558%


##### Single attention

In [65]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_4_max_single_attention_traditional


In [66]:
CONCATENATE_STAT = True

In [67]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 2560) (5482,)
(1723, 128, 2560) (1723,)
(1720, 128, 2560) (1720,)


In [68]:
(unique, counts) = np.unique(y_subsample_train, return_counts=True)
num_classes=len(unique)
cw=1/counts
cw/=cw.min()
class_weights = {i:cwi for i,cwi in zip(unique,cw)}
print(counts, class_weights, num_classes, unique)

[  34  214 2649 2585] {0: 77.91176470588235, 1: 12.378504672897195, 2: 1.0, 3: 1.0247582205029013} 4 [0 1 2 3]


In [69]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)
save_best_model = SaveBestModelByUAR(X_subsample_val, y_subsample_val)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2561      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [70]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val), class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 5s 101ms/step - loss: 2.8206 - val_loss: 1.3666 - val_uar: 0.3398 - val_bacc: 0.3398 - val_f1: 0.3017 - val_mcc: 0.0761
Epoch 2/20
43/43 [==============================] - 2s 57ms/step - loss: 2.1727 - val_loss: 1.1116 - val_uar: 0.3239 - val_bacc: 0.3239 - val_f1: 0.3901 - val_mcc: 0.1237
Epoch 3/20
43/43 [==============================] - 2s 58ms/step - loss: 2.1942 - val_loss: 1.1891 - val_uar: 0.2761 - val_bacc: 0.2761 - val_f1: 0.3953 - val_mcc: 0.1055
Epoch 4/20
43/43 [==============================] - 2s 56ms/step - loss: 1.9493 - val_loss: 1.2498 - val_uar: 0.3123 - val_bacc: 0.3123 - val_f1: 0.3419 - val_mcc: 0.1023
Epoch 5/20
43/43 [==============================] - 2s 58ms/step - loss: 1.7473 - val_loss: 1.2055 - val_uar: 0.3477 - val_bacc: 0.3477 - val_f1: 0.3779 - val_mcc: 0.1319
Epoch 6/20
43/43 [==============================] - 2s 58ms/step - loss: 1.7498 - val_loss: 1.1955 - val_uar: 0.3337 - val_bacc: 0.3337 - val_f1

In [71]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

In [72]:
print(pred)
print(test_labels)

[3 0 3 ... 3 1 3]
[2 1 3 ... 1 1 2]


In [73]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name, from_int=True)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_max_single_attention_traditional Accuracy:  0.5089959373186302 MSE:  0.07671346488682529 UAR:  0.3447078757038107 Recall:  N/A Precision:  N/A F1:  0.28946179418657036


(0.5089959373186302,
 0.07671346488682529,
 0.3447078757038107,
 'N/A',
 'N/A',
 0.28946179418657036)

In [74]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [75]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

In [76]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name, from_int=True)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_max_single_attention_traditional_best Accuracy:  0.4265815438189205 MSE:  0.1339059199071387 UAR:  0.40709364997982883 Recall:  N/A Precision:  N/A F1:  0.2672821730663535


(0.4265815438189205,
 0.1339059199071387,
 0.40709364997982883,
 'N/A',
 'N/A',
 0.2672821730663535)

##### Self attention

In [31]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_traditional


In [32]:
CONCATENATE_STAT = True
#IMAGE_SET_SIZE=32

In [33]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 2560) (5482,)
(1723, 128, 2560) (1723,)
(1720, 128, 2560) (1720,)


In [34]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

2024-08-07 21:09:58.494035: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38431 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:11:00.0, compute capability: 8.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention (Attention)       (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']           

In [35]:
if USE_GENERATORS:
    modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                     epochs=20, verbose=1, callbacks=[save_best_model],
                    validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
else:
    modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
             validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
best_model_weights = save_best_model.best_model_weights
print(save_best_model.best)

Epoch 1/20


2024-08-07 21:10:14.656351: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f86f8a32620 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-08-07 21:10:14.656520: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2024-08-07 21:10:14.660809: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-07 21:10:14.697393: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-08-07 21:10:14.749217: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


43/43 [==============================] - 8s 149ms/step - loss: 0.0813 - mae: 0.2003 - val_loss: 0.0812 - val_mae: 0.2037
Epoch 2/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.0812 - val_mae: 0.2037
Epoch 3/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.0812 - val_mae: 0.2037
Epoch 4/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.0812 - val_mae: 0.2037
Epoch 5/20
37/43 [========================>.....] - ETA: 0s - loss: 0.0797 - mae: 0.1962

KeyboardInterrupt: 

In [45]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 6s 120ms/step - loss: 0.0812 - mae: 0.2005 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 2/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 3/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 4/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 5/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 6/20
43/43 [==============================] - 4s 88ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 7/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 8/20
43/43 [==============================] - 4s 85ms/step - loss: 0.0796 - 

In [46]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [47]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_traditional Accuracy:  0.4509576320371445 MSE:  0.08119123621590248 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.1554


(0.4509576320371445, 0.08119123621590248, 0.25, 'N/A', 'N/A', 0.1554)

In [48]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [49]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [50]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_traditional_best Accuracy:  0.4509576320371445 MSE:  0.08119123621590248 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.1554


(0.4509576320371445, 0.08119123621590248, 0.25, 'N/A', 'N/A', 0.1554)

#### Balanced split

In [77]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [78]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  5482
Test:  1723


In [79]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))

train:
0 34/5482: 0.6202116016052536%
1 214/5482: 3.903684786574243%
2 2649/5482: 48.32178037212696%
3 2585/5482: 47.15432323969354%
test:
0 4/1723: 0.2321532211259431%
1 81/1723: 4.7011027278003485%
2 861/1723: 49.97098084735926%
3 777/1723: 45.095763203714455%


In [80]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.76)

(5482,) (5482,)
(1723,) (1723,)
(7205,) (7205,)
[0 1 2 3] [  38  295 3510 3362]
7205
5474
1731


In [81]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))

train:
0 28/5474: 0.5115089514066496%
1 224/5474: 4.092071611253197%
2 2667/5474: 48.72122762148338%
3 2555/5474: 46.67519181585678%
test:
0 10/1731: 0.5777007510109763%
1 71/1731: 4.101675332177932%
2 843/1731: 48.7001733102253%
3 807/1731: 46.62045060658579%


##### Single attention

In [82]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_4_max_single_attention_balanced


In [83]:
CONCATENATE_STAT = True

In [84]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5474, 128, 2560) (5474,)
(1731, 128, 2560) (1731,)


In [85]:
(unique, counts) = np.unique(y_subsample_train, return_counts=True)
num_classes=len(unique)
cw=1/counts
cw/=cw.min()
class_weights = {i:cwi for i,cwi in zip(unique,cw)}
print(counts, class_weights, num_classes, unique)

[  28  224 2667 2555] {0: 95.24999999999999, 1: 11.906249999999998, 2: 1.0, 3: 1.0438356164383562} 4 [0 1 2 3]


In [86]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)
save_best_model = SaveBestModelByUAR(X_subsample_test, y_subsample_test)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2561      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [87]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test), class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 5s 103ms/step - loss: 2.9690 - val_loss: 1.1538 - val_uar: 0.4879 - val_bacc: 0.4879 - val_f1: 0.5014 - val_mcc: 0.2059
Epoch 2/20
43/43 [==============================] - 3s 70ms/step - loss: 2.1228 - val_loss: 1.1295 - val_uar: 0.4565 - val_bacc: 0.4565 - val_f1: 0.4645 - val_mcc: 0.2017
Epoch 3/20
43/43 [==============================] - 3s 69ms/step - loss: 2.0177 - val_loss: 1.1079 - val_uar: 0.4449 - val_bacc: 0.4449 - val_f1: 0.4835 - val_mcc: 0.2197
Epoch 4/20
43/43 [==============================] - 3s 78ms/step - loss: 1.8869 - val_loss: 1.1620 - val_uar: 0.4830 - val_bacc: 0.4830 - val_f1: 0.4321 - val_mcc: 0.1871
Epoch 5/20
43/43 [==============================] - 3s 77ms/step - loss: 1.7714 - val_loss: 1.0371 - val_uar: 0.4639 - val_bacc: 0.4639 - val_f1: 0.5228 - val_mcc: 0.2363
Epoch 6/20
43/43 [==============================] - 3s 70ms/step - loss: 1.6597 - val_loss: 1.1303 - val_uar: 0.4729 - val_bacc: 0.4729 - val_f1

In [88]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

In [89]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name, from_int=True)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_max_single_attention_balanced Accuracy:  0.5522819179664934 MSE:  0.07418399768919698 UAR:  0.4860207197845378 Recall:  N/A Precision:  N/A F1:  0.3891380836798306


(0.5522819179664934,
 0.07418399768919698,
 0.4860207197845378,
 'N/A',
 'N/A',
 0.3891380836798306)

In [90]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [91]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

In [92]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name, from_int=True)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_max_single_attention_balanced_best Accuracy:  0.514153668399769 MSE:  0.09369428076256497 UAR:  0.5192991649243248 Recall:  N/A Precision:  N/A F1:  0.36274105669822676


(0.514153668399769,
 0.09369428076256497,
 0.5192991649243248,
 'N/A',
 'N/A',
 0.36274105669822676)

##### Self attention

In [75]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_balanced


In [76]:
CONCATENATE_STAT = True

In [77]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5474, 128, 2560) (5474,)
(1731, 128, 2560) (1731,)


In [78]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention_1 (Attention)     (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [79]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 6s 125ms/step - loss: 0.0812 - mae: 0.2012 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 2/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 3/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 4/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 5/20
43/43 [==============================] - 4s 88ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 6/20
43/43 [==============================] - 4s 88ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 7/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 8/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0798 - 

In [80]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [81]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_balanced Accuracy:  0.4662045060658579 MSE:  0.08048682842287694 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.15898345153664303


(0.4662045060658579,
 0.08048682842287694,
 0.25,
 'N/A',
 'N/A',
 0.15898345153664303)

In [82]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [83]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [84]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_balanced_best Accuracy:  0.4662045060658579 MSE:  0.08048682842287694 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.15898345153664303


(0.4662045060658579,
 0.08048682842287694,
 0.25,
 'N/A',
 'N/A',
 0.15898345153664303)

### STAT

In [75]:
stat_func = compute_descriptor
stat_name = "STAT"

#### Traditional split

In [28]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
val_pickle = FEATURES_DIR + 'val_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)
filename2features_val = load_features(val_pickle)

In [29]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))
print("Val: ", len(filename2features_val))

Train:  5482
Test:  1723
Val:  1720


In [30]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))
print("val:")
print_classes(get_labels(filename2features_val))

train:
0 34/5482: 0.6202116016052536%
1 214/5482: 3.903684786574243%
2 2649/5482: 48.32178037212696%
3 2585/5482: 47.15432323969354%
test:
0 4/1723: 0.2321532211259431%
1 81/1723: 4.7011027278003485%
2 861/1723: 49.97098084735926%
3 777/1723: 45.095763203714455%
val:
0 23/1720: 1.3372093023255813%
1 160/1720: 9.30232558139535%
2 912/1720: 53.02325581395349%
3 625/1720: 36.33720930232558%


##### Single attention

In [31]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_4_STAT_single_attention_traditional


In [32]:
CONCATENATE_STAT = True

In [33]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 3840) (5482,)
(1723, 128, 3840) (1723,)
(1720, 128, 3840) (1720,)


In [34]:
(unique, counts) = np.unique(y_subsample_train, return_counts=True)
num_classes=len(unique)
cw=1/counts
cw/=cw.min()
class_weights = {i:cwi for i,cwi in zip(unique,cw)}
print(counts, class_weights, num_classes, unique)

[  34  214 2649 2585] {0: 77.91176470588235, 1: 12.378504672897195, 2: 1.0, 3: 1.0247582205029013} 4 [0 1 2 3]


In [35]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)
save_best_model = SaveBestModelByUAR(X_subsample_val, y_subsample_val)

2024-08-10 09:14:07.086533: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38431 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:11:00.0, compute capability: 8.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3840)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              3841      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                              

In [36]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=11, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val), class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/11


2024-08-10 09:14:18.892667: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fc8001f51b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-08-10 09:14:18.892831: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2024-08-10 09:14:18.897138: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-10 09:14:18.931061: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-08-10 09:14:18.984733: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


43/43 [==============================] - 11s 214ms/step - loss: 2.7601 - val_loss: 1.4051 - val_uar: 0.3376 - val_bacc: 0.3376 - val_f1: 0.2558 - val_mcc: 0.1220
Epoch 2/11
43/43 [==============================] - 5s 127ms/step - loss: 2.1591 - val_loss: 1.2207 - val_uar: 0.3482 - val_bacc: 0.3482 - val_f1: 0.3727 - val_mcc: 0.1417
Epoch 3/11
43/43 [==============================] - 6s 132ms/step - loss: 2.0772 - val_loss: 1.3887 - val_uar: 0.3360 - val_bacc: 0.3360 - val_f1: 0.2936 - val_mcc: 0.1190
Epoch 4/11
43/43 [==============================] - 6s 131ms/step - loss: 1.9000 - val_loss: 1.3405 - val_uar: 0.3756 - val_bacc: 0.3756 - val_f1: 0.3157 - val_mcc: 0.1088
Epoch 5/11
43/43 [==============================] - 6s 135ms/step - loss: 1.8157 - val_loss: 1.5789 - val_uar: 0.3670 - val_bacc: 0.3670 - val_f1: 0.2448 - val_mcc: 0.0955
Epoch 6/11
43/43 [==============================] - 5s 128ms/step - loss: 1.8149 - val_loss: 1.4600 - val_uar: 0.4052 - val_bacc: 0.4052 - val_f1: 0.2

In [37]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

In [38]:
print(pred)
print(test_labels)

[3 0 3 ... 1 0 2]
[2 1 3 ... 1 1 2]


In [39]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name, from_int=True)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_STAT_single_attention_traditional Accuracy:  0.526407428903076 MSE:  0.09123847939640162 UAR:  0.44104862702423675 Recall:  N/A Precision:  N/A F1:  0.3261485401548058


(0.526407428903076,
 0.09123847939640162,
 0.44104862702423675,
 'N/A',
 'N/A',
 0.3261485401548058)

In [40]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [41]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

In [42]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name, from_int=True)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_STAT_single_attention_traditional_best Accuracy:  0.367382472431805 MSE:  0.15167486941381308 UAR:  0.4381775067750677 Recall:  N/A Precision:  N/A F1:  0.20662949234996184


(0.367382472431805,
 0.15167486941381308,
 0.4381775067750677,
 'N/A',
 'N/A',
 0.20662949234996184)

##### Self attention

In [31]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_traditional


In [32]:
CONCATENATE_STAT = True
#IMAGE_SET_SIZE=32

In [33]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 2560) (5482,)
(1723, 128, 2560) (1723,)
(1720, 128, 2560) (1720,)


In [34]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

2024-08-07 21:09:58.494035: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38431 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:11:00.0, compute capability: 8.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention (Attention)       (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']           

In [35]:
if USE_GENERATORS:
    modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                     epochs=20, verbose=1, callbacks=[save_best_model],
                    validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
else:
    modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
             validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
best_model_weights = save_best_model.best_model_weights
print(save_best_model.best)

Epoch 1/20


2024-08-07 21:10:14.656351: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f86f8a32620 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-08-07 21:10:14.656520: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2024-08-07 21:10:14.660809: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-07 21:10:14.697393: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-08-07 21:10:14.749217: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


43/43 [==============================] - 8s 149ms/step - loss: 0.0813 - mae: 0.2003 - val_loss: 0.0812 - val_mae: 0.2037
Epoch 2/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.0812 - val_mae: 0.2037
Epoch 3/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.0812 - val_mae: 0.2037
Epoch 4/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.0812 - val_mae: 0.2037
Epoch 5/20
37/43 [========================>.....] - ETA: 0s - loss: 0.0797 - mae: 0.1962

KeyboardInterrupt: 

In [45]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 6s 120ms/step - loss: 0.0812 - mae: 0.2005 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 2/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 3/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 4/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 5/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 6/20
43/43 [==============================] - 4s 88ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 7/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 8/20
43/43 [==============================] - 4s 85ms/step - loss: 0.0796 - 

In [46]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [47]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_traditional Accuracy:  0.4509576320371445 MSE:  0.08119123621590248 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.1554


(0.4509576320371445, 0.08119123621590248, 0.25, 'N/A', 'N/A', 0.1554)

In [48]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [49]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [50]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_traditional_best Accuracy:  0.4509576320371445 MSE:  0.08119123621590248 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.1554


(0.4509576320371445, 0.08119123621590248, 0.25, 'N/A', 'N/A', 0.1554)

#### Balanced split

In [43]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [44]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  5482
Test:  1723


In [45]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))

train:
0 34/5482: 0.6202116016052536%
1 214/5482: 3.903684786574243%
2 2649/5482: 48.32178037212696%
3 2585/5482: 47.15432323969354%
test:
0 4/1723: 0.2321532211259431%
1 81/1723: 4.7011027278003485%
2 861/1723: 49.97098084735926%
3 777/1723: 45.095763203714455%


In [46]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.76)

(5482, 2560) (5482,)
(1723, 2560) (1723,)
(7205, 2560) (7205,)
[0 1 2 3] [  38  295 3510 3362]
7205
5474
1731


In [47]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))

train:
0 28/5474: 0.5115089514066496%
1 224/5474: 4.092071611253197%
2 2667/5474: 48.72122762148338%
3 2555/5474: 46.67519181585678%
test:
0 10/1731: 0.5777007510109763%
1 71/1731: 4.101675332177932%
2 843/1731: 48.7001733102253%
3 807/1731: 46.62045060658579%


##### Single attention

In [48]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_4_STAT_single_attention_balanced


In [49]:
CONCATENATE_STAT = True

In [50]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5474, 128, 3840) (5474,)
(1731, 128, 3840) (1731,)


In [51]:
(unique, counts) = np.unique(y_subsample_train, return_counts=True)
num_classes=len(unique)
cw=1/counts
cw/=cw.min()
class_weights = {i:cwi for i,cwi in zip(unique,cw)}
print(counts, class_weights, num_classes, unique)

[  28  224 2667 2555] {0: 95.24999999999999, 1: 11.906249999999998, 2: 1.0, 3: 1.0438356164383562} 4 [0 1 2 3]


In [52]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)
save_best_model = SaveBestModelByUAR(X_subsample_test, y_subsample_test)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3840)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              3841      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [53]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=11, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test), class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/11
43/43 [==============================] - 7s 150ms/step - loss: 2.8207 - val_loss: 1.4596 - val_uar: 0.4073 - val_bacc: 0.4073 - val_f1: 0.3033 - val_mcc: 0.1223
Epoch 2/11
43/43 [==============================] - 4s 83ms/step - loss: 2.2515 - val_loss: 1.3171 - val_uar: 0.4337 - val_bacc: 0.4337 - val_f1: 0.3911 - val_mcc: 0.1543
Epoch 3/11
43/43 [==============================] - 4s 84ms/step - loss: 1.9962 - val_loss: 1.0595 - val_uar: 0.4939 - val_bacc: 0.4939 - val_f1: 0.5142 - val_mcc: 0.2301
Epoch 4/11
43/43 [==============================] - 4s 83ms/step - loss: 1.8844 - val_loss: 1.0038 - val_uar: 0.4905 - val_bacc: 0.4905 - val_f1: 0.5448 - val_mcc: 0.2479
Epoch 5/11
43/43 [==============================] - 4s 87ms/step - loss: 1.7452 - val_loss: 0.9520 - val_uar: 0.4975 - val_bacc: 0.4975 - val_f1: 0.5511 - val_mcc: 0.2571
Epoch 6/11
43/43 [==============================] - 4s 85ms/step - loss: 1.7521 - val_loss: 0.9479 - val_uar: 0.4483 - val_bacc: 0.4483 - val_f1

In [54]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

In [55]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name, from_int=True)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_STAT_single_attention_balanced Accuracy:  0.512420566146736 MSE:  0.0906271519352975 UAR:  0.4792070156766358 Recall:  N/A Precision:  N/A F1:  0.35138349223233034


(0.512420566146736,
 0.0906271519352975,
 0.4792070156766358,
 'N/A',
 'N/A',
 0.35138349223233034)

In [56]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [57]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

In [58]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name, from_int=True)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_STAT_single_attention_balanced_best Accuracy:  0.5309069901790873 MSE:  0.09154962449451182 UAR:  0.5113796847505633 Recall:  N/A Precision:  N/A F1:  0.36558525970956424


(0.5309069901790873,
 0.09154962449451182,
 0.5113796847505633,
 'N/A',
 'N/A',
 0.36558525970956424)

##### Self attention

In [75]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_balanced


In [76]:
CONCATENATE_STAT = True

In [77]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5474, 128, 2560) (5474,)
(1731, 128, 2560) (1731,)


In [78]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention_1 (Attention)     (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [79]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 6s 125ms/step - loss: 0.0812 - mae: 0.2012 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 2/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 3/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 4/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 5/20
43/43 [==============================] - 4s 88ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 6/20
43/43 [==============================] - 4s 88ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 7/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 8/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0798 - 

In [80]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [81]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_balanced Accuracy:  0.4662045060658579 MSE:  0.08048682842287694 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.15898345153664303


(0.4662045060658579,
 0.08048682842287694,
 0.25,
 'N/A',
 'N/A',
 0.15898345153664303)

In [82]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [83]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [84]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_balanced_best Accuracy:  0.4662045060658579 MSE:  0.08048682842287694 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.15898345153664303


(0.4662045060658579,
 0.08048682842287694,
 0.25,
 'N/A',
 'N/A',
 0.15898345153664303)

## MobileNet_7

In [34]:
base_model_key = 'mobilenet_7.h5'

In [35]:
ENGAGE_WILD_DATASET, BIN_CLASSIFICATION, N_CLASSES, DATASET_NAME, BASE_DATASET_DIR, ext = define_dataset(False, False)
video2label, labels_list = get_video2label()
print(labels_list)
print(video2label)

['very distracted', 'distracted', 'engaged', 'very engaged']
{'1100011002': 2, '1100011003': 2, '1100011004': 3, '1100011005': 3, '1100011006': 3, '1100011007': 2, '1100011008': 3, '1100011009': 2, '1100011010': 3, '1100011011': 3, '1100011012': 2, '1100011013': 3, '1100011014': 3, '1100011015': 3, '1100011016': 3, '1100011017': 3, '1100011018': 3, '1100011019': 3, '1100011020': 3, '1100011021': 3, '1100011022': 3, '1100011023': 3, '1100011025': 3, '1100011026': 3, '1100011027': 3, '1100011028': 3, '1100011029': 3, '1100011031': 3, '1100011032': 3, '1100011034': 3, '1100011035': 3, '1100011037': 3, '1100011038': 3, '1100011040': 2, '1100011046': 3, '1100011047': 3, '1100011048': 2, '1100011049': 3, '1100011050': 3, '1100011051': 3, '1100011052': 3, '1100011053': 3, '1100011054': 3, '1100011055': 3, '1100011056': 3, '1100011057': 3, '1100011058': 3, '1100011059': 3, '1100011060': 3, '1100011062': 3, '1100011063': 3, '1100011064': 3, '1100011066': 3, '1100011067': 3, '1100011068': 3, '11

### Std

In [36]:
stat_func = np.std
stat_name = "std"

#### Traditional split

In [37]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
val_pickle = FEATURES_DIR + 'val_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)
filename2features_val = load_features(val_pickle)

In [38]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))
print("Val: ", len(filename2features_val))

Train:  5482
Test:  1723
Val:  1720


In [39]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))
print("val:")
print_classes(get_labels(filename2features_val))

train:
0 34/5482: 0.6202116016052536%
1 214/5482: 3.903684786574243%
2 2649/5482: 48.32178037212696%
3 2585/5482: 47.15432323969354%
test:
0 4/1723: 0.2321532211259431%
1 81/1723: 4.7011027278003485%
2 861/1723: 49.97098084735926%
3 777/1723: 45.095763203714455%
val:
0 23/1720: 1.3372093023255813%
1 160/1720: 9.30232558139535%
2 912/1720: 53.02325581395349%
3 625/1720: 36.33720930232558%


##### Single attention

In [40]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

mobilenet_7.h5_DAiSEE_4_std_single_attention_traditional


In [41]:
CONCATENATE_STAT = True

In [42]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 2048) (5482,)
(1723, 128, 2048) (1723,)
(1720, 128, 2048) (1720,)


In [43]:
(unique, counts) = np.unique(y_subsample_train, return_counts=True)
num_classes=len(unique)
cw=1/counts
cw/=cw.min()
class_weights = {i:cwi for i,cwi in zip(unique,cw)}
print(counts, class_weights, num_classes, unique)

[  34  214 2649 2585] {0: 77.91176470588235, 1: 12.378504672897195, 2: 1.0, 3: 1.0247582205029013} 4 [0 1 2 3]


In [44]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)
save_best_model = SaveBestModelByUAR(X_subsample_val, y_subsample_val)

2024-08-10 09:28:38.764191: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38431 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:11:00.0, compute capability: 8.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2049      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                              

In [45]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val), class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20


2024-08-10 09:28:45.676980: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f4f2d2da680 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-08-10 09:28:45.677140: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2024-08-10 09:28:45.681443: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-10 09:28:45.713825: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-08-10 09:28:45.767543: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


43/43 [==============================] - 6s 114ms/step - loss: 2.8375 - val_loss: 1.2770 - val_uar: 0.3432 - val_bacc: 0.3432 - val_f1: 0.3180 - val_mcc: 0.1318
Epoch 2/20
43/43 [==============================] - 3s 65ms/step - loss: 2.2408 - val_loss: 1.5110 - val_uar: 0.3283 - val_bacc: 0.3283 - val_f1: 0.2390 - val_mcc: 0.0903
Epoch 3/20
43/43 [==============================] - 3s 65ms/step - loss: 2.0112 - val_loss: 1.3204 - val_uar: 0.3666 - val_bacc: 0.3666 - val_f1: 0.3297 - val_mcc: 0.1206
Epoch 4/20
43/43 [==============================] - 3s 76ms/step - loss: 1.9443 - val_loss: 1.3231 - val_uar: 0.3775 - val_bacc: 0.3775 - val_f1: 0.3547 - val_mcc: 0.1364
Epoch 5/20
43/43 [==============================] - 3s 72ms/step - loss: 1.7967 - val_loss: 1.2357 - val_uar: 0.3618 - val_bacc: 0.3618 - val_f1: 0.3459 - val_mcc: 0.1445
Epoch 6/20
43/43 [==============================] - 3s 74ms/step - loss: 1.8052 - val_loss: 1.1063 - val_uar: 0.3697 - val_bacc: 0.3697 - val_f1: 0.4477 - 

In [46]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

In [47]:
print(pred)
print(test_labels)

[1 1 2 ... 1 1 3]
[2 1 3 ... 1 1 2]


In [48]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name, from_int=True)

Metric_name:  mobilenet_7.h5_DAiSEE_4_std_single_attention_traditional Accuracy:  0.3847939640162507 MSE:  0.1150275101567034 UAR:  0.3215913358189781 Recall:  N/A Precision:  N/A F1:  0.2559559740639829


(0.3847939640162507,
 0.1150275101567034,
 0.3215913358189781,
 'N/A',
 'N/A',
 0.2559559740639829)

In [49]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [50]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

In [51]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name, from_int=True)

Metric_name:  mobilenet_7.h5_DAiSEE_4_std_single_attention_traditional_best Accuracy:  0.3232733604178758 MSE:  0.19624248403946606 UAR:  0.37646197300668843 Recall:  N/A Precision:  N/A F1:  0.2209256312589682


(0.3232733604178758,
 0.19624248403946606,
 0.37646197300668843,
 'N/A',
 'N/A',
 0.2209256312589682)

##### Self attention

In [43]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_traditional


In [44]:
CONCATENATE_STAT = True
#IMAGE_SET_SIZE=32

In [33]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 2560) (5482,)
(1723, 128, 2560) (1723,)
(1720, 128, 2560) (1720,)


In [34]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

2024-08-07 21:09:58.494035: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38431 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:11:00.0, compute capability: 8.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention (Attention)       (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']           

In [35]:
if USE_GENERATORS:
    modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                     epochs=20, verbose=1, callbacks=[save_best_model],
                    validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
else:
    modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
             validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
best_model_weights = save_best_model.best_model_weights
print(save_best_model.best)

Epoch 1/20


2024-08-07 21:10:14.656351: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f86f8a32620 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-08-07 21:10:14.656520: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2024-08-07 21:10:14.660809: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-07 21:10:14.697393: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-08-07 21:10:14.749217: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


43/43 [==============================] - 8s 149ms/step - loss: 0.0813 - mae: 0.2003 - val_loss: 0.0812 - val_mae: 0.2037
Epoch 2/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.0812 - val_mae: 0.2037
Epoch 3/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.0812 - val_mae: 0.2037
Epoch 4/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.0812 - val_mae: 0.2037
Epoch 5/20
37/43 [========================>.....] - ETA: 0s - loss: 0.0797 - mae: 0.1962

KeyboardInterrupt: 

In [45]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 6s 120ms/step - loss: 0.0812 - mae: 0.2005 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 2/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 3/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 4/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 5/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 6/20
43/43 [==============================] - 4s 88ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 7/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 8/20
43/43 [==============================] - 4s 85ms/step - loss: 0.0796 - 

In [46]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [47]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_traditional Accuracy:  0.4509576320371445 MSE:  0.08119123621590248 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.1554


(0.4509576320371445, 0.08119123621590248, 0.25, 'N/A', 'N/A', 0.1554)

In [48]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [49]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [50]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_traditional_best Accuracy:  0.4509576320371445 MSE:  0.08119123621590248 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.1554


(0.4509576320371445, 0.08119123621590248, 0.25, 'N/A', 'N/A', 0.1554)

#### Balanced split

In [52]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [53]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  5482
Test:  1723


In [54]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))

train:
0 34/5482: 0.6202116016052536%
1 214/5482: 3.903684786574243%
2 2649/5482: 48.32178037212696%
3 2585/5482: 47.15432323969354%
test:
0 4/1723: 0.2321532211259431%
1 81/1723: 4.7011027278003485%
2 861/1723: 49.97098084735926%
3 777/1723: 45.095763203714455%


In [55]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.76)

(5482,) (5482,)
(1723,) (1723,)
(7205,) (7205,)
[0 1 2 3] [  38  295 3510 3362]
7205
5474
1731


In [56]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))

train:
0 28/5474: 0.5115089514066496%
1 224/5474: 4.092071611253197%
2 2667/5474: 48.72122762148338%
3 2555/5474: 46.67519181585678%
test:
0 10/1731: 0.5777007510109763%
1 71/1731: 4.101675332177932%
2 843/1731: 48.7001733102253%
3 807/1731: 46.62045060658579%


##### Single attention

In [57]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

mobilenet_7.h5_DAiSEE_4_std_single_attention_balanced


In [58]:
CONCATENATE_STAT = True

In [59]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5474, 128, 2048) (5474,)
(1731, 128, 2048) (1731,)


In [60]:
(unique, counts) = np.unique(y_subsample_train, return_counts=True)
num_classes=len(unique)
cw=1/counts
cw/=cw.min()
class_weights = {i:cwi for i,cwi in zip(unique,cw)}
print(counts, class_weights, num_classes, unique)

[  28  224 2667 2555] {0: 95.24999999999999, 1: 11.906249999999998, 2: 1.0, 3: 1.0438356164383562} 4 [0 1 2 3]


In [61]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)
save_best_model = SaveBestModelByUAR(X_subsample_test, y_subsample_test)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2049      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [62]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test), class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 4s 84ms/step - loss: 2.8562 - val_loss: 1.0445 - val_uar: 0.4139 - val_bacc: 0.4139 - val_f1: 0.5737 - val_mcc: 0.2508
Epoch 2/20
43/43 [==============================] - 2s 56ms/step - loss: 2.3541 - val_loss: 1.0251 - val_uar: 0.4475 - val_bacc: 0.4475 - val_f1: 0.5540 - val_mcc: 0.2441
Epoch 3/20
43/43 [==============================] - 2s 57ms/step - loss: 2.1370 - val_loss: 1.1285 - val_uar: 0.4525 - val_bacc: 0.4525 - val_f1: 0.4922 - val_mcc: 0.2041
Epoch 4/20
43/43 [==============================] - 3s 60ms/step - loss: 2.0137 - val_loss: 1.0289 - val_uar: 0.4784 - val_bacc: 0.4784 - val_f1: 0.5448 - val_mcc: 0.2532
Epoch 5/20
43/43 [==============================] - 2s 55ms/step - loss: 1.8519 - val_loss: 1.0533 - val_uar: 0.4685 - val_bacc: 0.4685 - val_f1: 0.5251 - val_mcc: 0.2274
Epoch 6/20
43/43 [==============================] - 2s 55ms/step - loss: 1.8670 - val_loss: 1.0450 - val_uar: 0.4729 - val_bacc: 0.4729 - val_f1:

In [63]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

In [64]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name, from_int=True)

Metric_name:  mobilenet_7.h5_DAiSEE_4_std_single_attention_balanced Accuracy:  0.5389948006932409 MSE:  0.07840184864240322 UAR:  0.4795917764942945 Recall:  N/A Precision:  N/A F1:  0.38897043657279845


(0.5389948006932409,
 0.07840184864240322,
 0.4795917764942945,
 'N/A',
 'N/A',
 0.38897043657279845)

In [65]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [66]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

In [67]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name, from_int=True)

Metric_name:  mobilenet_7.h5_DAiSEE_4_std_single_attention_balanced_best Accuracy:  0.5326400924321202 MSE:  0.08611894858463315 UAR:  0.5286038697535108 Recall:  N/A Precision:  N/A F1:  0.3814932102066016


(0.5326400924321202,
 0.08611894858463315,
 0.5286038697535108,
 'N/A',
 'N/A',
 0.3814932102066016)

##### Self attention

In [75]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_balanced


In [76]:
CONCATENATE_STAT = True

In [77]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5474, 128, 2560) (5474,)
(1731, 128, 2560) (1731,)


In [78]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention_1 (Attention)     (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [79]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 6s 125ms/step - loss: 0.0812 - mae: 0.2012 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 2/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 3/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 4/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 5/20
43/43 [==============================] - 4s 88ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 6/20
43/43 [==============================] - 4s 88ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 7/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 8/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0798 - 

In [80]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [81]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_balanced Accuracy:  0.4662045060658579 MSE:  0.08048682842287694 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.15898345153664303


(0.4662045060658579,
 0.08048682842287694,
 0.25,
 'N/A',
 'N/A',
 0.15898345153664303)

In [82]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [83]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [84]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_balanced_best Accuracy:  0.4662045060658579 MSE:  0.08048682842287694 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.15898345153664303


(0.4662045060658579,
 0.08048682842287694,
 0.25,
 'N/A',
 'N/A',
 0.15898345153664303)

### Max

In [68]:
stat_func = np.max
stat_name = "max"

#### Traditional split

In [69]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
val_pickle = FEATURES_DIR + 'val_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)
filename2features_val = load_features(val_pickle)

In [70]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))
print("Val: ", len(filename2features_val))

Train:  5482
Test:  1723
Val:  1720


In [71]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))
print("val:")
print_classes(get_labels(filename2features_val))

train:
0 34/5482: 0.6202116016052536%
1 214/5482: 3.903684786574243%
2 2649/5482: 48.32178037212696%
3 2585/5482: 47.15432323969354%
test:
0 4/1723: 0.2321532211259431%
1 81/1723: 4.7011027278003485%
2 861/1723: 49.97098084735926%
3 777/1723: 45.095763203714455%
val:
0 23/1720: 1.3372093023255813%
1 160/1720: 9.30232558139535%
2 912/1720: 53.02325581395349%
3 625/1720: 36.33720930232558%


##### Single attention

In [72]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

mobilenet_7.h5_DAiSEE_4_max_single_attention_traditional


In [73]:
CONCATENATE_STAT = True

In [74]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 2048) (5482,)
(1723, 128, 2048) (1723,)
(1720, 128, 2048) (1720,)


In [75]:
(unique, counts) = np.unique(y_subsample_train, return_counts=True)
num_classes=len(unique)
cw=1/counts
cw/=cw.min()
class_weights = {i:cwi for i,cwi in zip(unique,cw)}
print(counts, class_weights, num_classes, unique)

[  34  214 2649 2585] {0: 77.91176470588235, 1: 12.378504672897195, 2: 1.0, 3: 1.0247582205029013} 4 [0 1 2 3]


In [76]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)
save_best_model = SaveBestModelByUAR(X_subsample_val, y_subsample_val)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2049      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [77]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val), class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 4s 81ms/step - loss: 2.8296 - val_loss: 1.1937 - val_uar: 0.3330 - val_bacc: 0.3330 - val_f1: 0.3837 - val_mcc: 0.0844
Epoch 2/20
43/43 [==============================] - 2s 44ms/step - loss: 2.2753 - val_loss: 1.0956 - val_uar: 0.3270 - val_bacc: 0.3270 - val_f1: 0.4808 - val_mcc: 0.1370
Epoch 3/20
43/43 [==============================] - 2s 44ms/step - loss: 2.0652 - val_loss: 2.2583 - val_uar: 0.3169 - val_bacc: 0.3169 - val_f1: 0.1343 - val_mcc: 0.0522
Epoch 4/20
43/43 [==============================] - 2s 44ms/step - loss: 2.0133 - val_loss: 1.2910 - val_uar: 0.3272 - val_bacc: 0.3272 - val_f1: 0.3959 - val_mcc: 0.0909
Epoch 5/20
43/43 [==============================] - 2s 44ms/step - loss: 1.8269 - val_loss: 1.2060 - val_uar: 0.3585 - val_bacc: 0.3585 - val_f1: 0.3628 - val_mcc: 0.1593
Epoch 6/20
43/43 [==============================] - 2s 44ms/step - loss: 1.8723 - val_loss: 1.0660 - val_uar: 0.3520 - val_bacc: 0.3520 - val_f1:

In [78]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

In [79]:
print(pred)
print(test_labels)

[1 1 3 ... 1 1 3]
[2 1 3 ... 1 1 2]


In [80]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name, from_int=True)

Metric_name:  mobilenet_7.h5_DAiSEE_4_max_single_attention_traditional Accuracy:  0.4016250725478816 MSE:  0.1195284968078932 UAR:  0.3219956309387204 Recall:  N/A Precision:  N/A F1:  0.2536220423748776


(0.4016250725478816,
 0.1195284968078932,
 0.3219956309387204,
 'N/A',
 'N/A',
 0.2536220423748776)

In [81]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [82]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

In [83]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name, from_int=True)

Metric_name:  mobilenet_7.h5_DAiSEE_4_max_single_attention_traditional_best Accuracy:  0.4904236796285548 MSE:  0.1248535693557748 UAR:  0.360146078133883 Recall:  N/A Precision:  N/A F1:  0.30801328315798226


(0.4904236796285548,
 0.1248535693557748,
 0.360146078133883,
 'N/A',
 'N/A',
 0.30801328315798226)

##### Self attention

In [31]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_traditional


In [32]:
CONCATENATE_STAT = True
#IMAGE_SET_SIZE=32

In [33]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 2560) (5482,)
(1723, 128, 2560) (1723,)
(1720, 128, 2560) (1720,)


In [34]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

2024-08-07 21:09:58.494035: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38431 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:11:00.0, compute capability: 8.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention (Attention)       (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']           

In [35]:
if USE_GENERATORS:
    modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                     epochs=20, verbose=1, callbacks=[save_best_model],
                    validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
else:
    modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
             validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
best_model_weights = save_best_model.best_model_weights
print(save_best_model.best)

Epoch 1/20


2024-08-07 21:10:14.656351: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f86f8a32620 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-08-07 21:10:14.656520: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2024-08-07 21:10:14.660809: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-07 21:10:14.697393: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-08-07 21:10:14.749217: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


43/43 [==============================] - 8s 149ms/step - loss: 0.0813 - mae: 0.2003 - val_loss: 0.0812 - val_mae: 0.2037
Epoch 2/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.0812 - val_mae: 0.2037
Epoch 3/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.0812 - val_mae: 0.2037
Epoch 4/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.0812 - val_mae: 0.2037
Epoch 5/20
37/43 [========================>.....] - ETA: 0s - loss: 0.0797 - mae: 0.1962

KeyboardInterrupt: 

In [45]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 6s 120ms/step - loss: 0.0812 - mae: 0.2005 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 2/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 3/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 4/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 5/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 6/20
43/43 [==============================] - 4s 88ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 7/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 8/20
43/43 [==============================] - 4s 85ms/step - loss: 0.0796 - 

In [46]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [47]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_traditional Accuracy:  0.4509576320371445 MSE:  0.08119123621590248 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.1554


(0.4509576320371445, 0.08119123621590248, 0.25, 'N/A', 'N/A', 0.1554)

In [48]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [49]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [50]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_traditional_best Accuracy:  0.4509576320371445 MSE:  0.08119123621590248 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.1554


(0.4509576320371445, 0.08119123621590248, 0.25, 'N/A', 'N/A', 0.1554)

#### Balanced split

In [84]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [85]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  5482
Test:  1723


In [86]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))

train:
0 34/5482: 0.6202116016052536%
1 214/5482: 3.903684786574243%
2 2649/5482: 48.32178037212696%
3 2585/5482: 47.15432323969354%
test:
0 4/1723: 0.2321532211259431%
1 81/1723: 4.7011027278003485%
2 861/1723: 49.97098084735926%
3 777/1723: 45.095763203714455%


In [87]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.76)

(5482,) (5482,)
(1723,) (1723,)
(7205,) (7205,)
[0 1 2 3] [  38  295 3510 3362]
7205
5474
1731


In [88]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))

train:
0 28/5474: 0.5115089514066496%
1 224/5474: 4.092071611253197%
2 2667/5474: 48.72122762148338%
3 2555/5474: 46.67519181585678%
test:
0 10/1731: 0.5777007510109763%
1 71/1731: 4.101675332177932%
2 843/1731: 48.7001733102253%
3 807/1731: 46.62045060658579%


##### Single attention

In [89]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

mobilenet_7.h5_DAiSEE_4_max_single_attention_balanced


In [90]:
CONCATENATE_STAT = True

In [91]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5474, 128, 2048) (5474,)
(1731, 128, 2048) (1731,)


In [92]:
(unique, counts) = np.unique(y_subsample_train, return_counts=True)
num_classes=len(unique)
cw=1/counts
cw/=cw.min()
class_weights = {i:cwi for i,cwi in zip(unique,cw)}
print(counts, class_weights, num_classes, unique)

[  28  224 2667 2555] {0: 95.24999999999999, 1: 11.906249999999998, 2: 1.0, 3: 1.0438356164383562} 4 [0 1 2 3]


In [93]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)
save_best_model = SaveBestModelByUAR(X_subsample_test, y_subsample_test)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2049      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [94]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test), class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 5s 87ms/step - loss: 3.3253 - val_loss: 1.4163 - val_uar: 0.3347 - val_bacc: 0.3347 - val_f1: 0.2871 - val_mcc: 0.1341
Epoch 2/20
43/43 [==============================] - 2s 58ms/step - loss: 2.3496 - val_loss: 0.9001 - val_uar: 0.4269 - val_bacc: 0.4269 - val_f1: 0.5904 - val_mcc: 0.2522
Epoch 3/20
43/43 [==============================] - 2s 57ms/step - loss: 2.1075 - val_loss: 1.1347 - val_uar: 0.3953 - val_bacc: 0.3953 - val_f1: 0.4702 - val_mcc: 0.1778
Epoch 4/20
43/43 [==============================] - 2s 56ms/step - loss: 1.9255 - val_loss: 1.1735 - val_uar: 0.4470 - val_bacc: 0.4470 - val_f1: 0.4327 - val_mcc: 0.1942
Epoch 5/20
43/43 [==============================] - 2s 58ms/step - loss: 1.8859 - val_loss: 1.0467 - val_uar: 0.4734 - val_bacc: 0.4734 - val_f1: 0.5009 - val_mcc: 0.2295
Epoch 6/20
43/43 [==============================] - 2s 56ms/step - loss: 1.9275 - val_loss: 0.9346 - val_uar: 0.4531 - val_bacc: 0.4531 - val_f1:

In [95]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

In [96]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name, from_int=True)

Metric_name:  mobilenet_7.h5_DAiSEE_4_max_single_attention_balanced Accuracy:  0.5401502021952629 MSE:  0.09019202772963603 UAR:  0.45626352562539063 Recall:  N/A Precision:  N/A F1:  0.36080485988128486


(0.5401502021952629,
 0.09019202772963603,
 0.45626352562539063,
 'N/A',
 'N/A',
 0.36080485988128486)

In [97]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [98]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

In [99]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name, from_int=True)

Metric_name:  mobilenet_7.h5_DAiSEE_4_max_single_attention_balanced_best Accuracy:  0.5014442518775274 MSE:  0.1020170999422299 UAR:  0.515562842408759 Recall:  N/A Precision:  N/A F1:  0.35582513498442814


(0.5014442518775274,
 0.1020170999422299,
 0.515562842408759,
 'N/A',
 'N/A',
 0.35582513498442814)

##### Self attention

In [75]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_balanced


In [76]:
CONCATENATE_STAT = True

In [77]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5474, 128, 2560) (5474,)
(1731, 128, 2560) (1731,)


In [78]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention_1 (Attention)     (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [79]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 6s 125ms/step - loss: 0.0812 - mae: 0.2012 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 2/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 3/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 4/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 5/20
43/43 [==============================] - 4s 88ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 6/20
43/43 [==============================] - 4s 88ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 7/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 8/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0798 - 

In [80]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [81]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_balanced Accuracy:  0.4662045060658579 MSE:  0.08048682842287694 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.15898345153664303


(0.4662045060658579,
 0.08048682842287694,
 0.25,
 'N/A',
 'N/A',
 0.15898345153664303)

In [82]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [83]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [84]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_balanced_best Accuracy:  0.4662045060658579 MSE:  0.08048682842287694 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.15898345153664303


(0.4662045060658579,
 0.08048682842287694,
 0.25,
 'N/A',
 'N/A',
 0.15898345153664303)

### STAT

In [100]:
stat_func = compute_descriptor
stat_name = "STAT"

#### Traditional split

In [101]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
val_pickle = FEATURES_DIR + 'val_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)
filename2features_val = load_features(val_pickle)

In [102]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))
print("Val: ", len(filename2features_val))

Train:  5482
Test:  1723
Val:  1720


In [103]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))
print("val:")
print_classes(get_labels(filename2features_val))

train:
0 34/5482: 0.6202116016052536%
1 214/5482: 3.903684786574243%
2 2649/5482: 48.32178037212696%
3 2585/5482: 47.15432323969354%
test:
0 4/1723: 0.2321532211259431%
1 81/1723: 4.7011027278003485%
2 861/1723: 49.97098084735926%
3 777/1723: 45.095763203714455%
val:
0 23/1720: 1.3372093023255813%
1 160/1720: 9.30232558139535%
2 912/1720: 53.02325581395349%
3 625/1720: 36.33720930232558%


##### Single attention

In [104]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

mobilenet_7.h5_DAiSEE_4_STAT_single_attention_traditional


In [105]:
CONCATENATE_STAT = True

In [106]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 3072) (5482,)
(1723, 128, 3072) (1723,)
(1720, 128, 3072) (1720,)


In [107]:
(unique, counts) = np.unique(y_subsample_train, return_counts=True)
num_classes=len(unique)
cw=1/counts
cw/=cw.min()
class_weights = {i:cwi for i,cwi in zip(unique,cw)}
print(counts, class_weights, num_classes, unique)

[  34  214 2649 2585] {0: 77.91176470588235, 1: 12.378504672897195, 2: 1.0, 3: 1.0247582205029013} 4 [0 1 2 3]


In [108]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)
save_best_model = SaveBestModelByUAR(X_subsample_val, y_subsample_val)

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3072)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              3073      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [109]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=11, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val), class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/11
43/43 [==============================] - 6s 126ms/step - loss: 2.7224 - val_loss: 1.2446 - val_uar: 0.3491 - val_bacc: 0.3491 - val_f1: 0.4273 - val_mcc: 0.1336
Epoch 2/11
43/43 [==============================] - 3s 79ms/step - loss: 2.1700 - val_loss: 1.3028 - val_uar: 0.3894 - val_bacc: 0.3894 - val_f1: 0.3593 - val_mcc: 0.1278
Epoch 3/11
43/43 [==============================] - 4s 83ms/step - loss: 2.0271 - val_loss: 1.5549 - val_uar: 0.3501 - val_bacc: 0.3501 - val_f1: 0.2576 - val_mcc: 0.0978
Epoch 4/11
43/43 [==============================] - 4s 89ms/step - loss: 1.9183 - val_loss: 1.4204 - val_uar: 0.3368 - val_bacc: 0.3368 - val_f1: 0.3169 - val_mcc: 0.1360
Epoch 5/11
43/43 [==============================] - 4s 86ms/step - loss: 1.8289 - val_loss: 1.1447 - val_uar: 0.3873 - val_bacc: 0.3873 - val_f1: 0.4488 - val_mcc: 0.1753
Epoch 6/11
43/43 [==============================] - 3s 81ms/step - loss: 1.7841 - val_loss: 1.3198 - val_uar: 0.3645 - val_bacc: 0.3645 - val_f1

In [110]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

In [111]:
print(pred)
print(test_labels)

[1 1 1 ... 1 1 2]
[2 1 3 ... 1 1 2]


In [112]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name, from_int=True)

Metric_name:  mobilenet_7.h5_DAiSEE_4_STAT_single_attention_traditional Accuracy:  0.3639001741149158 MSE:  0.1429021474172954 UAR:  0.35949560187365065 Recall:  N/A Precision:  N/A F1:  0.2454942384467854


(0.3639001741149158,
 0.1429021474172954,
 0.35949560187365065,
 'N/A',
 'N/A',
 0.2454942384467854)

In [113]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [114]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

In [115]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name, from_int=True)

Metric_name:  mobilenet_7.h5_DAiSEE_4_STAT_single_attention_traditional_best Accuracy:  0.3360417875798027 MSE:  0.1954635519442832 UAR:  0.3190707403105777 Recall:  N/A Precision:  N/A F1:  0.23396095006664563


(0.3360417875798027,
 0.1954635519442832,
 0.3190707403105777,
 'N/A',
 'N/A',
 0.23396095006664563)

##### Self attention

In [31]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_traditional


In [32]:
CONCATENATE_STAT = True
#IMAGE_SET_SIZE=32

In [33]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 2560) (5482,)
(1723, 128, 2560) (1723,)
(1720, 128, 2560) (1720,)


In [34]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

2024-08-07 21:09:58.494035: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38431 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:11:00.0, compute capability: 8.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention (Attention)       (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']           

In [35]:
if USE_GENERATORS:
    modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                     epochs=20, verbose=1, callbacks=[save_best_model],
                    validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
else:
    modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
             validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
best_model_weights = save_best_model.best_model_weights
print(save_best_model.best)

Epoch 1/20


2024-08-07 21:10:14.656351: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f86f8a32620 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-08-07 21:10:14.656520: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2024-08-07 21:10:14.660809: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-07 21:10:14.697393: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-08-07 21:10:14.749217: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


43/43 [==============================] - 8s 149ms/step - loss: 0.0813 - mae: 0.2003 - val_loss: 0.0812 - val_mae: 0.2037
Epoch 2/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.0812 - val_mae: 0.2037
Epoch 3/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.0812 - val_mae: 0.2037
Epoch 4/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.0812 - val_mae: 0.2037
Epoch 5/20
37/43 [========================>.....] - ETA: 0s - loss: 0.0797 - mae: 0.1962

KeyboardInterrupt: 

In [45]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 6s 120ms/step - loss: 0.0812 - mae: 0.2005 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 2/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 3/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 4/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 5/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 6/20
43/43 [==============================] - 4s 88ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 7/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 8/20
43/43 [==============================] - 4s 85ms/step - loss: 0.0796 - 

In [46]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [47]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_traditional Accuracy:  0.4509576320371445 MSE:  0.08119123621590248 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.1554


(0.4509576320371445, 0.08119123621590248, 0.25, 'N/A', 'N/A', 0.1554)

In [48]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [49]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [50]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_traditional_best Accuracy:  0.4509576320371445 MSE:  0.08119123621590248 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.1554


(0.4509576320371445, 0.08119123621590248, 0.25, 'N/A', 'N/A', 0.1554)

#### Balanced split

In [116]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [117]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  5482
Test:  1723


In [118]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))

train:
0 34/5482: 0.6202116016052536%
1 214/5482: 3.903684786574243%
2 2649/5482: 48.32178037212696%
3 2585/5482: 47.15432323969354%
test:
0 4/1723: 0.2321532211259431%
1 81/1723: 4.7011027278003485%
2 861/1723: 49.97098084735926%
3 777/1723: 45.095763203714455%


In [119]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.76)

(5482, 2048) (5482,)
(1723, 2048) (1723,)
(7205, 2048) (7205,)
[0 1 2 3] [  38  295 3510 3362]
7205
5474
1731


In [120]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))

train:
0 28/5474: 0.5115089514066496%
1 224/5474: 4.092071611253197%
2 2667/5474: 48.72122762148338%
3 2555/5474: 46.67519181585678%
test:
0 10/1731: 0.5777007510109763%
1 71/1731: 4.101675332177932%
2 843/1731: 48.7001733102253%
3 807/1731: 46.62045060658579%


##### Single attention

In [121]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

mobilenet_7.h5_DAiSEE_4_STAT_single_attention_balanced


In [122]:
CONCATENATE_STAT = True

In [123]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5474, 128, 3072) (5474,)
(1731, 128, 3072) (1731,)


In [124]:
(unique, counts) = np.unique(y_subsample_train, return_counts=True)
num_classes=len(unique)
cw=1/counts
cw/=cw.min()
class_weights = {i:cwi for i,cwi in zip(unique,cw)}
print(counts, class_weights, num_classes, unique)

[  28  224 2667 2555] {0: 95.24999999999999, 1: 11.906249999999998, 2: 1.0, 3: 1.0438356164383562} 4 [0 1 2 3]


In [125]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)
save_best_model = SaveBestModelByUAR(X_subsample_test, y_subsample_test)

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3072)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              3073      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [126]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=11, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test), class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/11
43/43 [==============================] - 6s 118ms/step - loss: 2.7714 - val_loss: 0.9823 - val_uar: 0.4267 - val_bacc: 0.4267 - val_f1: 0.5713 - val_mcc: 0.2471
Epoch 2/11
43/43 [==============================] - 3s 81ms/step - loss: 2.1089 - val_loss: 0.9668 - val_uar: 0.4284 - val_bacc: 0.4284 - val_f1: 0.5298 - val_mcc: 0.2038
Epoch 3/11
43/43 [==============================] - 3s 81ms/step - loss: 1.9601 - val_loss: 1.1450 - val_uar: 0.4215 - val_bacc: 0.4215 - val_f1: 0.4743 - val_mcc: 0.2053
Epoch 4/11
43/43 [==============================] - 4s 85ms/step - loss: 1.9238 - val_loss: 1.1922 - val_uar: 0.4843 - val_bacc: 0.4843 - val_f1: 0.4743 - val_mcc: 0.2148
Epoch 5/11
43/43 [==============================] - 3s 82ms/step - loss: 1.7696 - val_loss: 0.9927 - val_uar: 0.4488 - val_bacc: 0.4488 - val_f1: 0.5430 - val_mcc: 0.2456
Epoch 6/11
43/43 [==============================] - 3s 81ms/step - loss: 1.9063 - val_loss: 0.9947 - val_uar: 0.4608 - val_bacc: 0.4608 - val_f1

In [127]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

In [128]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name, from_int=True)

Metric_name:  mobilenet_7.h5_DAiSEE_4_STAT_single_attention_balanced Accuracy:  0.44771808203350666 MSE:  0.1268165222414789 UAR:  0.48116321325537525 Recall:  N/A Precision:  N/A F1:  0.31585300025484253


(0.44771808203350666,
 0.1268165222414789,
 0.48116321325537525,
 'N/A',
 'N/A',
 0.31585300025484253)

In [129]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [130]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

In [131]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name, from_int=True)

Metric_name:  mobilenet_7.h5_DAiSEE_4_STAT_single_attention_balanced_best Accuracy:  0.5713460427498556 MSE:  0.08364523396880415 UAR:  0.5045200968519092 Recall:  N/A Precision:  N/A F1:  0.38182567679984347


(0.5713460427498556,
 0.08364523396880415,
 0.5045200968519092,
 'N/A',
 'N/A',
 0.38182567679984347)

##### Self attention

In [75]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_balanced


In [76]:
CONCATENATE_STAT = True

In [77]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5474, 128, 2560) (5474,)
(1731, 128, 2560) (1731,)


In [78]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention_1 (Attention)     (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [79]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 6s 125ms/step - loss: 0.0812 - mae: 0.2012 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 2/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 3/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 4/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 5/20
43/43 [==============================] - 4s 88ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 6/20
43/43 [==============================] - 4s 88ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 7/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 8/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0798 - 

In [80]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [81]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_balanced Accuracy:  0.4662045060658579 MSE:  0.08048682842287694 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.15898345153664303


(0.4662045060658579,
 0.08048682842287694,
 0.25,
 'N/A',
 'N/A',
 0.15898345153664303)

In [82]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [83]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [84]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_balanced_best Accuracy:  0.4662045060658579 MSE:  0.08048682842287694 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.15898345153664303


(0.4662045060658579,
 0.08048682842287694,
 0.25,
 'N/A',
 'N/A',
 0.15898345153664303)

In [75]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_balanced


In [76]:
CONCATENATE_STAT = True

In [77]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5474, 128, 2560) (5474,)
(1731, 128, 2560) (1731,)


In [78]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention_1 (Attention)     (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [79]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 6s 125ms/step - loss: 0.0812 - mae: 0.2012 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 2/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 3/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 4/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 5/20
43/43 [==============================] - 4s 88ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 6/20
43/43 [==============================] - 4s 88ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 7/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 8/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0798 - 

In [80]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [81]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_balanced Accuracy:  0.4662045060658579 MSE:  0.08048682842287694 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.15898345153664303


(0.4662045060658579,
 0.08048682842287694,
 0.25,
 'N/A',
 'N/A',
 0.15898345153664303)

In [82]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [83]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [84]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_balanced_best Accuracy:  0.4662045060658579 MSE:  0.08048682842287694 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.15898345153664303


(0.4662045060658579,
 0.08048682842287694,
 0.25,
 'N/A',
 'N/A',
 0.15898345153664303)

# DAiSEE dataset (2 classes)

In [24]:
FEATURES_DIR = DATA_DIR + 'features_DAiSEE/'
WEIGHTS_DIR = DATA_DIR + 'weights_DAiSEE/'
WEIGHTS_DIR = DATA_DIR + 'weights_DAiSEE_classifier_sigmoid/'
TABLE_NAME = '02_DAiSEE_2_classes.xlsx'
TABLE_NAME = '02_DAiSEE_2_classes_classifier_sigmoid.xlsx'

## enet_b0_8_best_afew

In [25]:
base_model_key = 'enet_b0_8_best_afew.pt'

In [26]:
ENGAGE_WILD_DATASET, BIN_CLASSIFICATION, N_CLASSES, DATASET_NAME, BASE_DATASET_DIR, ext = define_dataset(False, True)
video2label, labels_list = get_video2label()
print(labels_list)
print(video2label)

['engaged', 'distracted']
{'1100011002': 0, '1100011003': 0, '1100011004': 0, '1100011005': 0, '1100011006': 0, '1100011007': 0, '1100011008': 0, '1100011009': 0, '1100011010': 0, '1100011011': 0, '1100011012': 0, '1100011013': 0, '1100011014': 0, '1100011015': 0, '1100011016': 0, '1100011017': 0, '1100011018': 0, '1100011019': 0, '1100011020': 0, '1100011021': 0, '1100011022': 0, '1100011023': 0, '1100011025': 0, '1100011026': 0, '1100011027': 0, '1100011028': 0, '1100011029': 0, '1100011031': 0, '1100011032': 0, '1100011034': 0, '1100011035': 0, '1100011037': 0, '1100011038': 0, '1100011040': 0, '1100011046': 0, '1100011047': 0, '1100011048': 0, '1100011049': 0, '1100011050': 0, '1100011051': 0, '1100011052': 0, '1100011053': 0, '1100011054': 0, '1100011055': 0, '1100011056': 0, '1100011057': 0, '1100011058': 0, '1100011059': 0, '1100011060': 0, '1100011062': 0, '1100011063': 0, '1100011064': 0, '1100011066': 0, '1100011067': 0, '1100011068': 0, '1100011069': 0, '1100011070': 0, '110

### Std

In [27]:
stat_func = np.std
stat_name = "std"

#### Traditional split

In [28]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
val_pickle = FEATURES_DIR + 'val_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)
filename2features_val = load_features(val_pickle)

In [29]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))
print("Val: ", len(filename2features_val))

Train:  5482
Test:  1723
Val:  1720


In [30]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))
print("val:")
print_classes(get_labels(filename2features_val))

train:
0 5234/5482: 95.4761036118205%
1 248/5482: 4.523896388179496%
test:
0 1638/1723: 95.06674405107371%
1 85/1723: 4.933255948926291%
val:
0 1537/1720: 89.36046511627907%
1 183/1720: 10.63953488372093%


##### Single attention

In [31]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_2_std_single_attention_traditional


In [32]:
CONCATENATE_STAT = True

In [33]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 2560) (5482,)
(1723, 128, 2560) (1723,)
(1720, 128, 2560) (1720,)


In [34]:
(unique, counts) = np.unique(y_subsample_train, return_counts=True)
num_classes=len(unique)
cw=1/counts
cw/=cw.min()
class_weights = {i:cwi for i,cwi in zip(unique,cw)}
print(counts, class_weights, num_classes, unique)

[5234  248] {0: 1.0, 1: 21.10483870967742} 2 [0 1]


In [35]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)
#save_best_model = SaveBestModelByUAR(X_subsample_val, y_subsample_val)

2024-08-11 21:14:39.863028: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38431 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:11:00.0, compute capability: 8.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2561      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                              

In [36]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val), class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20


2024-08-11 21:14:49.383482: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f8884061ed0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-08-11 21:14:49.383666: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2024-08-11 21:14:49.387767: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-11 21:14:49.438957: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-08-11 21:14:49.494201: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


43/43 [==============================] - 7s 88ms/step - loss: 1.2773 - acc: 0.6638 - auc: 0.6550 - binary_accuracy: 0.6638 - recall: 0.5806 - precision: 0.0765 - val_loss: 0.7971 - val_acc: 0.5459 - val_auc: 0.6771 - val_binary_accuracy: 0.5459 - val_recall: 0.7814 - val_precision: 0.1618
Epoch 2/20
43/43 [==============================] - 1s 31ms/step - loss: 1.0950 - acc: 0.7335 - auc: 0.7709 - binary_accuracy: 0.7335 - recall: 0.6895 - precision: 0.1100 - val_loss: 0.8405 - val_acc: 0.5233 - val_auc: 0.6811 - val_binary_accuracy: 0.5233 - val_recall: 0.8197 - val_precision: 0.1601
Epoch 3/20
43/43 [==============================] - 1s 31ms/step - loss: 1.0397 - acc: 0.7202 - auc: 0.8012 - binary_accuracy: 0.7202 - recall: 0.7218 - precision: 0.1089 - val_loss: 0.6223 - val_acc: 0.6535 - val_auc: 0.6792 - val_binary_accuracy: 0.6535 - val_recall: 0.6011 - val_precision: 0.1738
Epoch 4/20
43/43 [==============================] - 1s 31ms/step - loss: 1.0171 - acc: 0.7435 - auc: 0.8079 

In [37]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_2765461/1443316788.py:29: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [38]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_single_attention_traditional Accuracy:  0.4712710388856645 MSE:  0.5287289611143354 UAR:  0.6326833297421532 Recall:  0.8117647058823529 Precision:  0.07157676348547717 F1:  0.1315538608198284
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_single_attention_traditional Accuracy:  0.6482878699941962 MSE:  0.3517121300058038 UAR:  0.6588594412123824 Recall:  0.6705882352941176 Precision:  0.08976377952755905 F1:  0.15833333333333333
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_single_attention_traditional Accuracy:  0.7382472431804992 MSE:  0.2617527568195009 UAR:  0.6727106227106228 Recall:  0.6 Precision:  0.10897435897435898 F1:  0.18444846292947562
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_single_attention_traditional Accuracy:  0.803830528148578 MSE:  0.19616947185142194 UAR:  0.6737412913883503 Recall:  0.5294117647058824 Precision:  0.13119533527696792 F1:  0.2102803738317757
Metric_name:  enet_b0_8_best_afew.pt_DAiS

In [39]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [40]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_2765461/1443316788.py:29: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [41]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_single_attention_traditional_best Accuracy:  0.4097504352872896 MSE:  0.5902495647127104 UAR:  0.6226352079293256 Recall:  0.8588235294117647 Precision:  0.06771799628942486 F1:  0.12553740326741186
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_single_attention_traditional_best Accuracy:  0.6111433546140452 MSE:  0.3888566453859547 UAR:  0.6839402427637722 Recall:  0.7647058823529411 Precision:  0.09090909090909091 F1:  0.1625
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_single_attention_traditional_best Accuracy:  0.7777132907719094 MSE:  0.22228670922809055 UAR:  0.676736335559865 Recall:  0.5647058823529412 Precision:  0.1218274111675127 F1:  0.20041753653444677
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_single_attention_traditional_best Accuracy:  0.865351131746953 MSE:  0.134648868253047 UAR:  0.6559039000215471 Recall:  0.4235294117647059 Precision:  0.1643835616438356 F1:  0.23684210526315785
Metric_name:  enet_b0

##### Self attention

In [ ]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

In [199]:
CONCATENATE_STAT = True

In [200]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 2560) (5482,)
(1723, 128, 2560) (1723,)
(1720, 128, 2560) (1720,)


In [201]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_13"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention_6 (Attention)     (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']        

In [202]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 7s 127ms/step - loss: 0.3520 - acc: 0.9435 - auc: 0.5372 - binary_accuracy: 0.9435 - recall_1: 0.0121 - precision_1: 0.0441 - val_loss: 0.3655 - val_acc: 0.8936 - val_auc: 0.6054 - val_binary_accuracy: 0.8936 - val_recall_1: 0.0000e+00 - val_precision_1: 0.0000e+00
Epoch 2/20
43/43 [==============================] - 4s 89ms/step - loss: 0.1676 - acc: 0.9544 - auc: 0.7332 - binary_accuracy: 0.9544 - recall_1: 0.0081 - precision_1: 0.3333 - val_loss: 0.4156 - val_acc: 0.8936 - val_auc: 0.6291 - val_binary_accuracy: 0.8936 - val_recall_1: 0.0000e+00 - val_precision_1: 0.0000e+00
Epoch 3/20
43/43 [==============================] - 4s 89ms/step - loss: 0.1589 - acc: 0.9533 - auc: 0.7800 - binary_accuracy: 0.9533 - recall_1: 0.0081 - precision_1: 0.1667 - val_loss: 0.3692 - val_acc: 0.8907 - val_auc: 0.6405 - val_binary_accuracy: 0.8907 - val_recall_1: 0.0328 - val_precision_1: 0.3529
Epoch 4/20
43/43 [==============================] - 4s 9

In [203]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [204]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional Accuracy:  0.8415554265815438 MSE:  0.1584445734184562 UAR:  0.6043489190548015 Recall:  0.3411764705882353 Precision:  0.11788617886178862 F1:  0.17522658610271902
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional Accuracy:  0.930354033662217 MSE:  0.06964596633778293 UAR:  0.5450872656755009 Recall:  0.11764705882352941 Precision:  0.18181818181818182 F1:  0.14285714285714285
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional Accuracy:  0.9437028438769588 MSE:  0.05629715612304121 UAR:  0.5409538174244057 Recall:  0.09411764705882353 Precision:  0.2857142857142857 F1:  0.1415929203539823
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional Accuracy:  0.947185142193848 MSE:  0.05281485780615206 UAR:  0.526054011348129 Recall:  0.058823529411764705 Precision:  0.3125 F1:  0.09900990099009901
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE

In [205]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [206]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [207]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional_best Accuracy:  0.8340104468949506 MSE:  0.16598955310504934 UAR:  0.6282661782661783 Recall:  0.4 Precision:  0.12639405204460966 F1:  0.192090395480226
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional_best Accuracy:  0.9297736506094022 MSE:  0.0702263493905978 UAR:  0.5949759390935861 Recall:  0.2235294117647059 Precision:  0.25675675675675674 F1:  0.2389937106918239
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional_best Accuracy:  0.9477655252466628 MSE:  0.0522344747533372 UAR:  0.5653989801048624 Recall:  0.1411764705882353 Precision:  0.41379310344827586 F1:  0.21052631578947367
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional_best Accuracy:  0.9524085896691816 MSE:  0.04759141033081834 UAR:  0.5566867772750126 Recall:  0.11764705882352941 Precision:  0.5882352941176471 F1:  0.19607843137254902
Metric_name:  enet_b0_8_best

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### Balanced split

In [42]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [43]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  5482
Test:  1723


In [44]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 5234/5482: 95.4761036118205%
1 248/5482: 4.523896388179496%
val:
0 1638/1723: 95.06674405107371%
1 85/1723: 4.933255948926291%


In [45]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.76)

(5482,) (5482,)
(1723,) (1723,)
(7205,) (7205,)
[0 1] [6872  333]
7205
5475
1730


In [46]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 5222/5475: 95.37899543378995%
1 253/5475: 4.621004566210046%
val:
0 1650/1730: 95.3757225433526%
1 80/1730: 4.624277456647399%


##### Single attention

In [47]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_2_std_single_attention_balanced


In [48]:
CONCATENATE_STAT = True

In [49]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5475, 128, 2560) (5475,)
(1730, 128, 2560) (1730,)


In [50]:
(unique, counts) = np.unique(y_subsample_train, return_counts=True)
num_classes=len(unique)
cw=1/counts
cw/=cw.min()
class_weights = {i:cwi for i,cwi in zip(unique,cw)}
print(counts, class_weights, num_classes, unique)

[5222  253] {0: 1.0, 1: 20.640316205533598} 2 [0 1]


In [51]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)
#save_best_model = SaveBestModelByUAR(X_subsample_test, y_subsample_test)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2561      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [52]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test), class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 5s 78ms/step - loss: 1.3137 - acc: 0.6356 - auc: 0.6562 - binary_accuracy: 0.6356 - recall_1: 0.6008 - precision_1: 0.0743 - val_loss: 0.5989 - val_acc: 0.7451 - val_auc: 0.7696 - val_binary_accuracy: 0.7451 - val_recall_1: 0.6875 - val_precision_1: 0.1168
Epoch 2/20
43/43 [==============================] - 1s 30ms/step - loss: 1.0760 - acc: 0.7531 - auc: 0.7792 - binary_accuracy: 0.7531 - recall_1: 0.6719 - precision_1: 0.1181 - val_loss: 0.5128 - val_acc: 0.7780 - val_auc: 0.7834 - val_binary_accuracy: 0.7780 - val_recall_1: 0.6000 - val_precision_1: 0.1200
Epoch 3/20
43/43 [==============================] - 1s 30ms/step - loss: 1.0183 - acc: 0.7618 - auc: 0.8149 - binary_accuracy: 0.7618 - recall_1: 0.7115 - precision_1: 0.1276 - val_loss: 0.4452 - val_acc: 0.8197 - val_auc: 0.7822 - val_binary_accuracy: 0.8197 - val_recall_1: 0.5625 - val_precision_1: 0.1398
Epoch 4/20
43/43 [==============================] - 1s 32ms/step - loss: 

In [53]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_2765461/1443316788.py:29: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [54]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name, from_int=True)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_single_attention_balanced Accuracy:  0.4184971098265896 MSE:  0.5815028901734104 UAR:  0.6118939393939393 Recall:  0.825 Precision:  0.062381852551984876 F1:  0.11599297012302284
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_single_attention_balanced Accuracy:  0.5589595375722544 MSE:  0.44104046242774564 UAR:  0.6439015151515152 Recall:  0.7375 Precision:  0.07365792759051186 F1:  0.13393870601589103
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_single_attention_balanced Accuracy:  0.6485549132947976 MSE:  0.3514450867052023 UAR:  0.6730303030303031 Recall:  0.7 Precision:  0.0875 F1:  0.15555555555555553
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_single_attention_balanced Accuracy:  0.7375722543352601 MSE:  0.26242774566473986 UAR:  0.6899621212121212 Recall:  0.6375 Precision:  0.10714285714285714 F1:  0.18345323741007194
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_single_attention_balanced Accuracy:  0.8 MSE:  0

In [55]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [56]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_2765461/1443316788.py:29: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [57]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name, from_int=True)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_single_attention_balanced_best Accuracy:  0.47572254335260117 MSE:  0.5242774566473989 UAR:  0.6359469696969697 Recall:  0.8125 Precision:  0.06792058516196448 F1:  0.12536162005785922
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_single_attention_balanced_best Accuracy:  0.6566473988439306 MSE:  0.34335260115606936 UAR:  0.6713257575757576 Recall:  0.6875 Precision:  0.08814102564102565 F1:  0.15625
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_single_attention_balanced_best Accuracy:  0.7716763005780347 MSE:  0.22832369942196531 UAR:  0.7018939393939394 Recall:  0.625 Precision:  0.12048192771084337 F1:  0.20202020202020202
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_single_attention_balanced_best Accuracy:  0.8369942196531792 MSE:  0.1630057803468208 UAR:  0.7064015151515152 Recall:  0.5625 Precision:  0.1541095890410959 F1:  0.24193548387096775
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_single_attention_balanced

##### Self attention

In [223]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced


In [224]:
CONCATENATE_STAT = True

In [225]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5475, 128, 2560) (5475,)
(1730, 128, 2560) (1730,)


In [226]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_15"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention_7 (Attention)     (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']        

In [227]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 7s 125ms/step - loss: 0.3764 - acc: 0.9412 - auc: 0.5374 - binary_accuracy: 0.9412 - recall_3: 0.0277 - precision_3: 0.0843 - val_loss: 0.1875 - val_acc: 0.9526 - val_auc: 0.6787 - val_binary_accuracy: 0.9526 - val_recall_3: 0.0125 - val_precision_3: 0.2500
Epoch 2/20
43/43 [==============================] - 4s 89ms/step - loss: 0.1737 - acc: 0.9540 - auc: 0.7014 - binary_accuracy: 0.9540 - recall_3: 0.0514 - precision_3: 0.5200 - val_loss: 0.1701 - val_acc: 0.9526 - val_auc: 0.7431 - val_binary_accuracy: 0.9526 - val_recall_3: 0.0250 - val_precision_3: 0.3333
Epoch 3/20
43/43 [==============================] - 4s 88ms/step - loss: 0.1627 - acc: 0.9536 - auc: 0.7562 - binary_accuracy: 0.9536 - recall_3: 0.0435 - precision_3: 0.4783 - val_loss: 0.1690 - val_acc: 0.9526 - val_auc: 0.7421 - val_binary_accuracy: 0.9526 - val_recall_3: 0.0000e+00 - val_precision_3: 0.0000e+00
Epoch 4/20
43/43 [==============================] - 4s 88ms/step

In [228]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [229]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced Accuracy:  0.9445086705202312 MSE:  0.055491329479768786 UAR:  0.5903030303030303 Recall:  0.2 Precision:  0.3333333333333333 F1:  0.25
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced Accuracy:  0.9520231213872833 MSE:  0.04797687861271676 UAR:  0.5526136363636364 Recall:  0.1125 Precision:  0.42857142857142855 F1:  0.1782178217821782
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced Accuracy:  0.953757225433526 MSE:  0.046242774566473986 UAR:  0.5297348484848485 Recall:  0.0625 Precision:  0.5 F1:  0.1111111111111111
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced Accuracy:  0.954335260115607 MSE:  0.04566473988439306 UAR:  0.5181439393939394 Recall:  0.0375 Precision:  0.6 F1:  0.07058823529411765
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced Accuracy:  0.9531791907514451 MSE:  0.04682080924855491 UAR:  0.505

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [230]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [231]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [232]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced_best Accuracy:  0.8549132947976879 MSE:  0.14508670520231215 UAR:  0.6979545454545455 Recall:  0.525 Precision:  0.16470588235294117 F1:  0.2507462686567164
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced_best Accuracy:  0.9260115606936417 MSE:  0.07398843930635839 UAR:  0.6460227272727272 Recall:  0.3375 Precision:  0.2647058823529412 F1:  0.29670329670329665
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced_best Accuracy:  0.946242774566474 MSE:  0.05375722543352601 UAR:  0.5852651515151515 Recall:  0.1875 Precision:  0.3488372093023256 F1:  0.2439024390243903
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced_best Accuracy:  0.954335260115607 MSE:  0.04566473988439306 UAR:  0.5716666666666667 Recall:  0.15 Precision:  0.5217391304347826 F1:  0.23300970873786406
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced_best

### Max

In [58]:
stat_func = np.max
stat_name = "max"

#### Traditional split

In [59]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
val_pickle = FEATURES_DIR + 'val_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)
filename2features_val = load_features(val_pickle)

In [60]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))
print("Val: ", len(filename2features_val))

Train:  5482
Test:  1723
Val:  1720


In [61]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))
print("val:")
print_classes(get_labels(filename2features_val))

train:
0 5234/5482: 95.4761036118205%
1 248/5482: 4.523896388179496%
test:
0 1638/1723: 95.06674405107371%
1 85/1723: 4.933255948926291%
val:
0 1537/1720: 89.36046511627907%
1 183/1720: 10.63953488372093%


##### Single attention

In [62]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_2_max_single_attention_traditional


In [63]:
CONCATENATE_STAT = True

In [64]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 2560) (5482,)
(1723, 128, 2560) (1723,)
(1720, 128, 2560) (1720,)


In [65]:
(unique, counts) = np.unique(y_subsample_train, return_counts=True)
num_classes=len(unique)
cw=1/counts
cw/=cw.min()
class_weights = {i:cwi for i,cwi in zip(unique,cw)}
print(counts, class_weights, num_classes, unique)

[5234  248] {0: 1.0, 1: 21.10483870967742} 2 [0 1]


In [66]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)
#save_best_model = SaveBestModelByUAR(X_subsample_val, y_subsample_val)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2561      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [67]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val), class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 4s 66ms/step - loss: 1.3745 - acc: 0.6554 - auc: 0.6534 - binary_accuracy: 0.6554 - recall_2: 0.5887 - precision_2: 0.0755 - val_loss: 0.5669 - val_acc: 0.7105 - val_auc: 0.6817 - val_binary_accuracy: 0.7105 - val_recall_2: 0.4754 - val_precision_2: 0.1779
Epoch 2/20
43/43 [==============================] - 1s 31ms/step - loss: 1.1120 - acc: 0.7242 - auc: 0.7738 - binary_accuracy: 0.7242 - recall_2: 0.7056 - precision_2: 0.1084 - val_loss: 0.5996 - val_acc: 0.6895 - val_auc: 0.6817 - val_binary_accuracy: 0.6895 - val_recall_2: 0.5246 - val_precision_2: 0.1768
Epoch 3/20
43/43 [==============================] - 1s 31ms/step - loss: 1.0626 - acc: 0.7340 - auc: 0.7947 - binary_accuracy: 0.7340 - recall_2: 0.7016 - precision_2: 0.1117 - val_loss: 0.6415 - val_acc: 0.6657 - val_auc: 0.6711 - val_binary_accuracy: 0.6657 - val_recall_2: 0.5738 - val_precision_2: 0.1744
Epoch 4/20
43/43 [==============================] - 1s 30ms/step - loss: 

In [68]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_2765461/1443316788.py:29: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [69]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name, from_int=True)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_single_attention_traditional Accuracy:  0.5612304120719674 MSE:  0.4387695879280325 UAR:  0.6409574086044674 Recall:  0.7294117647058823 Precision:  0.0779874213836478 F1:  0.1409090909090909
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_single_attention_traditional Accuracy:  0.7301218804410912 MSE:  0.2698781195589089 UAR:  0.6684371184371185 Recall:  0.6 Precision:  0.10580912863070539 F1:  0.1798941798941799
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_single_attention_traditional Accuracy:  0.8189204875217644 MSE:  0.18107951247823564 UAR:  0.6705235940530059 Recall:  0.5058823529411764 Precision:  0.13738019169329074 F1:  0.21608040201005024
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_single_attention_traditional Accuracy:  0.8630295995356936 MSE:  0.13697040046430645 UAR:  0.6491057961646196 Recall:  0.4117647058823529 Precision:  0.1583710407239819 F1:  0.2287581699346405
Metric_name:  enet_b0_8_best_afew.pt_DAiSE

In [70]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [71]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_2765461/1443316788.py:29: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [72]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name, from_int=True)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_single_attention_traditional_best Accuracy:  0.2547881601857226 MSE:  0.7452118398142774 UAR:  0.574595992243051 Recall:  0.9294117647058824 Precision:  0.05821665438467207 F1:  0.10957004160887657
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_single_attention_traditional_best Accuracy:  0.6018572257690076 MSE:  0.3981427742309925 UAR:  0.6400165194282841 Recall:  0.6823529411764706 Precision:  0.08089260808926081 F1:  0.14463840399002495
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_single_attention_traditional_best Accuracy:  0.820661636680209 MSE:  0.17933836331979106 UAR:  0.6658622423328306 Recall:  0.49411764705882355 Precision:  0.13636363636363635 F1:  0.21374045801526717
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_single_attention_traditional_best Accuracy:  0.914683691236216 MSE:  0.0853163087637841 UAR:  0.6205020469726352 Recall:  0.29411764705882354 Precision:  0.22321428571428573 F1:  0.25380710659898476
Metr

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### Self attention

In [198]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional


In [199]:
CONCATENATE_STAT = True

In [200]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 2560) (5482,)
(1723, 128, 2560) (1723,)
(1720, 128, 2560) (1720,)


In [201]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_13"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention_6 (Attention)     (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']        

In [202]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 7s 127ms/step - loss: 0.3520 - acc: 0.9435 - auc: 0.5372 - binary_accuracy: 0.9435 - recall_1: 0.0121 - precision_1: 0.0441 - val_loss: 0.3655 - val_acc: 0.8936 - val_auc: 0.6054 - val_binary_accuracy: 0.8936 - val_recall_1: 0.0000e+00 - val_precision_1: 0.0000e+00
Epoch 2/20
43/43 [==============================] - 4s 89ms/step - loss: 0.1676 - acc: 0.9544 - auc: 0.7332 - binary_accuracy: 0.9544 - recall_1: 0.0081 - precision_1: 0.3333 - val_loss: 0.4156 - val_acc: 0.8936 - val_auc: 0.6291 - val_binary_accuracy: 0.8936 - val_recall_1: 0.0000e+00 - val_precision_1: 0.0000e+00
Epoch 3/20
43/43 [==============================] - 4s 89ms/step - loss: 0.1589 - acc: 0.9533 - auc: 0.7800 - binary_accuracy: 0.9533 - recall_1: 0.0081 - precision_1: 0.1667 - val_loss: 0.3692 - val_acc: 0.8907 - val_auc: 0.6405 - val_binary_accuracy: 0.8907 - val_recall_1: 0.0328 - val_precision_1: 0.3529
Epoch 4/20
43/43 [==============================] - 4s 9

In [203]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [204]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional Accuracy:  0.8415554265815438 MSE:  0.1584445734184562 UAR:  0.6043489190548015 Recall:  0.3411764705882353 Precision:  0.11788617886178862 F1:  0.17522658610271902
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional Accuracy:  0.930354033662217 MSE:  0.06964596633778293 UAR:  0.5450872656755009 Recall:  0.11764705882352941 Precision:  0.18181818181818182 F1:  0.14285714285714285
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional Accuracy:  0.9437028438769588 MSE:  0.05629715612304121 UAR:  0.5409538174244057 Recall:  0.09411764705882353 Precision:  0.2857142857142857 F1:  0.1415929203539823
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional Accuracy:  0.947185142193848 MSE:  0.05281485780615206 UAR:  0.526054011348129 Recall:  0.058823529411764705 Precision:  0.3125 F1:  0.09900990099009901
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE

In [205]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [206]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [207]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional_best Accuracy:  0.8340104468949506 MSE:  0.16598955310504934 UAR:  0.6282661782661783 Recall:  0.4 Precision:  0.12639405204460966 F1:  0.192090395480226
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional_best Accuracy:  0.9297736506094022 MSE:  0.0702263493905978 UAR:  0.5949759390935861 Recall:  0.2235294117647059 Precision:  0.25675675675675674 F1:  0.2389937106918239
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional_best Accuracy:  0.9477655252466628 MSE:  0.0522344747533372 UAR:  0.5653989801048624 Recall:  0.1411764705882353 Precision:  0.41379310344827586 F1:  0.21052631578947367
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional_best Accuracy:  0.9524085896691816 MSE:  0.04759141033081834 UAR:  0.5566867772750126 Recall:  0.11764705882352941 Precision:  0.5882352941176471 F1:  0.19607843137254902
Metric_name:  enet_b0_8_best

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### Balanced split

In [73]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [74]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  5482
Test:  1723


In [75]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 5234/5482: 95.4761036118205%
1 248/5482: 4.523896388179496%
val:
0 1638/1723: 95.06674405107371%
1 85/1723: 4.933255948926291%


In [76]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.76)

(5482,) (5482,)
(1723,) (1723,)
(7205,) (7205,)
[0 1] [6872  333]
7205
5475
1730


In [77]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 5222/5475: 95.37899543378995%
1 253/5475: 4.621004566210046%
val:
0 1650/1730: 95.3757225433526%
1 80/1730: 4.624277456647399%


##### Single attention

In [78]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_2_max_single_attention_balanced


In [79]:
CONCATENATE_STAT = True

In [80]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5475, 128, 2560) (5475,)
(1730, 128, 2560) (1730,)


In [81]:
(unique, counts) = np.unique(y_subsample_train, return_counts=True)
num_classes=len(unique)
cw=1/counts
cw/=cw.min()
class_weights = {i:cwi for i,cwi in zip(unique,cw)}
print(counts, class_weights, num_classes, unique)

[5222  253] {0: 1.0, 1: 20.640316205533598} 2 [0 1]


In [82]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)
#save_best_model = SaveBestModelByUAR(X_subsample_test, y_subsample_test)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2561      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [83]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test), class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 4s 68ms/step - loss: 1.3650 - acc: 0.6424 - auc: 0.6571 - binary_accuracy: 0.6424 - recall_3: 0.5810 - precision_3: 0.0735 - val_loss: 0.7326 - val_acc: 0.5751 - val_auc: 0.7461 - val_binary_accuracy: 0.5751 - val_recall_3: 0.7750 - val_precision_3: 0.0796
Epoch 2/20
43/43 [==============================] - 1s 31ms/step - loss: 1.0975 - acc: 0.7299 - auc: 0.7768 - binary_accuracy: 0.7299 - recall_3: 0.6838 - precision_3: 0.1101 - val_loss: 0.5188 - val_acc: 0.7809 - val_auc: 0.7722 - val_binary_accuracy: 0.7809 - val_recall_3: 0.6000 - val_precision_3: 0.1215
Epoch 3/20
43/43 [==============================] - 1s 31ms/step - loss: 1.0762 - acc: 0.7598 - auc: 0.8020 - binary_accuracy: 0.7598 - recall_3: 0.6917 - precision_3: 0.1239 - val_loss: 0.5161 - val_acc: 0.7954 - val_auc: 0.7711 - val_binary_accuracy: 0.7954 - val_recall_3: 0.6125 - val_precision_3: 0.1317
Epoch 4/20
43/43 [==============================] - 1s 31ms/step - loss: 

In [84]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_2765461/1443316788.py:29: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [85]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name, from_int=True)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_single_attention_balanced Accuracy:  0.44913294797687864 MSE:  0.5508670520231214 UAR:  0.6101136363636364 Recall:  0.7875 Precision:  0.06306306306306306 F1:  0.11677479147358663
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_single_attention_balanced Accuracy:  0.5508670520231214 MSE:  0.44913294797687864 UAR:  0.6515530303030304 Recall:  0.7625 Precision:  0.07448107448107448 F1:  0.13570634037819798
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_single_attention_balanced Accuracy:  0.63121387283237 MSE:  0.36878612716763004 UAR:  0.6639393939393939 Recall:  0.7 Precision:  0.08358208955223881 F1:  0.14933333333333335
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_single_attention_balanced Accuracy:  0.7023121387283237 MSE:  0.2976878612716763 UAR:  0.6952651515151516 Recall:  0.6875 Precision:  0.10091743119266056 F1:  0.176
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_single_attention_balanced Accuracy:  0.75895953757

In [86]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [87]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_2765461/1443316788.py:29: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [88]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name, from_int=True)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_single_attention_balanced_best Accuracy:  0.523121387283237 MSE:  0.476878612716763 UAR:  0.6548484848484848 Recall:  0.8 Precision:  0.07331042382588775 F1:  0.13431269674711438
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_single_attention_balanced_best Accuracy:  0.6890173410404624 MSE:  0.3109826589595376 UAR:  0.6882954545454545 Recall:  0.6875 Precision:  0.09683098591549295 F1:  0.16975308641975306
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_single_attention_balanced_best Accuracy:  0.7826589595375723 MSE:  0.21734104046242775 UAR:  0.7135984848484849 Recall:  0.6375 Precision:  0.12814070351758794 F1:  0.21338912133891216
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_single_attention_balanced_best Accuracy:  0.8398843930635839 MSE:  0.1601156069364162 UAR:  0.7257575757575758 Recall:  0.6 Precision:  0.16382252559726962 F1:  0.257372654155496
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_single_attention_balanc

##### Self attention

In [223]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced


In [224]:
CONCATENATE_STAT = True

In [225]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5475, 128, 2560) (5475,)
(1730, 128, 2560) (1730,)


In [226]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_15"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention_7 (Attention)     (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']        

In [227]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 7s 125ms/step - loss: 0.3764 - acc: 0.9412 - auc: 0.5374 - binary_accuracy: 0.9412 - recall_3: 0.0277 - precision_3: 0.0843 - val_loss: 0.1875 - val_acc: 0.9526 - val_auc: 0.6787 - val_binary_accuracy: 0.9526 - val_recall_3: 0.0125 - val_precision_3: 0.2500
Epoch 2/20
43/43 [==============================] - 4s 89ms/step - loss: 0.1737 - acc: 0.9540 - auc: 0.7014 - binary_accuracy: 0.9540 - recall_3: 0.0514 - precision_3: 0.5200 - val_loss: 0.1701 - val_acc: 0.9526 - val_auc: 0.7431 - val_binary_accuracy: 0.9526 - val_recall_3: 0.0250 - val_precision_3: 0.3333
Epoch 3/20
43/43 [==============================] - 4s 88ms/step - loss: 0.1627 - acc: 0.9536 - auc: 0.7562 - binary_accuracy: 0.9536 - recall_3: 0.0435 - precision_3: 0.4783 - val_loss: 0.1690 - val_acc: 0.9526 - val_auc: 0.7421 - val_binary_accuracy: 0.9526 - val_recall_3: 0.0000e+00 - val_precision_3: 0.0000e+00
Epoch 4/20
43/43 [==============================] - 4s 88ms/step

In [228]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [229]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced Accuracy:  0.9445086705202312 MSE:  0.055491329479768786 UAR:  0.5903030303030303 Recall:  0.2 Precision:  0.3333333333333333 F1:  0.25
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced Accuracy:  0.9520231213872833 MSE:  0.04797687861271676 UAR:  0.5526136363636364 Recall:  0.1125 Precision:  0.42857142857142855 F1:  0.1782178217821782
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced Accuracy:  0.953757225433526 MSE:  0.046242774566473986 UAR:  0.5297348484848485 Recall:  0.0625 Precision:  0.5 F1:  0.1111111111111111
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced Accuracy:  0.954335260115607 MSE:  0.04566473988439306 UAR:  0.5181439393939394 Recall:  0.0375 Precision:  0.6 F1:  0.07058823529411765
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced Accuracy:  0.9531791907514451 MSE:  0.04682080924855491 UAR:  0.505

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [230]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [231]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [232]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced_best Accuracy:  0.8549132947976879 MSE:  0.14508670520231215 UAR:  0.6979545454545455 Recall:  0.525 Precision:  0.16470588235294117 F1:  0.2507462686567164
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced_best Accuracy:  0.9260115606936417 MSE:  0.07398843930635839 UAR:  0.6460227272727272 Recall:  0.3375 Precision:  0.2647058823529412 F1:  0.29670329670329665
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced_best Accuracy:  0.946242774566474 MSE:  0.05375722543352601 UAR:  0.5852651515151515 Recall:  0.1875 Precision:  0.3488372093023256 F1:  0.2439024390243903
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced_best Accuracy:  0.954335260115607 MSE:  0.04566473988439306 UAR:  0.5716666666666667 Recall:  0.15 Precision:  0.5217391304347826 F1:  0.23300970873786406
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced_best

### STAT

In [89]:
stat_func = compute_descriptor
stat_name = "STAT"

#### Traditional split

In [90]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
val_pickle = FEATURES_DIR + 'val_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)
filename2features_val = load_features(val_pickle)

In [91]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))
print("Val: ", len(filename2features_val))

Train:  5482
Test:  1723
Val:  1720


In [92]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))
print("val:")
print_classes(get_labels(filename2features_val))

train:
0 5234/5482: 95.4761036118205%
1 248/5482: 4.523896388179496%
test:
0 1638/1723: 95.06674405107371%
1 85/1723: 4.933255948926291%
val:
0 1537/1720: 89.36046511627907%
1 183/1720: 10.63953488372093%


##### Single attention

In [93]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_2_STAT_single_attention_traditional


In [94]:
CONCATENATE_STAT = True

In [95]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 3840) (5482,)
(1723, 128, 3840) (1723,)
(1720, 128, 3840) (1720,)


In [96]:
(unique, counts) = np.unique(y_subsample_train, return_counts=True)
num_classes=len(unique)
cw=1/counts
cw/=cw.min()
class_weights = {i:cwi for i,cwi in zip(unique,cw)}
print(counts, class_weights, num_classes, unique)

[5234  248] {0: 1.0, 1: 21.10483870967742} 2 [0 1]


In [97]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)
#save_best_model = SaveBestModelByUAR(X_subsample_val, y_subsample_val)

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3840)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              3841      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [98]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=11, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val), class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/11
43/43 [==============================] - 6s 95ms/step - loss: 1.3552 - acc: 0.6476 - auc: 0.6517 - binary_accuracy: 0.6476 - recall_4: 0.5927 - precision_4: 0.0743 - val_loss: 0.7934 - val_acc: 0.5988 - val_auc: 0.6766 - val_binary_accuracy: 0.5988 - val_recall_4: 0.6721 - val_precision_4: 0.1633
Epoch 2/11
43/43 [==============================] - 2s 44ms/step - loss: 1.0784 - acc: 0.7344 - auc: 0.7831 - binary_accuracy: 0.7344 - recall_4: 0.7097 - precision_4: 0.1128 - val_loss: 0.7322 - val_acc: 0.6145 - val_auc: 0.6804 - val_binary_accuracy: 0.6145 - val_recall_4: 0.6175 - val_precision_4: 0.1601
Epoch 3/11
43/43 [==============================] - 2s 44ms/step - loss: 1.0498 - acc: 0.7381 - auc: 0.7994 - binary_accuracy: 0.7381 - recall_4: 0.7097 - precision_4: 0.1143 - val_loss: 0.5746 - val_acc: 0.6738 - val_auc: 0.6748 - val_binary_accuracy: 0.6738 - val_recall_4: 0.5301 - val_precision_4: 0.1696
Epoch 4/11
43/43 [==============================] - 2s 44ms/step - loss: 

In [99]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_2765461/1443316788.py:29: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [100]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name, from_int=True)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_single_attention_traditional Accuracy:  0.403946604759141 MSE:  0.596053395240859 UAR:  0.6195827048768225 Recall:  0.8588235294117647 Precision:  0.06709558823529412 F1:  0.12446717817561806
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_single_attention_traditional Accuracy:  0.6987811955890888 MSE:  0.3012188044109112 UAR:  0.6686849098613804 Recall:  0.6352941176470588 Precision:  0.0996309963099631 F1:  0.17224880382775118
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_single_attention_traditional Accuracy:  0.8183401044689496 MSE:  0.1816598955310505 UAR:  0.6702183437477556 Recall:  0.5058823529411764 Precision:  0.13694267515923567 F1:  0.21553884711779447
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_single_attention_traditional Accuracy:  0.881601857225769 MSE:  0.118398142774231 UAR:  0.6700280112044819 Recall:  0.43529411764705883 Precision:  0.19170984455958548 F1:  0.2661870503597122
Metric_name:  enet_b0_8_b

In [101]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [102]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_2765461/1443316788.py:29: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [103]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name, from_int=True)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_single_attention_traditional_best Accuracy:  0.403946604759141 MSE:  0.596053395240859 UAR:  0.6195827048768225 Recall:  0.8588235294117647 Precision:  0.06709558823529412 F1:  0.12446717817561806
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_single_attention_traditional_best Accuracy:  0.6987811955890888 MSE:  0.3012188044109112 UAR:  0.6686849098613804 Recall:  0.6352941176470588 Precision:  0.0996309963099631 F1:  0.17224880382775118
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_single_attention_traditional_best Accuracy:  0.8183401044689496 MSE:  0.1816598955310505 UAR:  0.6702183437477556 Recall:  0.5058823529411764 Precision:  0.13694267515923567 F1:  0.21553884711779447
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_single_attention_traditional_best Accuracy:  0.881601857225769 MSE:  0.118398142774231 UAR:  0.6700280112044819 Recall:  0.43529411764705883 Precision:  0.19170984455958548 F1:  0.2661870503597122
Metri

##### Self attention

In [ ]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

In [199]:
CONCATENATE_STAT = True

In [200]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 2560) (5482,)
(1723, 128, 2560) (1723,)
(1720, 128, 2560) (1720,)


In [201]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_13"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention_6 (Attention)     (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']        

In [202]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 7s 127ms/step - loss: 0.3520 - acc: 0.9435 - auc: 0.5372 - binary_accuracy: 0.9435 - recall_1: 0.0121 - precision_1: 0.0441 - val_loss: 0.3655 - val_acc: 0.8936 - val_auc: 0.6054 - val_binary_accuracy: 0.8936 - val_recall_1: 0.0000e+00 - val_precision_1: 0.0000e+00
Epoch 2/20
43/43 [==============================] - 4s 89ms/step - loss: 0.1676 - acc: 0.9544 - auc: 0.7332 - binary_accuracy: 0.9544 - recall_1: 0.0081 - precision_1: 0.3333 - val_loss: 0.4156 - val_acc: 0.8936 - val_auc: 0.6291 - val_binary_accuracy: 0.8936 - val_recall_1: 0.0000e+00 - val_precision_1: 0.0000e+00
Epoch 3/20
43/43 [==============================] - 4s 89ms/step - loss: 0.1589 - acc: 0.9533 - auc: 0.7800 - binary_accuracy: 0.9533 - recall_1: 0.0081 - precision_1: 0.1667 - val_loss: 0.3692 - val_acc: 0.8907 - val_auc: 0.6405 - val_binary_accuracy: 0.8907 - val_recall_1: 0.0328 - val_precision_1: 0.3529
Epoch 4/20
43/43 [==============================] - 4s 9

In [203]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [204]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional Accuracy:  0.8415554265815438 MSE:  0.1584445734184562 UAR:  0.6043489190548015 Recall:  0.3411764705882353 Precision:  0.11788617886178862 F1:  0.17522658610271902
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional Accuracy:  0.930354033662217 MSE:  0.06964596633778293 UAR:  0.5450872656755009 Recall:  0.11764705882352941 Precision:  0.18181818181818182 F1:  0.14285714285714285
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional Accuracy:  0.9437028438769588 MSE:  0.05629715612304121 UAR:  0.5409538174244057 Recall:  0.09411764705882353 Precision:  0.2857142857142857 F1:  0.1415929203539823
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional Accuracy:  0.947185142193848 MSE:  0.05281485780615206 UAR:  0.526054011348129 Recall:  0.058823529411764705 Precision:  0.3125 F1:  0.09900990099009901
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE

In [205]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [206]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [207]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional_best Accuracy:  0.8340104468949506 MSE:  0.16598955310504934 UAR:  0.6282661782661783 Recall:  0.4 Precision:  0.12639405204460966 F1:  0.192090395480226
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional_best Accuracy:  0.9297736506094022 MSE:  0.0702263493905978 UAR:  0.5949759390935861 Recall:  0.2235294117647059 Precision:  0.25675675675675674 F1:  0.2389937106918239
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional_best Accuracy:  0.9477655252466628 MSE:  0.0522344747533372 UAR:  0.5653989801048624 Recall:  0.1411764705882353 Precision:  0.41379310344827586 F1:  0.21052631578947367
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional_best Accuracy:  0.9524085896691816 MSE:  0.04759141033081834 UAR:  0.5566867772750126 Recall:  0.11764705882352941 Precision:  0.5882352941176471 F1:  0.19607843137254902
Metric_name:  enet_b0_8_best

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### Balanced split

In [104]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [105]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  5482
Test:  1723


In [106]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 5234/5482: 95.4761036118205%
1 248/5482: 4.523896388179496%
val:
0 1638/1723: 95.06674405107371%
1 85/1723: 4.933255948926291%


In [107]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.76)

(5482, 2560) (5482,)
(1723, 2560) (1723,)
(7205, 2560) (7205,)
[0 1] [6872  333]
7205
5475
1730


In [108]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 5222/5475: 95.37899543378995%
1 253/5475: 4.621004566210046%
val:
0 1650/1730: 95.3757225433526%
1 80/1730: 4.624277456647399%


##### Single attention

In [109]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_2_STAT_single_attention_balanced


In [110]:
CONCATENATE_STAT = True

In [111]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5475, 128, 3840) (5475,)
(1730, 128, 3840) (1730,)


In [112]:
(unique, counts) = np.unique(y_subsample_train, return_counts=True)
num_classes=len(unique)
cw=1/counts
cw/=cw.min()
class_weights = {i:cwi for i,cwi in zip(unique,cw)}
print(counts, class_weights, num_classes, unique)

[5222  253] {0: 1.0, 1: 20.640316205533598} 2 [0 1]


In [113]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)
#save_best_model = SaveBestModelByUAR(X_subsample_test, y_subsample_test)

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3840)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              3841      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [114]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=11, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test), class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/11
43/43 [==============================] - 5s 93ms/step - loss: 1.2319 - acc: 0.6530 - auc: 0.6958 - binary_accuracy: 0.6530 - recall_5: 0.6166 - precision_5: 0.0796 - val_loss: 0.5303 - val_acc: 0.7821 - val_auc: 0.7755 - val_binary_accuracy: 0.7821 - val_recall_5: 0.6500 - val_precision_5: 0.1297
Epoch 2/11
43/43 [==============================] - 2s 44ms/step - loss: 1.1106 - acc: 0.7507 - auc: 0.7852 - binary_accuracy: 0.7507 - recall_5: 0.7036 - precision_5: 0.1213 - val_loss: 0.4811 - val_acc: 0.7954 - val_auc: 0.7831 - val_binary_accuracy: 0.7954 - val_recall_5: 0.6500 - val_precision_5: 0.1376
Epoch 3/11
43/43 [==============================] - 2s 44ms/step - loss: 1.0441 - acc: 0.7468 - auc: 0.7991 - binary_accuracy: 0.7468 - recall_5: 0.7194 - precision_5: 0.1216 - val_loss: 0.4548 - val_acc: 0.8197 - val_auc: 0.7902 - val_binary_accuracy: 0.8197 - val_recall_5: 0.6125 - val_precision_5: 0.1485
Epoch 4/11
43/43 [==============================] - 2s 43ms/step - loss: 

In [115]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_2765461/1443316788.py:29: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [116]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name, from_int=True)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_single_attention_balanced Accuracy:  0.26878612716763006 MSE:  0.7312138728323699 UAR:  0.5869318181818182 Recall:  0.9375 Precision:  0.056179775280898875 F1:  0.10600706713780918
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_single_attention_balanced Accuracy:  0.3832369942196532 MSE:  0.6167630057803468 UAR:  0.6171969696969697 Recall:  0.875 Precision:  0.062111801242236024 F1:  0.115990057995029
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_single_attention_balanced Accuracy:  0.5 MSE:  0.5 UAR:  0.6665151515151515 Recall:  0.85 Precision:  0.0738327904451683 F1:  0.13586413586413587
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_single_attention_balanced Accuracy:  0.6138728323699422 MSE:  0.3861271676300578 UAR:  0.6964772727272728 Recall:  0.7875 Precision:  0.08823529411764706 F1:  0.15869017632241814
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_single_attention_balanced Accuracy:  0.708092485549133 MSE:  0

In [117]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [118]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_2765461/1443316788.py:29: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [119]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name, from_int=True)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_single_attention_balanced_best Accuracy:  0.2514450867052023 MSE:  0.7485549132947977 UAR:  0.5837878787878787 Recall:  0.95 Precision:  0.055596196049743966 F1:  0.1050449205252246
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_single_attention_balanced_best Accuracy:  0.4485549132947977 MSE:  0.5514450867052023 UAR:  0.6395454545454545 Recall:  0.85 Precision:  0.06732673267326733 F1:  0.12477064220183488
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_single_attention_balanced_best Accuracy:  0.6161849710982659 MSE:  0.3838150289017341 UAR:  0.697689393939394 Recall:  0.7875 Precision:  0.08873239436619719 F1:  0.15949367088607597
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_single_attention_balanced_best Accuracy:  0.7514450867052023 MSE:  0.24855491329479767 UAR:  0.7210227272727272 Recall:  0.6875 Precision:  0.11956521739130435 F1:  0.20370370370370372
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_single_attent

##### Self attention

In [223]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced


In [224]:
CONCATENATE_STAT = True

In [225]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5475, 128, 2560) (5475,)
(1730, 128, 2560) (1730,)


In [226]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_15"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention_7 (Attention)     (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']        

In [227]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 7s 125ms/step - loss: 0.3764 - acc: 0.9412 - auc: 0.5374 - binary_accuracy: 0.9412 - recall_3: 0.0277 - precision_3: 0.0843 - val_loss: 0.1875 - val_acc: 0.9526 - val_auc: 0.6787 - val_binary_accuracy: 0.9526 - val_recall_3: 0.0125 - val_precision_3: 0.2500
Epoch 2/20
43/43 [==============================] - 4s 89ms/step - loss: 0.1737 - acc: 0.9540 - auc: 0.7014 - binary_accuracy: 0.9540 - recall_3: 0.0514 - precision_3: 0.5200 - val_loss: 0.1701 - val_acc: 0.9526 - val_auc: 0.7431 - val_binary_accuracy: 0.9526 - val_recall_3: 0.0250 - val_precision_3: 0.3333
Epoch 3/20
43/43 [==============================] - 4s 88ms/step - loss: 0.1627 - acc: 0.9536 - auc: 0.7562 - binary_accuracy: 0.9536 - recall_3: 0.0435 - precision_3: 0.4783 - val_loss: 0.1690 - val_acc: 0.9526 - val_auc: 0.7421 - val_binary_accuracy: 0.9526 - val_recall_3: 0.0000e+00 - val_precision_3: 0.0000e+00
Epoch 4/20
43/43 [==============================] - 4s 88ms/step

In [228]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [229]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced Accuracy:  0.9445086705202312 MSE:  0.055491329479768786 UAR:  0.5903030303030303 Recall:  0.2 Precision:  0.3333333333333333 F1:  0.25
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced Accuracy:  0.9520231213872833 MSE:  0.04797687861271676 UAR:  0.5526136363636364 Recall:  0.1125 Precision:  0.42857142857142855 F1:  0.1782178217821782
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced Accuracy:  0.953757225433526 MSE:  0.046242774566473986 UAR:  0.5297348484848485 Recall:  0.0625 Precision:  0.5 F1:  0.1111111111111111
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced Accuracy:  0.954335260115607 MSE:  0.04566473988439306 UAR:  0.5181439393939394 Recall:  0.0375 Precision:  0.6 F1:  0.07058823529411765
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced Accuracy:  0.9531791907514451 MSE:  0.04682080924855491 UAR:  0.505

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [230]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [231]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [232]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced_best Accuracy:  0.8549132947976879 MSE:  0.14508670520231215 UAR:  0.6979545454545455 Recall:  0.525 Precision:  0.16470588235294117 F1:  0.2507462686567164
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced_best Accuracy:  0.9260115606936417 MSE:  0.07398843930635839 UAR:  0.6460227272727272 Recall:  0.3375 Precision:  0.2647058823529412 F1:  0.29670329670329665
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced_best Accuracy:  0.946242774566474 MSE:  0.05375722543352601 UAR:  0.5852651515151515 Recall:  0.1875 Precision:  0.3488372093023256 F1:  0.2439024390243903
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced_best Accuracy:  0.954335260115607 MSE:  0.04566473988439306 UAR:  0.5716666666666667 Recall:  0.15 Precision:  0.5217391304347826 F1:  0.23300970873786406
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced_best

## MobileNet_7

In [25]:
base_model_key = 'mobilenet_7.h5'

In [26]:
ENGAGE_WILD_DATASET, BIN_CLASSIFICATION, N_CLASSES, DATASET_NAME, BASE_DATASET_DIR, ext = define_dataset(False, True)
video2label, labels_list = get_video2label()
print(labels_list)
print(video2label)

['engaged', 'distracted']
{'1100011002': 0, '1100011003': 0, '1100011004': 0, '1100011005': 0, '1100011006': 0, '1100011007': 0, '1100011008': 0, '1100011009': 0, '1100011010': 0, '1100011011': 0, '1100011012': 0, '1100011013': 0, '1100011014': 0, '1100011015': 0, '1100011016': 0, '1100011017': 0, '1100011018': 0, '1100011019': 0, '1100011020': 0, '1100011021': 0, '1100011022': 0, '1100011023': 0, '1100011025': 0, '1100011026': 0, '1100011027': 0, '1100011028': 0, '1100011029': 0, '1100011031': 0, '1100011032': 0, '1100011034': 0, '1100011035': 0, '1100011037': 0, '1100011038': 0, '1100011040': 0, '1100011046': 0, '1100011047': 0, '1100011048': 0, '1100011049': 0, '1100011050': 0, '1100011051': 0, '1100011052': 0, '1100011053': 0, '1100011054': 0, '1100011055': 0, '1100011056': 0, '1100011057': 0, '1100011058': 0, '1100011059': 0, '1100011060': 0, '1100011062': 0, '1100011063': 0, '1100011064': 0, '1100011066': 0, '1100011067': 0, '1100011068': 0, '1100011069': 0, '1100011070': 0, '110

### Std

In [27]:
stat_func = np.std
stat_name = "std"

#### Traditional split

In [28]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
val_pickle = FEATURES_DIR + 'val_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)
filename2features_val = load_features(val_pickle)

In [29]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))
print("Val: ", len(filename2features_val))

Train:  5482
Test:  1723
Val:  1720


In [30]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))
print("val:")
print_classes(get_labels(filename2features_val))

train:
0 5234/5482: 95.4761036118205%
1 248/5482: 4.523896388179496%
test:
0 1638/1723: 95.06674405107371%
1 85/1723: 4.933255948926291%
val:
0 1537/1720: 89.36046511627907%
1 183/1720: 10.63953488372093%


##### Single attention

In [31]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

mobilenet_7.h5_DAiSEE_2_std_single_attention_traditional


In [32]:
CONCATENATE_STAT = True

In [33]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 2048) (5482,)
(1723, 128, 2048) (1723,)
(1720, 128, 2048) (1720,)


In [34]:
(unique, counts) = np.unique(y_subsample_train, return_counts=True)
num_classes=len(unique)
cw=1/counts
cw/=cw.min()
class_weights = {i:cwi for i,cwi in zip(unique,cw)}
print(counts, class_weights, num_classes, unique)

[5234  248] {0: 1.0, 1: 21.10483870967742} 2 [0 1]


In [35]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)
#save_best_model = SaveBestModelByUAR(X_subsample_val, y_subsample_val)

2024-08-11 21:37:55.974305: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38431 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:11:00.0, compute capability: 8.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2049      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                              

In [36]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val), class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20


2024-08-11 21:38:03.987877: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f85c00bc310 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-08-11 21:38:03.988038: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2024-08-11 21:38:03.992173: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-11 21:38:04.039536: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-08-11 21:38:04.091079: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


43/43 [==============================] - 6s 82ms/step - loss: 1.2558 - acc: 0.6410 - auc: 0.6688 - binary_accuracy: 0.6410 - recall: 0.6129 - precision: 0.0751 - val_loss: 0.7303 - val_acc: 0.5930 - val_auc: 0.6772 - val_binary_accuracy: 0.5930 - val_recall: 0.6721 - val_precision: 0.1612
Epoch 2/20
43/43 [==============================] - 1s 27ms/step - loss: 1.1178 - acc: 0.7162 - auc: 0.7648 - binary_accuracy: 0.7162 - recall: 0.7016 - precision: 0.1051 - val_loss: 0.5479 - val_acc: 0.7320 - val_auc: 0.6853 - val_binary_accuracy: 0.7320 - val_recall: 0.4809 - val_precision: 0.1938
Epoch 3/20
43/43 [==============================] - 1s 25ms/step - loss: 1.0737 - acc: 0.7324 - auc: 0.7823 - binary_accuracy: 0.7324 - recall: 0.6935 - precision: 0.1100 - val_loss: 1.0138 - val_acc: 0.3331 - val_auc: 0.7081 - val_binary_accuracy: 0.3331 - val_recall: 0.9344 - val_precision: 0.1309
Epoch 4/20
43/43 [==============================] - 1s 25ms/step - loss: 1.0101 - acc: 0.7499 - auc: 0.8130 

In [37]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3108480/1443316788.py:29: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [38]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name, from_int=True)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_DAiSEE_2_std_single_attention_traditional Accuracy:  0.3087637840975044 MSE:  0.6912362159024956 UAR:  0.552790346907994 Recall:  0.8235294117647058 Precision:  0.056179775280898875 F1:  0.10518407212622088
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_single_attention_traditional Accuracy:  0.4341265235055136 MSE:  0.5658734764944864 UAR:  0.5573762838468721 Recall:  0.6941176470588235 Precision:  0.05853174603174603 F1:  0.10795974382433669
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_single_attention_traditional Accuracy:  0.5536854323853744 MSE:  0.4463145676146257 UAR:  0.5756410256410256 Recall:  0.6 Precision:  0.0648854961832061 F1:  0.11710677382319173
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_single_attention_traditional Accuracy:  0.6604759141033082 MSE:  0.3395240858966918 UAR:  0.6318070818070818 Recall:  0.6 Precision:  0.08471760797342193 F1:  0.14847161572052403
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_single_attention_traditional Accuracy:  0.752

In [39]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [40]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3108480/1443316788.py:29: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [41]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name, from_int=True)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_DAiSEE_2_std_single_attention_traditional_best Accuracy:  0.3064422518862449 MSE:  0.693557748113755 UAR:  0.5794548588666235 Recall:  0.8823529411764706 Precision:  0.05952380952380952 F1:  0.11152416356877322
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_single_attention_traditional_best Accuracy:  0.4904236796285548 MSE:  0.5095763203714452 UAR:  0.598139768728004 Recall:  0.7176470588235294 Precision:  0.06666666666666667 F1:  0.122
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_single_attention_traditional_best Accuracy:  0.632617527568195 MSE:  0.367382472431805 UAR:  0.5948466566113625 Recall:  0.5529411764705883 Precision:  0.07320872274143302 F1:  0.1292984869325997
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_single_attention_traditional_best Accuracy:  0.7544979686593152 MSE:  0.24550203134068485 UAR:  0.6422179128061482 Recall:  0.5176470588235295 Precision:  0.10328638497652583 F1:  0.1722113502935421
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_single_attent

##### Self attention

In [42]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

mobilenet_7.h5_DAiSEE_2_std_self_attention_traditional


In [43]:
CONCATENATE_STAT = True

In [200]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 2560) (5482,)
(1723, 128, 2560) (1723,)
(1720, 128, 2560) (1720,)


In [201]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_13"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention_6 (Attention)     (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']        

In [202]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 7s 127ms/step - loss: 0.3520 - acc: 0.9435 - auc: 0.5372 - binary_accuracy: 0.9435 - recall_1: 0.0121 - precision_1: 0.0441 - val_loss: 0.3655 - val_acc: 0.8936 - val_auc: 0.6054 - val_binary_accuracy: 0.8936 - val_recall_1: 0.0000e+00 - val_precision_1: 0.0000e+00
Epoch 2/20
43/43 [==============================] - 4s 89ms/step - loss: 0.1676 - acc: 0.9544 - auc: 0.7332 - binary_accuracy: 0.9544 - recall_1: 0.0081 - precision_1: 0.3333 - val_loss: 0.4156 - val_acc: 0.8936 - val_auc: 0.6291 - val_binary_accuracy: 0.8936 - val_recall_1: 0.0000e+00 - val_precision_1: 0.0000e+00
Epoch 3/20
43/43 [==============================] - 4s 89ms/step - loss: 0.1589 - acc: 0.9533 - auc: 0.7800 - binary_accuracy: 0.9533 - recall_1: 0.0081 - precision_1: 0.1667 - val_loss: 0.3692 - val_acc: 0.8907 - val_auc: 0.6405 - val_binary_accuracy: 0.8907 - val_recall_1: 0.0328 - val_precision_1: 0.3529
Epoch 4/20
43/43 [==============================] - 4s 9

In [203]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [204]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional Accuracy:  0.8415554265815438 MSE:  0.1584445734184562 UAR:  0.6043489190548015 Recall:  0.3411764705882353 Precision:  0.11788617886178862 F1:  0.17522658610271902
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional Accuracy:  0.930354033662217 MSE:  0.06964596633778293 UAR:  0.5450872656755009 Recall:  0.11764705882352941 Precision:  0.18181818181818182 F1:  0.14285714285714285
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional Accuracy:  0.9437028438769588 MSE:  0.05629715612304121 UAR:  0.5409538174244057 Recall:  0.09411764705882353 Precision:  0.2857142857142857 F1:  0.1415929203539823
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional Accuracy:  0.947185142193848 MSE:  0.05281485780615206 UAR:  0.526054011348129 Recall:  0.058823529411764705 Precision:  0.3125 F1:  0.09900990099009901
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE

In [205]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [206]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [207]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional_best Accuracy:  0.8340104468949506 MSE:  0.16598955310504934 UAR:  0.6282661782661783 Recall:  0.4 Precision:  0.12639405204460966 F1:  0.192090395480226
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional_best Accuracy:  0.9297736506094022 MSE:  0.0702263493905978 UAR:  0.5949759390935861 Recall:  0.2235294117647059 Precision:  0.25675675675675674 F1:  0.2389937106918239
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional_best Accuracy:  0.9477655252466628 MSE:  0.0522344747533372 UAR:  0.5653989801048624 Recall:  0.1411764705882353 Precision:  0.41379310344827586 F1:  0.21052631578947367
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional_best Accuracy:  0.9524085896691816 MSE:  0.04759141033081834 UAR:  0.5566867772750126 Recall:  0.11764705882352941 Precision:  0.5882352941176471 F1:  0.19607843137254902
Metric_name:  enet_b0_8_best

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### Balanced split

In [44]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [45]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  5482
Test:  1723


In [46]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 5234/5482: 95.4761036118205%
1 248/5482: 4.523896388179496%
val:
0 1638/1723: 95.06674405107371%
1 85/1723: 4.933255948926291%


In [47]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.76)

(5482,) (5482,)
(1723,) (1723,)
(7205,) (7205,)
[0 1] [6872  333]
7205
5475
1730


In [48]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 5222/5475: 95.37899543378995%
1 253/5475: 4.621004566210046%
val:
0 1650/1730: 95.3757225433526%
1 80/1730: 4.624277456647399%


##### Single attention

In [49]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

mobilenet_7.h5_DAiSEE_2_std_single_attention_balanced


In [50]:
CONCATENATE_STAT = True

In [51]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5475, 128, 2048) (5475,)
(1730, 128, 2048) (1730,)


In [52]:
(unique, counts) = np.unique(y_subsample_train, return_counts=True)
num_classes=len(unique)
cw=1/counts
cw/=cw.min()
class_weights = {i:cwi for i,cwi in zip(unique,cw)}
print(counts, class_weights, num_classes, unique)

[5222  253] {0: 1.0, 1: 20.640316205533598} 2 [0 1]


In [53]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)
#save_best_model = SaveBestModelByUAR(X_subsample_test, y_subsample_test)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2049      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [54]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test), class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 4s 62ms/step - loss: 1.2829 - acc: 0.6261 - auc: 0.6488 - binary_accuracy: 0.6261 - recall_1: 0.5850 - precision_1: 0.0708 - val_loss: 0.4208 - val_acc: 0.9000 - val_auc: 0.7464 - val_binary_accuracy: 0.9000 - val_recall_1: 0.3625 - val_precision_1: 0.1921
Epoch 2/20
43/43 [==============================] - 1s 25ms/step - loss: 1.1081 - acc: 0.7257 - auc: 0.7643 - binary_accuracy: 0.7257 - recall_1: 0.6798 - precision_1: 0.1080 - val_loss: 0.6158 - val_acc: 0.6919 - val_auc: 0.7683 - val_binary_accuracy: 0.6919 - val_recall_1: 0.7125 - val_precision_1: 0.1005
Epoch 3/20
43/43 [==============================] - 1s 26ms/step - loss: 1.0464 - acc: 0.7711 - auc: 0.7955 - binary_accuracy: 0.7711 - recall_1: 0.6759 - precision_1: 0.1274 - val_loss: 0.6546 - val_acc: 0.6405 - val_auc: 0.7692 - val_binary_accuracy: 0.6405 - val_recall_1: 0.7625 - val_precision_1: 0.0919
Epoch 4/20
43/43 [==============================] - 1s 27ms/step - loss: 

In [55]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3108480/1443316788.py:29: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [56]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name, from_int=True)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_DAiSEE_2_std_single_attention_balanced Accuracy:  0.36416184971098264 MSE:  0.6358381502890174 UAR:  0.5953030303030302 Recall:  0.85 Precision:  0.058823529411764705 F1:  0.11003236245954692
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_single_attention_balanced Accuracy:  0.47398843930635837 MSE:  0.5260115606936416 UAR:  0.6231439393939394 Recall:  0.7875 Precision:  0.06589958158995816 F1:  0.12162162162162164
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_single_attention_balanced Accuracy:  0.5716763005780346 MSE:  0.4283236994219653 UAR:  0.6624621212121211 Recall:  0.7625 Precision:  0.07790549169859515 F1:  0.1413673232908459
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_single_attention_balanced Accuracy:  0.6456647398843931 MSE:  0.35433526011560695 UAR:  0.6715151515151515 Recall:  0.7 Precision:  0.08682170542635659 F1:  0.15448275862068966
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_single_attention_balanced Accuracy:  0.715028901734104 MSE:  0.2849710982658

In [57]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [58]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3108480/1443316788.py:29: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [59]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name, from_int=True)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_DAiSEE_2_std_single_attention_balanced_best Accuracy:  0.29132947976878615 MSE:  0.7086705202312139 UAR:  0.5630681818181819 Recall:  0.8625 Precision:  0.053738317757009345 F1:  0.10117302052785923
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_single_attention_balanced_best Accuracy:  0.5410404624277456 MSE:  0.45895953757225433 UAR:  0.6582954545454545 Recall:  0.7875 Precision:  0.075 F1:  0.13695652173913045
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_single_attention_balanced_best Accuracy:  0.6976878612716763 MSE:  0.3023121387283237 UAR:  0.7047348484848486 Recall:  0.7125 Precision:  0.10233393177737882 F1:  0.17896389324960754
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_single_attention_balanced_best Accuracy:  0.7971098265895954 MSE:  0.20289017341040463 UAR:  0.6914393939393939 Recall:  0.575 Precision:  0.12672176308539945 F1:  0.20767494356659144
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_single_attention_balanced_best Accuracy:  0.8722543352601156 MSE:

##### Self attention

In [223]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced


In [224]:
CONCATENATE_STAT = True

In [225]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5475, 128, 2560) (5475,)
(1730, 128, 2560) (1730,)


In [226]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_15"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention_7 (Attention)     (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']        

In [227]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 7s 125ms/step - loss: 0.3764 - acc: 0.9412 - auc: 0.5374 - binary_accuracy: 0.9412 - recall_3: 0.0277 - precision_3: 0.0843 - val_loss: 0.1875 - val_acc: 0.9526 - val_auc: 0.6787 - val_binary_accuracy: 0.9526 - val_recall_3: 0.0125 - val_precision_3: 0.2500
Epoch 2/20
43/43 [==============================] - 4s 89ms/step - loss: 0.1737 - acc: 0.9540 - auc: 0.7014 - binary_accuracy: 0.9540 - recall_3: 0.0514 - precision_3: 0.5200 - val_loss: 0.1701 - val_acc: 0.9526 - val_auc: 0.7431 - val_binary_accuracy: 0.9526 - val_recall_3: 0.0250 - val_precision_3: 0.3333
Epoch 3/20
43/43 [==============================] - 4s 88ms/step - loss: 0.1627 - acc: 0.9536 - auc: 0.7562 - binary_accuracy: 0.9536 - recall_3: 0.0435 - precision_3: 0.4783 - val_loss: 0.1690 - val_acc: 0.9526 - val_auc: 0.7421 - val_binary_accuracy: 0.9526 - val_recall_3: 0.0000e+00 - val_precision_3: 0.0000e+00
Epoch 4/20
43/43 [==============================] - 4s 88ms/step

In [228]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [229]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced Accuracy:  0.9445086705202312 MSE:  0.055491329479768786 UAR:  0.5903030303030303 Recall:  0.2 Precision:  0.3333333333333333 F1:  0.25
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced Accuracy:  0.9520231213872833 MSE:  0.04797687861271676 UAR:  0.5526136363636364 Recall:  0.1125 Precision:  0.42857142857142855 F1:  0.1782178217821782
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced Accuracy:  0.953757225433526 MSE:  0.046242774566473986 UAR:  0.5297348484848485 Recall:  0.0625 Precision:  0.5 F1:  0.1111111111111111
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced Accuracy:  0.954335260115607 MSE:  0.04566473988439306 UAR:  0.5181439393939394 Recall:  0.0375 Precision:  0.6 F1:  0.07058823529411765
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced Accuracy:  0.9531791907514451 MSE:  0.04682080924855491 UAR:  0.505

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [230]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [231]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [232]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced_best Accuracy:  0.8549132947976879 MSE:  0.14508670520231215 UAR:  0.6979545454545455 Recall:  0.525 Precision:  0.16470588235294117 F1:  0.2507462686567164
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced_best Accuracy:  0.9260115606936417 MSE:  0.07398843930635839 UAR:  0.6460227272727272 Recall:  0.3375 Precision:  0.2647058823529412 F1:  0.29670329670329665
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced_best Accuracy:  0.946242774566474 MSE:  0.05375722543352601 UAR:  0.5852651515151515 Recall:  0.1875 Precision:  0.3488372093023256 F1:  0.2439024390243903
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced_best Accuracy:  0.954335260115607 MSE:  0.04566473988439306 UAR:  0.5716666666666667 Recall:  0.15 Precision:  0.5217391304347826 F1:  0.23300970873786406
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced_best

### Max

In [60]:
stat_func = np.max
stat_name = "max"

#### Traditional split

In [61]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
val_pickle = FEATURES_DIR + 'val_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)
filename2features_val = load_features(val_pickle)

In [62]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))
print("Val: ", len(filename2features_val))

Train:  5482
Test:  1723
Val:  1720


In [63]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))
print("val:")
print_classes(get_labels(filename2features_val))

train:
0 5234/5482: 95.4761036118205%
1 248/5482: 4.523896388179496%
test:
0 1638/1723: 95.06674405107371%
1 85/1723: 4.933255948926291%
val:
0 1537/1720: 89.36046511627907%
1 183/1720: 10.63953488372093%


##### Single attention

In [64]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

mobilenet_7.h5_DAiSEE_2_max_single_attention_traditional


In [65]:
CONCATENATE_STAT = True

In [66]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 2048) (5482,)
(1723, 128, 2048) (1723,)
(1720, 128, 2048) (1720,)


In [67]:
(unique, counts) = np.unique(y_subsample_train, return_counts=True)
num_classes=len(unique)
cw=1/counts
cw/=cw.min()
class_weights = {i:cwi for i,cwi in zip(unique,cw)}
print(counts, class_weights, num_classes, unique)

[5234  248] {0: 1.0, 1: 21.10483870967742} 2 [0 1]


In [68]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)
#save_best_model = SaveBestModelByUAR(X_subsample_val, y_subsample_val)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2049      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [69]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val), class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 4s 56ms/step - loss: 1.3729 - acc: 0.6012 - auc: 0.6168 - binary_accuracy: 0.6012 - recall_2: 0.5605 - precision_2: 0.0627 - val_loss: 0.7126 - val_acc: 0.6221 - val_auc: 0.6898 - val_binary_accuracy: 0.6221 - val_recall_2: 0.6831 - val_precision_2: 0.1743
Epoch 2/20
43/43 [==============================] - 1s 25ms/step - loss: 1.1179 - acc: 0.7231 - auc: 0.7574 - binary_accuracy: 0.7231 - recall_2: 0.6573 - precision_2: 0.1021 - val_loss: 0.8344 - val_acc: 0.4750 - val_auc: 0.6674 - val_binary_accuracy: 0.4750 - val_recall_2: 0.7814 - val_precision_2: 0.1421
Epoch 3/20
43/43 [==============================] - 1s 26ms/step - loss: 1.0699 - acc: 0.7401 - auc: 0.7847 - binary_accuracy: 0.7401 - recall_2: 0.6976 - precision_2: 0.1136 - val_loss: 0.9737 - val_acc: 0.4378 - val_auc: 0.6840 - val_binary_accuracy: 0.4378 - val_recall_2: 0.8634 - val_precision_2: 0.1436
Epoch 4/20
43/43 [==============================] - 1s 27ms/step - loss: 

In [70]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3108480/1443316788.py:29: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [71]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name, from_int=True)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_DAiSEE_2_max_single_attention_traditional Accuracy:  0.4312246082414393 MSE:  0.5687753917585606 UAR:  0.5502729296846944 Recall:  0.6823529411764706 Precision:  0.057368941641938676 F1:  0.10583941605839416
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_single_attention_traditional Accuracy:  0.5722576900754498 MSE:  0.4277423099245502 UAR:  0.5909861380449616 Recall:  0.611764705882353 Precision:  0.06878306878306878 F1:  0.12366230677764564
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_single_attention_traditional Accuracy:  0.6796285548461984 MSE:  0.3203714451538015 UAR:  0.608417726064785 Recall:  0.5294117647058824 Precision:  0.0807899461400359 F1:  0.14018691588785048
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_single_attention_traditional Accuracy:  0.7777132907719094 MSE:  0.22228670922809055 UAR:  0.6265424118365295 Recall:  0.4588235294117647 Precision:  0.10372340425531915 F1:  0.1691973969631236
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_single_attention

In [72]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [73]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3108480/1443316788.py:29: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [74]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name, from_int=True)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_DAiSEE_2_max_single_attention_traditional_best Accuracy:  0.20951828206616366 MSE:  0.7904817179338364 UAR:  0.5675177763413057 Recall:  0.9647058823529412 Precision:  0.05690492713393477 F1:  0.10747051114023591
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_single_attention_traditional_best Accuracy:  0.4515380150899594 MSE:  0.5484619849100406 UAR:  0.5999964088199382 Recall:  0.7647058823529411 Precision:  0.06565656565656566 F1:  0.12093023255813955
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_single_attention_traditional_best Accuracy:  0.6500290191526408 MSE:  0.3499709808473593 UAR:  0.6263125763125763 Recall:  0.6 Precision:  0.08225806451612903 F1:  0.14468085106382977
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_single_attention_traditional_best Accuracy:  0.7893209518282066 MSE:  0.2106790481717934 UAR:  0.6270703153056094 Recall:  0.4470588235294118 Precision:  0.10734463276836158 F1:  0.17312072892938496
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_single_a

##### Self attention

In [198]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional


In [199]:
CONCATENATE_STAT = True

In [200]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 2560) (5482,)
(1723, 128, 2560) (1723,)
(1720, 128, 2560) (1720,)


In [201]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_13"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention_6 (Attention)     (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']        

In [202]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 7s 127ms/step - loss: 0.3520 - acc: 0.9435 - auc: 0.5372 - binary_accuracy: 0.9435 - recall_1: 0.0121 - precision_1: 0.0441 - val_loss: 0.3655 - val_acc: 0.8936 - val_auc: 0.6054 - val_binary_accuracy: 0.8936 - val_recall_1: 0.0000e+00 - val_precision_1: 0.0000e+00
Epoch 2/20
43/43 [==============================] - 4s 89ms/step - loss: 0.1676 - acc: 0.9544 - auc: 0.7332 - binary_accuracy: 0.9544 - recall_1: 0.0081 - precision_1: 0.3333 - val_loss: 0.4156 - val_acc: 0.8936 - val_auc: 0.6291 - val_binary_accuracy: 0.8936 - val_recall_1: 0.0000e+00 - val_precision_1: 0.0000e+00
Epoch 3/20
43/43 [==============================] - 4s 89ms/step - loss: 0.1589 - acc: 0.9533 - auc: 0.7800 - binary_accuracy: 0.9533 - recall_1: 0.0081 - precision_1: 0.1667 - val_loss: 0.3692 - val_acc: 0.8907 - val_auc: 0.6405 - val_binary_accuracy: 0.8907 - val_recall_1: 0.0328 - val_precision_1: 0.3529
Epoch 4/20
43/43 [==============================] - 4s 9

In [203]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [204]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional Accuracy:  0.8415554265815438 MSE:  0.1584445734184562 UAR:  0.6043489190548015 Recall:  0.3411764705882353 Precision:  0.11788617886178862 F1:  0.17522658610271902
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional Accuracy:  0.930354033662217 MSE:  0.06964596633778293 UAR:  0.5450872656755009 Recall:  0.11764705882352941 Precision:  0.18181818181818182 F1:  0.14285714285714285
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional Accuracy:  0.9437028438769588 MSE:  0.05629715612304121 UAR:  0.5409538174244057 Recall:  0.09411764705882353 Precision:  0.2857142857142857 F1:  0.1415929203539823
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional Accuracy:  0.947185142193848 MSE:  0.05281485780615206 UAR:  0.526054011348129 Recall:  0.058823529411764705 Precision:  0.3125 F1:  0.09900990099009901
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE

In [205]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [206]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [207]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional_best Accuracy:  0.8340104468949506 MSE:  0.16598955310504934 UAR:  0.6282661782661783 Recall:  0.4 Precision:  0.12639405204460966 F1:  0.192090395480226
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional_best Accuracy:  0.9297736506094022 MSE:  0.0702263493905978 UAR:  0.5949759390935861 Recall:  0.2235294117647059 Precision:  0.25675675675675674 F1:  0.2389937106918239
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional_best Accuracy:  0.9477655252466628 MSE:  0.0522344747533372 UAR:  0.5653989801048624 Recall:  0.1411764705882353 Precision:  0.41379310344827586 F1:  0.21052631578947367
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional_best Accuracy:  0.9524085896691816 MSE:  0.04759141033081834 UAR:  0.5566867772750126 Recall:  0.11764705882352941 Precision:  0.5882352941176471 F1:  0.19607843137254902
Metric_name:  enet_b0_8_best

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### Balanced split

In [75]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [76]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  5482
Test:  1723


In [77]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 5234/5482: 95.4761036118205%
1 248/5482: 4.523896388179496%
val:
0 1638/1723: 95.06674405107371%
1 85/1723: 4.933255948926291%


In [78]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.76)

(5482,) (5482,)
(1723,) (1723,)
(7205,) (7205,)
[0 1] [6872  333]
7205
5475
1730


In [79]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 5222/5475: 95.37899543378995%
1 253/5475: 4.621004566210046%
val:
0 1650/1730: 95.3757225433526%
1 80/1730: 4.624277456647399%


##### Single attention

In [80]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

mobilenet_7.h5_DAiSEE_2_max_single_attention_balanced


In [81]:
CONCATENATE_STAT = True

In [82]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5475, 128, 2048) (5475,)
(1730, 128, 2048) (1730,)


In [83]:
(unique, counts) = np.unique(y_subsample_train, return_counts=True)
num_classes=len(unique)
cw=1/counts
cw/=cw.min()
class_weights = {i:cwi for i,cwi in zip(unique,cw)}
print(counts, class_weights, num_classes, unique)

[5222  253] {0: 1.0, 1: 20.640316205533598} 2 [0 1]


In [84]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)
#save_best_model = SaveBestModelByUAR(X_subsample_test, y_subsample_test)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2049      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [85]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test), class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 4s 59ms/step - loss: 1.5547 - acc: 0.6031 - auc: 0.6044 - binary_accuracy: 0.6031 - recall_3: 0.5810 - precision_3: 0.0664 - val_loss: 0.6103 - val_acc: 0.7260 - val_auc: 0.7356 - val_binary_accuracy: 0.7260 - val_recall_3: 0.6375 - val_precision_3: 0.1028
Epoch 2/20
43/43 [==============================] - 1s 25ms/step - loss: 1.1779 - acc: 0.6795 - auc: 0.7111 - binary_accuracy: 0.6795 - recall_3: 0.6364 - precision_3: 0.0883 - val_loss: 0.7102 - val_acc: 0.6023 - val_auc: 0.7705 - val_binary_accuracy: 0.6023 - val_recall_3: 0.8250 - val_precision_3: 0.0892
Epoch 3/20
43/43 [==============================] - 1s 27ms/step - loss: 1.0750 - acc: 0.7622 - auc: 0.7820 - binary_accuracy: 0.7622 - recall_3: 0.6877 - precision_3: 0.1246 - val_loss: 0.4661 - val_acc: 0.8364 - val_auc: 0.7632 - val_binary_accuracy: 0.8364 - val_recall_3: 0.6000 - val_precision_3: 0.1605
Epoch 4/20
43/43 [==============================] - 1s 26ms/step - loss: 

In [86]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3108480/1443316788.py:29: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [87]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name, from_int=True)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_DAiSEE_2_max_single_attention_balanced Accuracy:  0.28265895953757225 MSE:  0.7173410404624277 UAR:  0.564469696969697 Recall:  0.875 Precision:  0.053804765564950036 F1:  0.10137581462708181
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_single_attention_balanced Accuracy:  0.4023121387283237 MSE:  0.5976878612716763 UAR:  0.6093560606060606 Recall:  0.8375 Precision:  0.06158088235294118 F1:  0.11472602739726026
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_single_attention_balanced Accuracy:  0.5144508670520231 MSE:  0.48554913294797686 UAR:  0.6503030303030304 Recall:  0.8 Precision:  0.07207207207207207 F1:  0.13223140495867766
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_single_attention_balanced Accuracy:  0.5976878612716763 MSE:  0.4023121387283237 UAR:  0.6760984848484848 Recall:  0.7625 Precision:  0.08265582655826559 F1:  0.14914425427872863
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_single_attention_balanced Accuracy:  0.6797687861271676 MSE:  0.320231213872

In [88]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [89]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3108480/1443316788.py:29: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [90]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name, from_int=True)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_DAiSEE_2_max_single_attention_balanced_best Accuracy:  0.4601156069364162 MSE:  0.5398843930635838 UAR:  0.6277651515151516 Recall:  0.8125 Precision:  0.06605691056910569 F1:  0.12218045112781956
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_single_attention_balanced_best Accuracy:  0.6323699421965318 MSE:  0.3676300578034682 UAR:  0.676439393939394 Recall:  0.725 Precision:  0.08630952380952381 F1:  0.15425531914893617
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_single_attention_balanced_best Accuracy:  0.7468208092485549 MSE:  0.25317919075144507 UAR:  0.7007575757575757 Recall:  0.65 Precision:  0.11255411255411256 F1:  0.1918819188191882
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_single_attention_balanced_best Accuracy:  0.823121387283237 MSE:  0.176878612716763 UAR:  0.699128787878788 Recall:  0.5625 Precision:  0.14240506329113925 F1:  0.22727272727272732
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_single_attention_balanced_best Accuracy:  0.8820809248554913 

##### Self attention

In [223]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced


In [224]:
CONCATENATE_STAT = True

In [225]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5475, 128, 2560) (5475,)
(1730, 128, 2560) (1730,)


In [226]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_15"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention_7 (Attention)     (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']        

In [227]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 7s 125ms/step - loss: 0.3764 - acc: 0.9412 - auc: 0.5374 - binary_accuracy: 0.9412 - recall_3: 0.0277 - precision_3: 0.0843 - val_loss: 0.1875 - val_acc: 0.9526 - val_auc: 0.6787 - val_binary_accuracy: 0.9526 - val_recall_3: 0.0125 - val_precision_3: 0.2500
Epoch 2/20
43/43 [==============================] - 4s 89ms/step - loss: 0.1737 - acc: 0.9540 - auc: 0.7014 - binary_accuracy: 0.9540 - recall_3: 0.0514 - precision_3: 0.5200 - val_loss: 0.1701 - val_acc: 0.9526 - val_auc: 0.7431 - val_binary_accuracy: 0.9526 - val_recall_3: 0.0250 - val_precision_3: 0.3333
Epoch 3/20
43/43 [==============================] - 4s 88ms/step - loss: 0.1627 - acc: 0.9536 - auc: 0.7562 - binary_accuracy: 0.9536 - recall_3: 0.0435 - precision_3: 0.4783 - val_loss: 0.1690 - val_acc: 0.9526 - val_auc: 0.7421 - val_binary_accuracy: 0.9526 - val_recall_3: 0.0000e+00 - val_precision_3: 0.0000e+00
Epoch 4/20
43/43 [==============================] - 4s 88ms/step

In [228]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [229]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced Accuracy:  0.9445086705202312 MSE:  0.055491329479768786 UAR:  0.5903030303030303 Recall:  0.2 Precision:  0.3333333333333333 F1:  0.25
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced Accuracy:  0.9520231213872833 MSE:  0.04797687861271676 UAR:  0.5526136363636364 Recall:  0.1125 Precision:  0.42857142857142855 F1:  0.1782178217821782
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced Accuracy:  0.953757225433526 MSE:  0.046242774566473986 UAR:  0.5297348484848485 Recall:  0.0625 Precision:  0.5 F1:  0.1111111111111111
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced Accuracy:  0.954335260115607 MSE:  0.04566473988439306 UAR:  0.5181439393939394 Recall:  0.0375 Precision:  0.6 F1:  0.07058823529411765
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced Accuracy:  0.9531791907514451 MSE:  0.04682080924855491 UAR:  0.505

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [230]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [231]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [232]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced_best Accuracy:  0.8549132947976879 MSE:  0.14508670520231215 UAR:  0.6979545454545455 Recall:  0.525 Precision:  0.16470588235294117 F1:  0.2507462686567164
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced_best Accuracy:  0.9260115606936417 MSE:  0.07398843930635839 UAR:  0.6460227272727272 Recall:  0.3375 Precision:  0.2647058823529412 F1:  0.29670329670329665
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced_best Accuracy:  0.946242774566474 MSE:  0.05375722543352601 UAR:  0.5852651515151515 Recall:  0.1875 Precision:  0.3488372093023256 F1:  0.2439024390243903
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced_best Accuracy:  0.954335260115607 MSE:  0.04566473988439306 UAR:  0.5716666666666667 Recall:  0.15 Precision:  0.5217391304347826 F1:  0.23300970873786406
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced_best

### STAT

In [91]:
stat_func = compute_descriptor
stat_name = "STAT"

#### Traditional split

In [92]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
val_pickle = FEATURES_DIR + 'val_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)
filename2features_val = load_features(val_pickle)

In [93]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))
print("Val: ", len(filename2features_val))

Train:  5482
Test:  1723
Val:  1720


In [94]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))
print("val:")
print_classes(get_labels(filename2features_val))

train:
0 5234/5482: 95.4761036118205%
1 248/5482: 4.523896388179496%
test:
0 1638/1723: 95.06674405107371%
1 85/1723: 4.933255948926291%
val:
0 1537/1720: 89.36046511627907%
1 183/1720: 10.63953488372093%


##### Single attention

In [95]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

mobilenet_7.h5_DAiSEE_2_STAT_single_attention_traditional


In [96]:
CONCATENATE_STAT = True

In [97]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 3072) (5482,)
(1723, 128, 3072) (1723,)
(1720, 128, 3072) (1720,)


In [98]:
(unique, counts) = np.unique(y_subsample_train, return_counts=True)
num_classes=len(unique)
cw=1/counts
cw/=cw.min()
class_weights = {i:cwi for i,cwi in zip(unique,cw)}
print(counts, class_weights, num_classes, unique)

[5234  248] {0: 1.0, 1: 21.10483870967742} 2 [0 1]


In [99]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)
#save_best_model = SaveBestModelByUAR(X_subsample_val, y_subsample_val)

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3072)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              3073      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [100]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val), class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 5s 78ms/step - loss: 1.3927 - acc: 0.6441 - auc: 0.5977 - binary_accuracy: 0.6441 - recall_4: 0.5444 - precision_4: 0.0684 - val_loss: 0.8600 - val_acc: 0.3843 - val_auc: 0.6910 - val_binary_accuracy: 0.3843 - val_recall_4: 0.8907 - val_precision_4: 0.1356
Epoch 2/20
43/43 [==============================] - 2s 35ms/step - loss: 1.1498 - acc: 0.6946 - auc: 0.7366 - binary_accuracy: 0.6946 - recall_4: 0.6452 - precision_4: 0.0916 - val_loss: 0.8465 - val_acc: 0.4826 - val_auc: 0.6889 - val_binary_accuracy: 0.4826 - val_recall_4: 0.8306 - val_precision_4: 0.1503
Epoch 3/20
43/43 [==============================] - 2s 42ms/step - loss: 1.0800 - acc: 0.7588 - auc: 0.7820 - binary_accuracy: 0.7588 - recall_4: 0.6613 - precision_4: 0.1170 - val_loss: 0.5910 - val_acc: 0.7215 - val_auc: 0.6926 - val_binary_accuracy: 0.7215 - val_recall_4: 0.5246 - val_precision_4: 0.1967
Epoch 4/20
43/43 [==============================] - 2s 42ms/step - loss: 

In [101]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3108480/1443316788.py:29: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [102]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name, from_int=True)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_single_attention_traditional Accuracy:  0.3691236215902496 MSE:  0.6308763784097504 UAR:  0.5845363786540256 Recall:  0.8235294117647058 Precision:  0.06129597197898424 F1:  0.11409942950285247
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_single_attention_traditional Accuracy:  0.5037724898432966 MSE:  0.4962275101567034 UAR:  0.5772750125691302 Recall:  0.6588235294117647 Precision:  0.06349206349206349 F1:  0.11582213029989658
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_single_attention_traditional Accuracy:  0.6157864190365642 MSE:  0.3842135809634359 UAR:  0.6027257056668822 Recall:  0.5882352941176471 Precision:  0.07385524372230429 F1:  0.13123359580052496
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_single_attention_traditional Accuracy:  0.7208357515960534 MSE:  0.2791642484039466 UAR:  0.6300904977375565 Recall:  0.5294117647058824 Precision:  0.09259259259259259 F1:  0.1576182136602452
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_single_att

In [103]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [104]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3108480/1443316788.py:29: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [105]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name, from_int=True)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_single_attention_traditional_best Accuracy:  0.387695879280325 MSE:  0.612304120719675 UAR:  0.5831501831501832 Recall:  0.8 Precision:  0.06148282097649186 F1:  0.11418975650713686
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_single_attention_traditional_best Accuracy:  0.5972141613464886 MSE:  0.4027858386535113 UAR:  0.6096890038066509 Recall:  0.6235294117647059 Precision:  0.07412587412587412 F1:  0.1325
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_single_attention_traditional_best Accuracy:  0.7446314567614626 MSE:  0.2553685432385374 UAR:  0.6202973497091144 Recall:  0.4823529411764706 Precision:  0.09382151029748284 F1:  0.15708812260536398
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_single_attention_traditional_best Accuracy:  0.8270458502611724 MSE:  0.17295414973882764 UAR:  0.6301802772391008 Recall:  0.4117647058823529 Precision:  0.12367491166077739 F1:  0.19021739130434784
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_single_attention_t

##### Self attention

In [198]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional


In [199]:
CONCATENATE_STAT = True

In [200]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 2560) (5482,)
(1723, 128, 2560) (1723,)
(1720, 128, 2560) (1720,)


In [201]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_13"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention_6 (Attention)     (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']        

In [202]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 7s 127ms/step - loss: 0.3520 - acc: 0.9435 - auc: 0.5372 - binary_accuracy: 0.9435 - recall_1: 0.0121 - precision_1: 0.0441 - val_loss: 0.3655 - val_acc: 0.8936 - val_auc: 0.6054 - val_binary_accuracy: 0.8936 - val_recall_1: 0.0000e+00 - val_precision_1: 0.0000e+00
Epoch 2/20
43/43 [==============================] - 4s 89ms/step - loss: 0.1676 - acc: 0.9544 - auc: 0.7332 - binary_accuracy: 0.9544 - recall_1: 0.0081 - precision_1: 0.3333 - val_loss: 0.4156 - val_acc: 0.8936 - val_auc: 0.6291 - val_binary_accuracy: 0.8936 - val_recall_1: 0.0000e+00 - val_precision_1: 0.0000e+00
Epoch 3/20
43/43 [==============================] - 4s 89ms/step - loss: 0.1589 - acc: 0.9533 - auc: 0.7800 - binary_accuracy: 0.9533 - recall_1: 0.0081 - precision_1: 0.1667 - val_loss: 0.3692 - val_acc: 0.8907 - val_auc: 0.6405 - val_binary_accuracy: 0.8907 - val_recall_1: 0.0328 - val_precision_1: 0.3529
Epoch 4/20
43/43 [==============================] - 4s 9

In [203]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [204]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional Accuracy:  0.8415554265815438 MSE:  0.1584445734184562 UAR:  0.6043489190548015 Recall:  0.3411764705882353 Precision:  0.11788617886178862 F1:  0.17522658610271902
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional Accuracy:  0.930354033662217 MSE:  0.06964596633778293 UAR:  0.5450872656755009 Recall:  0.11764705882352941 Precision:  0.18181818181818182 F1:  0.14285714285714285
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional Accuracy:  0.9437028438769588 MSE:  0.05629715612304121 UAR:  0.5409538174244057 Recall:  0.09411764705882353 Precision:  0.2857142857142857 F1:  0.1415929203539823
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional Accuracy:  0.947185142193848 MSE:  0.05281485780615206 UAR:  0.526054011348129 Recall:  0.058823529411764705 Precision:  0.3125 F1:  0.09900990099009901
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE

In [205]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [206]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [207]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional_best Accuracy:  0.8340104468949506 MSE:  0.16598955310504934 UAR:  0.6282661782661783 Recall:  0.4 Precision:  0.12639405204460966 F1:  0.192090395480226
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional_best Accuracy:  0.9297736506094022 MSE:  0.0702263493905978 UAR:  0.5949759390935861 Recall:  0.2235294117647059 Precision:  0.25675675675675674 F1:  0.2389937106918239
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional_best Accuracy:  0.9477655252466628 MSE:  0.0522344747533372 UAR:  0.5653989801048624 Recall:  0.1411764705882353 Precision:  0.41379310344827586 F1:  0.21052631578947367
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional_best Accuracy:  0.9524085896691816 MSE:  0.04759141033081834 UAR:  0.5566867772750126 Recall:  0.11764705882352941 Precision:  0.5882352941176471 F1:  0.19607843137254902
Metric_name:  enet_b0_8_best

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### Balanced split

In [106]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [107]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  5482
Test:  1723


In [108]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 5234/5482: 95.4761036118205%
1 248/5482: 4.523896388179496%
val:
0 1638/1723: 95.06674405107371%
1 85/1723: 4.933255948926291%


In [109]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.76)

(5482, 2048) (5482,)
(1723, 2048) (1723,)
(7205, 2048) (7205,)
[0 1] [6872  333]
7205
5475
1730


In [110]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 5222/5475: 95.37899543378995%
1 253/5475: 4.621004566210046%
val:
0 1650/1730: 95.3757225433526%
1 80/1730: 4.624277456647399%


##### Single attention

In [111]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

mobilenet_7.h5_DAiSEE_2_STAT_single_attention_balanced


In [112]:
CONCATENATE_STAT = True

In [113]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5475, 128, 3072) (5475,)
(1730, 128, 3072) (1730,)


In [114]:
(unique, counts) = np.unique(y_subsample_train, return_counts=True)
num_classes=len(unique)
cw=1/counts
cw/=cw.min()
class_weights = {i:cwi for i,cwi in zip(unique,cw)}
print(counts, class_weights, num_classes, unique)

[5222  253] {0: 1.0, 1: 20.640316205533598} 2 [0 1]


In [115]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)
#save_best_model = SaveBestModelByUAR(X_subsample_test, y_subsample_test)

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3072)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              3073      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [116]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test), class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 5s 84ms/step - loss: 1.3211 - acc: 0.6230 - auc: 0.6326 - binary_accuracy: 0.6230 - recall_5: 0.5692 - precision_5: 0.0686 - val_loss: 0.4929 - val_acc: 0.8549 - val_auc: 0.7493 - val_binary_accuracy: 0.8549 - val_recall_5: 0.4750 - val_precision_5: 0.1538
Epoch 2/20
43/43 [==============================] - 2s 36ms/step - loss: 1.1427 - acc: 0.7366 - auc: 0.7472 - binary_accuracy: 0.7366 - recall_5: 0.6166 - precision_5: 0.1039 - val_loss: 0.8909 - val_acc: 0.3908 - val_auc: 0.7612 - val_binary_accuracy: 0.3908 - val_recall_5: 0.9125 - val_precision_5: 0.0652
Epoch 3/20
43/43 [==============================] - 2s 36ms/step - loss: 1.0640 - acc: 0.7251 - auc: 0.7860 - binary_accuracy: 0.7251 - recall_5: 0.7115 - precision_5: 0.1117 - val_loss: 0.5565 - val_acc: 0.7428 - val_auc: 0.7623 - val_binary_accuracy: 0.7428 - val_recall_5: 0.7000 - val_precision_5: 0.1174
Epoch 4/20
43/43 [==============================] - 2s 36ms/step - loss: 

In [117]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3108480/1443316788.py:29: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [118]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name, from_int=True)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_single_attention_balanced Accuracy:  0.3861271676300578 MSE:  0.6138728323699422 UAR:  0.5889772727272727 Recall:  0.8125 Precision:  0.05845323741007194 F1:  0.10906040268456375
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_single_attention_balanced Accuracy:  0.5173410404624278 MSE:  0.48265895953757226 UAR:  0.6399242424242424 Recall:  0.775 Precision:  0.07053469852104664 F1:  0.12930135557872782
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_single_attention_balanced Accuracy:  0.6086705202312138 MSE:  0.3913294797687861 UAR:  0.6521212121212121 Recall:  0.7 Precision:  0.07898448519040903 F1:  0.14195183776932827
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_single_attention_balanced Accuracy:  0.6826589595375723 MSE:  0.3173410404624277 UAR:  0.6671212121212121 Recall:  0.65 Precision:  0.09075043630017451 F1:  0.1592649310872894
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_single_attention_balanced Accuracy:  0.7450867052023121 MSE:  0.25491329479

In [119]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [120]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3108480/1443316788.py:29: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [121]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name, from_int=True)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_single_attention_balanced_best Accuracy:  0.4416184971098266 MSE:  0.5583815028901734 UAR:  0.6240151515151515 Recall:  0.825 Precision:  0.06483300589390963 F1:  0.12021857923497267
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_single_attention_balanced_best Accuracy:  0.6098265895953757 MSE:  0.3901734104046243 UAR:  0.6765151515151515 Recall:  0.75 Precision:  0.08391608391608392 F1:  0.1509433962264151
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_single_attention_balanced_best Accuracy:  0.7127167630057804 MSE:  0.28728323699421965 UAR:  0.6828787878787879 Recall:  0.65 Precision:  0.09980806142034548 F1:  0.173044925124792
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_single_attention_balanced_best Accuracy:  0.7803468208092486 MSE:  0.21965317919075145 UAR:  0.6885984848484848 Recall:  0.5875 Precision:  0.11928934010152284 F1:  0.19831223628691982
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_single_attention_balanced_best Accuracy:  0.83236994219

##### Self attention

In [223]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced


In [224]:
CONCATENATE_STAT = True

In [225]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5475, 128, 2560) (5475,)
(1730, 128, 2560) (1730,)


In [226]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_15"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention_7 (Attention)     (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']        

In [227]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 7s 125ms/step - loss: 0.3764 - acc: 0.9412 - auc: 0.5374 - binary_accuracy: 0.9412 - recall_3: 0.0277 - precision_3: 0.0843 - val_loss: 0.1875 - val_acc: 0.9526 - val_auc: 0.6787 - val_binary_accuracy: 0.9526 - val_recall_3: 0.0125 - val_precision_3: 0.2500
Epoch 2/20
43/43 [==============================] - 4s 89ms/step - loss: 0.1737 - acc: 0.9540 - auc: 0.7014 - binary_accuracy: 0.9540 - recall_3: 0.0514 - precision_3: 0.5200 - val_loss: 0.1701 - val_acc: 0.9526 - val_auc: 0.7431 - val_binary_accuracy: 0.9526 - val_recall_3: 0.0250 - val_precision_3: 0.3333
Epoch 3/20
43/43 [==============================] - 4s 88ms/step - loss: 0.1627 - acc: 0.9536 - auc: 0.7562 - binary_accuracy: 0.9536 - recall_3: 0.0435 - precision_3: 0.4783 - val_loss: 0.1690 - val_acc: 0.9526 - val_auc: 0.7421 - val_binary_accuracy: 0.9526 - val_recall_3: 0.0000e+00 - val_precision_3: 0.0000e+00
Epoch 4/20
43/43 [==============================] - 4s 88ms/step

In [228]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [229]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced Accuracy:  0.9445086705202312 MSE:  0.055491329479768786 UAR:  0.5903030303030303 Recall:  0.2 Precision:  0.3333333333333333 F1:  0.25
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced Accuracy:  0.9520231213872833 MSE:  0.04797687861271676 UAR:  0.5526136363636364 Recall:  0.1125 Precision:  0.42857142857142855 F1:  0.1782178217821782
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced Accuracy:  0.953757225433526 MSE:  0.046242774566473986 UAR:  0.5297348484848485 Recall:  0.0625 Precision:  0.5 F1:  0.1111111111111111
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced Accuracy:  0.954335260115607 MSE:  0.04566473988439306 UAR:  0.5181439393939394 Recall:  0.0375 Precision:  0.6 F1:  0.07058823529411765
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced Accuracy:  0.9531791907514451 MSE:  0.04682080924855491 UAR:  0.505

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [230]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [231]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [232]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced_best Accuracy:  0.8549132947976879 MSE:  0.14508670520231215 UAR:  0.6979545454545455 Recall:  0.525 Precision:  0.16470588235294117 F1:  0.2507462686567164
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced_best Accuracy:  0.9260115606936417 MSE:  0.07398843930635839 UAR:  0.6460227272727272 Recall:  0.3375 Precision:  0.2647058823529412 F1:  0.29670329670329665
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced_best Accuracy:  0.946242774566474 MSE:  0.05375722543352601 UAR:  0.5852651515151515 Recall:  0.1875 Precision:  0.3488372093023256 F1:  0.2439024390243903
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced_best Accuracy:  0.954335260115607 MSE:  0.04566473988439306 UAR:  0.5716666666666667 Recall:  0.15 Precision:  0.5217391304347826 F1:  0.23300970873786406
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced_best